In [2]:
# --- Quick sanity check for local summary + keys ---
from pathlib import Path
import os, json, re

tic = 37749396
print("cwd:", os.getcwd())
p = Path(f"results/TIC{tic}_download_clean_summary.json")
print("summary path:", p, "| exists:", p.exists())

if p.exists():
    data = json.loads(p.read_text())
    print("JSON keys:", sorted(data.keys()))
    for key in ("sectors_used","pdcsap_sectors","sectors","sector_list"):
        if key in data:
            print(f"{key} =", data[key])
else:
    print("No local summary JSON found.")

cwd: /Users/kobi.weitzman/Documents/tess-ephem
summary path: results/TIC37749396_download_clean_summary.json | exists: True
JSON keys: ['cdpp1h_flat_ppm', 'flatten_window_days', 'n_points_flat', 'n_points_raw', 'n_sectors_ffi', 'n_sectors_pdcsap', 'notes', 'rms_flat_ppm', 'rms_raw_ppm', 'sectors_ffi', 'sectors_pdcsap', 'target_tic', 'target_toi']


In [4]:
# --- Target B: pull sectors from existing download summary ---
from pathlib import Path
import json

TARGET_TIC = 37749396
TARGET_TOI = "Target B"  # replace with the real TOI label if you have it

data = json.loads(Path(f"results/TIC{TARGET_TIC}_download_clean_summary.json").read_text())

SECTORS_PDCSAP = sorted({int(s) for s in (data.get("sectors_pdcsap") or [])})
SECTORS_FFI    = sorted({int(s) for s in (data.get("sectors_ffi") or [])})
SECTORS        = SECTORS_PDCSAP if SECTORS_PDCSAP else SECTORS_FFI

print(f"TIC {TARGET_TIC} — PDCSAP sectors: {SECTORS_PDCSAP} | FFI sectors: {SECTORS_FFI}")
print(f"Using SECTORS = {SECTORS}")

# Fallback: if still empty, try a quick MAST query (only if needed)
if not SECTORS:
    import lightkurve as lk
    print("No sectors in summary; trying MAST quickly …")
    sr = lk.search_lightcurve(f"TIC {TARGET_TIC}", mission="TESS")
    SECTORS = sorted({int(r.sector) for r in sr if getattr(r, "sector", None) is not None})
    print("MAST sectors:", SECTORS)

TIC 37749396 — PDCSAP sectors: [3, 42, 70] | FFI sectors: []
Using SECTORS = [3, 42, 70]


In [6]:
# --- Lightkurve loader for PDCSAP by sector (with a safe fallback) ---
import warnings
import lightkurve as lk

# Quiet some noisy warnings across versions
try:
    from lightkurve.utils import LightkurveWarning
except Exception:
    class LightkurveWarning(Warning): ...
warnings.filterwarnings("ignore", category=LightkurveWarning)

def load_pdcsap_sector(tic, sector):
    """
    Return a LightCurve with PDCSAP flux for a given TIC and TESS sector.
    Tries the modern search_lightcurve path first; falls back to LightCurveFile.
    """
    # 1) Preferred path: search_lightcurve(..., author='SPOC')
    try:
        sr = lk.search_lightcurve(f"TIC {tic}", mission="TESS", author="SPOC", sector=sector)
        if len(sr) == 0:
            raise RuntimeError("No SPOC LC via search_lightcurve")
        # flux_column works on newer Lightkurve; if not, .download() defaults to PDCSAP for SPOC
        try:
            lc = sr.download(flux_column="pdcsap_flux")
        except Exception:
            lc = sr.download()
        return lc.remove_nans()
    except Exception:
        # 2) Fallback: LightCurveFile -> PDCSAP_FLUX attribute
        sr2 = lk.search_lightcurvefile(f"TIC {tic}", mission="TESS", sector=sector)
        lcf = sr2.download()
        try:
            lc = lcf.PDCSAP_FLUX
        except Exception:
            # last resort (shouldn't happen for SPOC sectors): use SAP
            lc = lcf.SAP_FLUX
        return lc.remove_nans()

In [7]:
for s in SECTORS:
    lc = load_pdcsap_sector(TARGET_TIC, s)
    n = len(getattr(lc.time, "value", lc.time))
    print(f"TIC {TARGET_TIC} S{s}: loaded PDCSAP, N={n}")

TIC 37749396 S3: loaded PDCSAP, N=12978
TIC 37749396 S42: loaded PDCSAP, N=11473
TIC 37749396 S70: loaded PDCSAP, N=86180


In [11]:
# === BLS→TLS helpers & config (paste once per fresh notebook) =================
import os, csv, time, warnings
import numpy as np
import matplotlib.pyplot as plt

import lightkurve as lk
from astropy.timeseries import BoxLeastSquares
from transitleastsquares import transitleastsquares

# Quiet Lightkurve warnings across versions
try:
    from lightkurve.utils import LightkurveWarning
except Exception:
    class LightkurveWarning(Warning): ...
warnings.filterwarnings("ignore", category=LightkurveWarning)

# ---- Config (keep modest to stay fast) ----
BLS_PERIOD_MIN = 0.5
BLS_PERIOD_MAX = 50.0
BLS_NPER       = 5000                 # BLS grid size
BLS_DURATIONS_HR = np.linspace(0.5, 3.0, 18)  # candidate durations for BLS

TLS_WINDOW_FRAC  = 0.01               # ±1% TLS window around each BLS peak
TLS_THREADS      = max(1, (os.cpu_count() or 1))  # use all cores
TLS_MIN_TRANSITS = 2                  # be a bit strict to speed up

FIGDIR = "figures"; RESDIR = "results"
os.makedirs(FIGDIR, exist_ok=True)
os.makedirs(RESDIR, exist_ok=True)

# If you know star params, set them here; 1.0/1.0 is fine if unknown.
R_STAR = 1.0   # R_sun
M_STAR = 1.0   # M_sun

# ---- Small utility helpers ----
def lc_to_arrays(lc):
    """Return (t,f) float arrays, normalized by median; robust to masked arrays/NaNs."""
    t = getattr(lc.time, "value", lc.time)
    f = getattr(lc.flux, "value", lc.flux)
    t = np.asarray(t, dtype=float)
    f = np.asarray(f, dtype=float)
    if np.ma.isMaskedArray(t): t = t.filled(np.nan)
    if np.ma.isMaskedArray(f): f = f.filled(np.nan)
    m = np.isfinite(t) & np.isfinite(f)
    f_med = np.nanmedian(f[m]) if np.any(m) else 1.0
    if not np.isfinite(f_med) or f_med == 0: f_med = 1.0
    return t[m], (f[m]/f_med)

def unique_peaks(periods, power, k=3, tol_frac=0.01):
    """Pick top-k unique periods (avoid near-duplicates within tol_frac)."""
    idx = np.argsort(power)[::-1]
    picks = []
    for i in idx:
        p = float(periods[i])
        if all(abs(p - q)/q > tol_frac for q in picks if q != 0):
            picks.append(p)
        if len(picks) == k:
            break
    return picks

def plot_periodogram(x, y, xlabel, title, outpng):
    plt.figure(figsize=(8,4), dpi=140)
    plt.plot(x, y, lw=1)
    plt.xlabel(xlabel); plt.ylabel("Power"); plt.title(title)
    plt.tight_layout(); plt.savefig(outpng); plt.close()

def fold_and_plot(t, f, period, t0, title, outpng, nbins=200):
    phase = ((t - t0 + 0.5*period) % period) / period - 0.5
    order = np.argsort(phase); phase, f = phase[order], f[order]
    bins = np.linspace(-0.5, 0.5, nbins+1)
    which = np.digitize(phase, bins) - 1
    yb = np.array([np.nanmean(f[which==i]) if np.any(which==i) else np.nan for i in range(nbins)])
    xb = 0.5*(bins[:-1]+bins[1:])
    plt.figure(figsize=(8,4), dpi=140)
    plt.plot(phase, f, ".", ms=2, alpha=0.35)
    plt.plot(xb, yb, "-", lw=1.5)
    plt.axvline(0.0, color="k", lw=1, alpha=0.3)
    plt.xlabel("Phase (cycles)"); plt.ylabel("Relative flux"); plt.title(title)
    plt.tight_layout(); plt.savefig(outpng); plt.close()

def append_csv(path, rows, header=None):
    new = not os.path.exists(path)
    with open(path, "a", newline="") as f:
        w = csv.writer(f)
        if new and header: w.writerow(header)
        for r in rows: w.writerow(r)

def bls_power_safe(t, f, periods, durations):
    """Run BLS, trying objective='snr' first; if not supported, fall back."""
    bls = BoxLeastSquares(t, f)
    try:
        res = bls.power(periods, durations, objective="snr")
    except TypeError:
        res = bls.power(periods, durations)
    return res

def tls_narrow(t, f, p_center, frac=TLS_WINDOW_FRAC, nthreads=TLS_THREADS, nmin=TLS_MIN_TRANSITS):
    """
    TLS around a single candidate period (±frac). Returns (period, SDE, T0, res).
    NOTE: R_star and M_star must be passed to .power(...), not the constructor.
    """
    tls = transitleastsquares(t, f)
    pmin = p_center*(1-frac)
    pmax = p_center*(1+frac)
    if not np.isfinite(pmin) or not np.isfinite(pmax) or pmin <= 0 or pmax <= pmin:
        pmin, pmax = max(0.5, p_center*0.98), p_center*1.02

    # Try with all threads; if TLS complains, fall back to 1 thread.
    try:
        res = tls.power(
            period_min=pmin, period_max=pmax,
            show_progress_bar=False,
            use_threads=int(nthreads),
            n_transits_min=int(nmin),
            R_star=R_STAR, M_star=M_STAR
        )
    except ValueError as e:
        if "use_threads" in str(e):
            res = tls.power(
                period_min=pmin, period_max=pmax,
                show_progress_bar=False,
                use_threads=1,
                n_transits_min=int(nmin),
                R_star=R_STAR, M_star=M_STAR
            )
        else:
            res = tls.power(
                period_min=p_center*(1-2*frac), period_max=p_center*(1+2*frac),
                show_progress_bar=False,
                use_threads=int(nthreads),
                n_transits_min=int(nmin),
                R_star=R_STAR, M_star=M_STAR
            )
    return float(res.period), float(res.SDE), float(res.T0), res

def run_block(label, t, f, target_tic, target_toi):
    """Run BLS wide (clamped to data span), then TLS narrow for top-3; save artifacts."""
    print(f"\n[{label}] points={t.size}  threads={TLS_THREADS}")

    # ---- BLS (wide, but clamp to data span for per-sector speed) ----
    t0 = time.time()
    span = float(np.nanmax(t) - np.nanmin(t))
    bls_pmax = min(BLS_PERIOD_MAX, max(BLS_PERIOD_MIN*1.2, 0.90*span))
    periods   = np.linspace(BLS_PERIOD_MIN, bls_pmax, BLS_NPER)
    durations = BLS_DURATIONS_HR / 24.0
    bls_res   = bls_power_safe(t, f, periods, durations)
    print(f"[{label}] BLS done in {time.time()-t0:.1f}s (Pmax used={bls_pmax:.2f} d)")

    plot_periodogram(
        bls_res.period, bls_res.power,
        "Period (days)", f"{target_toi} ({label}) — BLS periodogram",
        f"{FIGDIR}/TIC{target_tic}_{label}_BLS_periodogram.png"
    )

    bls_topP = unique_peaks(bls_res.period, bls_res.power, k=3, tol_frac=0.01)
    append_csv(
        f"{RESDIR}/TIC{target_tic}_{label}_BLS_top3.csv",
        [[target_tic, target_toi, label, float(p),
          float(bls_res.power[np.argmin(abs(bls_res.period-p))])] for p in bls_topP],
        header=["tic","toi","label","period_days","power"]
    )

    # ---- TLS (narrow around each BLS peak) ----
    tls_rows = []
    for p in bls_topP:
        print(f"[{label}] TLS refine around {p:.5f} d (±{TLS_WINDOW_FRAC*100:.1f}%) …")
        t1 = time.time()
        p_best, sde, t0_best, res = tls_narrow(t, f, p)
        print(f"[{label}]   TLS best P={p_best:.6f} d, SDE={sde:.2f} (took {time.time()-t1:.1f}s)")
        tls_rows.append([target_tic, target_toi, label, p_best, sde, t0_best])

        # Save TLS periodogram and fold for this candidate
        plot_periodogram(
            res.periods, res.power,
            "Period (days)", f"{target_toi} ({label}) — TLS @ {p:.5f}±{TLS_WINDOW_FRAC*100:.1f}%",
            f"{FIGDIR}/TIC{target_tic}_{label}_TLS_periodogram_around_{p:.5f}.png"
        )
        fold_and_plot(
            t, f, p_best, t0_best,
            f"{target_toi} ({label}) — TLS fold @ P={p_best:.5f} d",
            f"{FIGDIR}/TIC{target_tic}_{label}_TLS_fold_P{p_best:.5f}.png"
        )

    append_csv(
        f"{RESDIR}/TIC{target_tic}_{label}_TLS_top3.csv",
        tls_rows, header=["tic","toi","label","period_days","SDE","T0_BTJD"]
    )
# ============================================================================ 

In [13]:
# --- Run per-sector and stitched on Target B (fixed calls) ---
import numpy as np

assert SECTORS, "No sectors to run on — SECTORS is empty."

t_all_list, f_all_list = [], []
for s in SECTORS:
    lc = load_pdcsap_sector(TARGET_TIC, s).normalize()
    t, f = lc_to_arrays(lc)
    print(f"{TARGET_TOI} — S{s}: N={t.size}")
    # pass TIC/TOI into run_block:
    run_block(f"S{s}", t, f, TARGET_TIC, TARGET_TOI)
    t_all_list.append(t); f_all_list.append(f)

# stitched
t_all = np.concatenate(t_all_list); f_all = np.concatenate(f_all_list)
order = np.argsort(t_all); t_all, f_all = t_all[order], f_all[order]
run_block("stitched", t_all, f_all, TARGET_TIC, TARGET_TOI)

print("\nDone for Target B.")

Target B — S3: N=12978

[S3] points=12978  threads=8
[S3] BLS done in 1.7s (Pmax used=18.25 d)
[S3] TLS refine around 6.73492 d (±1.0%) …
Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 36 durations
Searching 12978 data points, 1745 periods from 0.602 to 10.139 days
Using all 8 CPU threads


/Users/kobi.weitzman/miniforge3/envs/tess-ephem/lib/python3.10/site-packages/transitleastsquares/grid.py:149: UserWarning: period_grid defaults to R_star=1 and M_star=1 as given density yielded grid with too few values
  warnings.warn(


Searching for best T0 for period 2.62263 days
[S3]   TLS best P=2.622627 d, SDE=7.68 (took 7.7s)
[S3] TLS refine around 6.56093 d (±1.0%) …
Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 36 durations
Searching 12978 data points, 1745 periods from 0.602 to 10.139 days
Using all 8 CPU threads


/Users/kobi.weitzman/miniforge3/envs/tess-ephem/lib/python3.10/site-packages/transitleastsquares/grid.py:149: UserWarning: period_grid defaults to R_star=1 and M_star=1 as given density yielded grid with too few values
  warnings.warn(


Searching for best T0 for period 2.62263 days
[S3]   TLS best P=2.622627 d, SDE=7.68 (took 6.8s)
[S3] TLS refine around 18.24963 d (±1.0%) …
Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 36 durations
Searching 12978 data points, 1745 periods from 0.602 to 10.139 days
Using all 8 CPU threads
Searching for best T0 for period 2.62263 days
[S3]   TLS best P=2.622627 d, SDE=7.68 (took 6.3s)
Target B — S42: N=11473

[S42] points=11473  threads=8
[S42] BLS done in 1.8s (Pmax used=21.03 d)
[S42] TLS refine around 16.55722 d (±1.0%) …
Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 37 durations
Searching 11473 data points, 2070 periods from 0.601 to 11.683 days
Using all 8 CPU threads


...

[stitched]   TLS best P=13.475725 d, SDE=6.63 (took 28.4s)
[stitched] TLS refine around 15.67974 d (±1.0%) …
Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 29 durations
Searching 110631 data points, 585 periods from 15.523 to 15.836 days
Using all 8 CPU threads
Searching for best T0 for period 15.83220 days


/Users/kobi.weitzman/miniforge3/envs/tess-ephem/lib/python3.10/site-packages/transitleastsquares/main.py:411: UserWarning: 115 of 116 transits without data. The true period may be twice the given period.
  warnings.warn(text)


[stitched]   TLS best P=15.832202 d, SDE=3.72 (took 24.7s)
[stitched] TLS refine around 10.68914 d (±1.0%) …
Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 29 durations
Searching 110631 data points, 665 periods from 10.583 to 10.796 days
Using all 8 CPU threads
Searching for best T0 for period 10.78088 days


/Users/kobi.weitzman/miniforge3/envs/tess-ephem/lib/python3.10/site-packages/transitleastsquares/main.py:411: UserWarning: 168 of 171 transits without data. The true period may be twice the given period.
  warnings.warn(text)


[stitched]   TLS best P=10.780883 d, SDE=3.33 (took 31.1s)

Done for Target B.


In [17]:
def plot_periodogram(x, y, xlabel, title, outpng,
                     clip_q=PLOT_CLIP_Q, smooth_win=SMOOTH_WIN, mark_artifacts=True):
    x = np.asarray(x, float); y = np.asarray(y, float)
    yclip = np.copy(y)
    ymax = np.nanpercentile(y, clip_q) if np.isfinite(y).any() else np.nan
    if np.isfinite(ymax):
        yclip = np.minimum(yclip, ymax)

    plt.figure(figsize=(8, 4), dpi=140)
    plt.plot(x, yclip, lw=0.7, color="0.6", label="power (clipped)")
    ys = smooth1d(yclip, smooth_win)
    if ys is not None:                         # ← fixed condition
        plt.plot(x, ys, lw=1.5, label=f"smoothed (w={smooth_win})")

    if mark_artifacts:
        for p in ARTIFACT_PERIODS:
            if np.nanmin(x) < p < np.nanmax(x):
                plt.axvline(p, color="k", lw=1, alpha=0.15)

    plt.xlabel(xlabel); plt.ylabel("Power"); plt.title(title)
    plt.legend(loc="upper right", fontsize=8, framealpha=0.3)
    plt.tight_layout(); plt.savefig(outpng); plt.close()

In [18]:
# ==== Run Target B (uses the new cleaning + prettier plots) ====
import os, numpy as np

# Config
TARGET_TOI = "Target B"
TARGET_TIC = 37749396
SECTORS    = [3, 42, 70]       # from your earlier discovery

# Ensure dirs/constants exist (safe defaults if missing)
FIGDIR = "figures"; RESDIR = "results"
os.makedirs(FIGDIR, exist_ok=True); os.makedirs(RESDIR, exist_ok=True)

try: TLS_THREADS
except NameError:
    TLS_THREADS = max(1, (os.cpu_count() or 1))
try: TLS_MIN_TRANSITS
except NameError:
    TLS_MIN_TRANSITS = 3        # stricter for single sectors
try: BLS_PERIOD_MIN
except NameError:
    BLS_PERIOD_MIN = 0.5
try: BLS_PERIOD_MAX
except NameError:
    BLS_PERIOD_MAX = 50.0
try: BLS_NPER
except NameError:
    BLS_NPER = 5000
try: BLS_DURATIONS_HR
except NameError:
    BLS_DURATIONS_HR = np.linspace(0.5, 3.0, 18)
try: TLS_WINDOW_FRAC
except NameError:
    TLS_WINDOW_FRAC = 0.01

# Make sure required helpers exist (define minimal loader if needed)
try:
    load_pdcsap_sector
except NameError:
    import lightkurve as lk
    def load_pdcsap_sector(tic, sector):
        sr = lk.search_lightcurve(f"TIC {tic}", mission="TESS", sector=sector, author="SPOC")
        if len(sr) == 0: raise RuntimeError("No SPOC PDCSAP LC")
        return sr.download().remove_nans()

# Run per-sector with new cleaner
t_all_list, f_all_list = [], []
for s in SECTORS:
    lc = load_pdcsap_sector(TARGET_TIC, s)           # raw PDCSAP
    t, f = prep_arrays_for_search(lc)                # << clean + detrend
    print(f"{TARGET_TOI} — S{s}: N={t.size}")
    run_block(f"S{s}", t, f, TARGET_TIC, TARGET_TOI) # saves figs/CSVs
    t_all_list.append(t); f_all_list.append(f)

# Stitched (combined)
t_all = np.concatenate(t_all_list); f_all = np.concatenate(f_all_list)
order = np.argsort(t_all); t_all, f_all = t_all[order], f_all[order]
run_block("stitched", t_all, f_all, TARGET_TIC, TARGET_TOI)

print("\nDone for Target B with robust prep. Artifacts saved in figures/ and results/.")

Target B — S3: N=12977

[S3] points=12977  threads=8
[S3] BLS done in 1.1s (Pmax used=9.63 d)
[S3] with nmin=3, periods ≲ 9.63 d have ≥3 transits
[S3] TLS refine around 6.73640 d (±1.0%) …
Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 36 durations
Searching 12977 data points, 1745 periods from 0.602 to 10.139 days
Using all 8 CPU threads


/Users/kobi.weitzman/miniforge3/envs/tess-ephem/lib/python3.10/site-packages/transitleastsquares/grid.py:149: UserWarning: period_grid defaults to R_star=1 and M_star=1 as given density yielded grid with too few values
  warnings.warn(


Searching for best T0 for period 6.73332 days
[S3]   TLS best P=6.733319 d, SDE=7.24 (took 8.9s)
[S3] TLS refine around 9.14036 d (±1.0%) …
Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 36 durations
Searching 12977 data points, 1745 periods from 0.602 to 10.139 days
Using all 8 CPU threads
Searching for best T0 for period 6.73332 days
[S3]   TLS best P=6.733319 d, SDE=7.24 (took 7.5s)
[S3] TLS refine around 9.63175 d (±1.0%) …
Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 36 durations
Searching 12977 data points, 1745 periods from 0.602 to 10.139 days
Using all 8 CPU threads
Searching for best T0 for period 6.73332 days
[S3]   TLS best P=6.733319 d, SDE=7.24 (took 7.2s)
Target B — S42: N=11469

[S42] points=11469  threads=8
[S42] BLS done in 1.3s (Pmax used=11.10 d)
[S42] with nmin=3, periods ≲ 11.10 d have ≥3 transits
[S42] TLS refine around 8.58219 d (±1.0%) …
Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 37 duratio

/Users/kobi.weitzman/miniforge3/envs/tess-ephem/lib/python3.10/site-packages/transitleastsquares/grid.py:149: UserWarning: period_grid defaults to R_star=1 and M_star=1 as given density yielded grid with too few values
  warnings.warn(


Searching for best T0 for period 8.57691 days
[S42]   TLS best P=8.576914 d, SDE=7.77 (took 7.3s)
[S42] TLS refine around 8.26204 d (±1.0%) …
Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 37 durations
Searching 11469 data points, 2070 periods from 0.601 to 11.683 days
Using all 8 CPU threads
Searching for best T0 for period 8.57691 days
[S42]   TLS best P=8.576914 d, SDE=7.77 (took 7.0s)
[S42] TLS refine around 7.01537 d (±1.0%) …
Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 37 durations
Searching 11469 data points, 2070 periods from 0.601 to 11.683 days
Using all 8 CPU threads
Searching for best T0 for period 8.57691 days
[S42]   TLS best P=8.576914 d, SDE=7.77 (took 7.3s)
Target B — S70: N=85945

[S70] points=85945  threads=8
[S70] BLS done in 2.8s (Pmax used=11.60 d)
[S70] with nmin=3, periods ≲ 11.60 d have ≥3 transits
[S70] TLS refine around 7.83016 d (±1.0%) …
Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 37 du

...

[stitched]   TLS best P=13.475725 d, SDE=10.63 (took 26.0s)
[stitched] TLS refine around 15.65993 d (±1.0%) …
Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 29 durations
Searching 110391 data points, 586 periods from 15.504 to 15.816 days
Using all 8 CPU threads
Searching for best T0 for period 15.52011 days


/Users/kobi.weitzman/miniforge3/envs/tess-ephem/lib/python3.10/site-packages/transitleastsquares/main.py:411: UserWarning: 117 of 119 transits without data. The true period may be twice the given period.
  warnings.warn(text)


[stitched]   TLS best P=15.520105 d, SDE=5.53 (took 27.3s)
[stitched] TLS refine around 36.38478 d (±1.0%) …
Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 29 durations
Searching 110391 data points, 442 periods from 36.022 to 36.747 days
Using all 8 CPU threads
Searching for best T0 for period 36.38454 days


/Users/kobi.weitzman/miniforge3/envs/tess-ephem/lib/python3.10/site-packages/transitleastsquares/main.py:411: UserWarning: 49 of 51 transits without data. The true period may be twice the given period.
  warnings.warn(text)


[stitched]   TLS best P=36.384540 d, SDE=3.12 (took 32.0s)

Done for Target B with robust prep. Artifacts saved in figures/ and results/.


In [19]:
import glob
sorted(glob.glob("figures/TIC37749396*"))[:10], sorted(glob.glob("results/TIC37749396*"))

(['figures/TIC37749396_S3_BLS_periodogram.png',
  'figures/TIC37749396_S3_TLS_fold_P2.62263.png',
  'figures/TIC37749396_S3_TLS_fold_P6.73332.png',
  'figures/TIC37749396_S3_TLS_periodogram_around_18.24963.png',
  'figures/TIC37749396_S3_TLS_periodogram_around_6.56093.png',
  'figures/TIC37749396_S3_TLS_periodogram_around_6.73492.png',
  'figures/TIC37749396_S3_TLS_periodogram_around_6.73640.png',
  'figures/TIC37749396_S3_TLS_periodogram_around_9.14036.png',
  'figures/TIC37749396_S3_TLS_periodogram_around_9.63175.png',
  'figures/TIC37749396_S42_BLS_periodogram.png'],
 ['results/TIC37749396_S3_BLS_top3.csv',
  'results/TIC37749396_S3_TLS_top3.csv',
  'results/TIC37749396_S42_BLS_top3.csv',
  'results/TIC37749396_S42_TLS_top3.csv',
  'results/TIC37749396_S70_BLS_top3.csv',
  'results/TIC37749396_S70_TLS_top3.csv',
  'results/TIC37749396_download_clean_summary.json',
  'results/TIC37749396_stitched_BLS_top3.csv',
  'results/TIC37749396_stitched_TLS_top3.csv'])

In [20]:
# === Patch 1: star params + stricter TLS ===
# rough TIC-based guesses are fine; refine later if you have better values
R_STAR = 0.8   # R_sun
M_STAR = 0.8   # M_sun

# keep stricter to reduce spurious single-sector hits
TLS_MIN_TRANSITS = 3

In [21]:
# === Patch 2: artifact veto + improved TLS + run_block update ===
import time
import numpy as np

# If your notebook already defined these, this safely reuses them.
try:
    bls_power_safe
except NameError:
    from astropy.timeseries import BoxLeastSquares
    def bls_power_safe(t, f, periods, durations):
        bls = BoxLeastSquares(t, f)
        try:
            return bls.power(periods, durations, objective="snr")
        except TypeError:
            return bls.power(periods, durations)

# obvious spacecraft/system periods to de-emphasize (±2%)
ARTIFACT_PERIODS = (0.5, 1.0, 2.0, 6.85, 13.7, 27.4)
def is_artifact_period(p, avoid=ARTIFACT_PERIODS, tol_frac=0.02):
    return any(abs(p - a)/a < tol_frac for a in avoid)

# Re-define TLS narrow window to pass R_star/M_star and handle threads cleanly
from transitleastsquares import transitleastsquares
def tls_narrow(t, f, p_center, frac=TLS_WINDOW_FRAC, nthreads=TLS_THREADS, nmin=TLS_MIN_TRANSITS):
    tls = transitleastsquares(t, f)
    pmin = max(0.5, p_center*(1-frac))
    pmax = p_center*(1+frac)
    try:
        res = tls.power(period_min=pmin, period_max=pmax,
                        show_progress_bar=True,
                        use_threads=int(nthreads),
                        n_transits_min=int(nmin),
                        R_star=R_STAR, M_star=M_STAR)
    except ValueError:
        # fallback for TLS builds that reject use_threads<1, etc.
        res = tls.power(period_min=pmin, period_max=pmax,
                        show_progress_bar=True,
                        use_threads=1,
                        n_transits_min=int(nmin),
                        R_star=R_STAR, M_star=M_STAR)
    return float(res.period), float(res.SDE), float(res.T0), res

# Re-define run_block to filter artifact periods before TLS
def run_block(label, t, f, target_tic, target_toi):
    """BLS (capped) → TLS around top peaks (artifact-veto). Saves periodograms & folds."""
    print(f"\n[{label}] points={t.size}  threads={TLS_THREADS}")

    # stricter TLS for single sectors
    nmin = 3 if label.startswith("S") else TLS_MIN_TRANSITS

    # Cap BLS period so single-sector has >= nmin transits
    span = float(np.nanmax(t) - np.nanmin(t))
    bls_cap = min(BLS_PERIOD_MAX, max(BLS_PERIOD_MIN*1.2, 0.90*span))
    if label.startswith("S") and nmin >= 2:
        bls_cap = min(bls_cap, 0.95 * span / (nmin - 1))

    periods   = np.linspace(BLS_PERIOD_MIN, bls_cap, BLS_NPER)
    durations = BLS_DURATIONS_HR / 24.0

    t0 = time.time()
    bls_res = bls_power_safe(t, f, periods, durations)
    print(f"[{label}] BLS done in {time.time()-t0:.1f}s (Pmax used={bls_cap:.2f} d)")
    if label.startswith("S") and nmin >= 3:
        print(f"[{label}] with nmin={nmin}, periods ≲ {0.95*span/(nmin-1):.2f} d have ≥{nmin} transits")

    # periodogram + save
    plot_periodogram(
        bls_res.period, bls_res.power,
        "Period (days)", f"{target_toi} ({label}) — BLS periodogram",
        f"{FIGDIR}/TIC{target_tic}_{label}_BLS_periodogram.png"
    )

    # top-3 unique, then drop obvious artifact periods
    bls_topP = unique_peaks(bls_res.period, bls_res.power, k=3, tol_frac=0.01)
    bls_topP = [p for p in bls_topP if not is_artifact_period(p)]
    if not bls_topP:
        print(f"[{label}] all top BLS peaks fell on artifact periods; keeping strongest anyway")
        bls_topP = [float(bls_res.period[np.argmax(bls_res.power)])]

    append_csv(
        f"{RESDIR}/TIC{target_tic}_{label}_BLS_top3.csv",
        [[target_tic, target_toi, label, float(p),
          float(bls_res.power[np.argmin(np.abs(bls_res.period-p))])] for p in bls_topP],
        header=["tic","toi","label","period_days","power"]
    )

    # TLS refine around each remaining BLS peak
    tls_rows = []
    for p in bls_topP:
        print(f"[{label}] TLS refine around {p:.5f} d (±{TLS_WINDOW_FRAC*100:.1f}%) …")
        t1 = time.time()
        p_best, sde, t0_best, res = tls_narrow(
            t, f, p, frac=TLS_WINDOW_FRAC, nthreads=TLS_THREADS, nmin=nmin
        )
        print(f"[{label}]   TLS best P={p_best:.6f} d, SDE={sde:.2f} (took {time.time()-t1:.1f}s)")
        tls_rows.append([target_tic, target_toi, label, p_best, sde, t0_best])

        plot_periodogram(
            res.periods, res.power,
            "Period (days)", f"{target_toi} ({label}) — TLS @ {p:.5f}±{TLS_WINDOW_FRAC*100:.1f}%",
            f"{FIGDIR}/TIC{target_tic}_{label}_TLS_periodogram_around_{p:.5f}.png"
        )
        fold_and_plot(
            t, f, p_best, t0_best,
            f"{target_toi} ({label}) — TLS fold @ P={p_best:.5f} d",
            f"{FIGDIR}/TIC{target_tic}_{label}_TLS_fold_P{p_best:.5f}.png"
        )

    append_csv(
        f"{RESDIR}/TIC{target_tic}_{label}_TLS_top3.csv",
        tls_rows, header=["tic","toi","label","period_days","SDE","T0_BTJD"]
    )

In [22]:
# === Patch 3: stitched-only re-run (Target B) ===
import numpy as np, lightkurve as lk

TARGET_TOI = "Target B"
TARGET_TIC = 37749396
SECTORS    = [3, 42, 70]  # from your earlier discovery

def load_pdcsap_sector(tic, sector):
    sr = lk.search_lightcurve(f"TIC {tic}", mission="TESS", sector=sector, author="SPOC")
    if len(sr) == 0:
        raise RuntimeError(f"No SPOC PDCSAP for TIC {tic} sector {sector}")
    return sr.download().remove_nans()

# minimal fallback in case prep_arrays_for_search isn't in this notebook cell
def _to_arrays_basic(lc):
    t = getattr(lc.time, "value", lc.time)
    f = getattr(lc.flux, "value", lc.flux)
    t = np.asarray(t, float); f = np.asarray(f, float)
    m = np.isfinite(t) & np.isfinite(f)
    f_med = np.nanmedian(f[m]) if np.any(m) else 1.0
    return t[m], (f[m]/(f_med if f_med else 1.0))

t_all_list, f_all_list = [], []
for s in SECTORS:
    lc = load_pdcsap_sector(TARGET_TIC, s)
    if "prep_arrays_for_search" in globals():
        t, f = prep_arrays_for_search(lc)  # uses your robust clean/detrend
    else:
        t, f = _to_arrays_basic(lc.normalize())
    t_all_list.append(t); f_all_list.append(f)

t_all = np.concatenate(t_all_list); f_all = np.concatenate(f_all_list)
order = np.argsort(t_all); t_all, f_all = t_all[order], f_all[order]

run_block("stitched", t_all, f_all, TARGET_TIC, TARGET_TOI)
print("stitched re-run complete. Check figures/TIC37749396_stitched_*")


[stitched] points=110391  threads=8
[stitched] BLS done in 6.2s (Pmax used=50.00 d)
[stitched] TLS refine around 15.65993 d (±1.0%) …
Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 29 durations
Searching 110391 data points, 679 periods from 15.504 to 15.816 days
Using all 8 CPU threads


100%|████████████████████████████████████████████| 679/679 periods | 00:16<00:00


Searching for best T0 for period 15.52014 days


100%|███████████████████████████████████| 13885/13885 [00:11<00:00, 1178.40it/s]
/Users/kobi.weitzman/miniforge3/envs/tess-ephem/lib/python3.10/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/kobi.weitzman/miniforge3/envs/tess-ephem/lib/python3.10/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/kobi.weitzman/miniforge3/envs/tess-ephem/lib/python3.10/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/kobi.weitzman/miniforge3/envs/tess-ephem/lib/python3.10/site-packages/transitleastsquares/main.py:411: UserWarning: 117 of 119 transits without data. The true period may be twice the given period.
  warnings.warn(text)


[stitched]   TLS best P=15.520138 d, SDE=5.93 (took 30.0s)
[stitched] TLS refine around 36.38478 d (±1.0%) …
Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 29 durations
Searching 110391 data points, 514 periods from 36.021 to 36.748 days
Using all 8 CPU threads


100%|████████████████████████████████████████████| 514/514 periods | 00:12<00:00


Searching for best T0 for period 36.42690 days


100%|███████████████████████████████████| 26793/26793 [00:19<00:00, 1347.51it/s]
/Users/kobi.weitzman/miniforge3/envs/tess-ephem/lib/python3.10/site-packages/transitleastsquares/main.py:411: UserWarning: 49 of 51 transits without data. The true period may be twice the given period.
  warnings.warn(text)


[stitched]   TLS best P=36.426902 d, SDE=4.06 (took 33.5s)
stitched re-run complete. Check figures/TIC37749396_stitched_*


In [24]:
from pathlib import Path
import csv

def clean_top3_csv(p):
    rows = list(csv.DictReader(open(p)))
    if not rows: 
        return
    metric = "power" if "BLS" in p.name else "SDE"
    best = {}
    for r in rows:
        try:
            per = round(float(r["period_days"]), 5)
            val = float(r[metric])
        except Exception:
            continue
        if per not in best or val > float(best[per][metric]):
            best[per] = r
    cleaned = sorted(best.values(), key=lambda r: float(r[metric]), reverse=True)[:3]
    with open(p, "w", newline="") as f:
        w = csv.DictWriter(f, fieldnames=cleaned[0].keys())
        w.writeheader(); w.writerows(cleaned)
    print("Cleaned:", p.name, "kept", len(cleaned))

for p in Path("results").glob("TIC37749396_*_{BLS,TLS}_top3.csv"):
    clean_top3_csv(p)

In [26]:
# === Pick Target C from your list (robust column matching) ===
from pathlib import Path
import pandas as pd, re

# Mark targets already used (A & B); add more here if needed
USED_TICS = {119584412, 37749396}

# Prefer priority list, fall back to ranked list
candidates = [Path("results/priority_targets.csv"),
              Path("results/targets_ranked.csv")]
src = next((p for p in candidates if p.exists()), None)
assert src is not None, "No priority/ranked CSV found in results/. Create one first."

df = pd.read_csv(src)

def norm(s: str) -> str:
    return re.sub(r"[^a-z0-9]+", "", s.lower())

# Map normalized -> original column names
cols_norm = {norm(c): c for c in df.columns}

def find_col(options):
    """Return the first matching column name (by fuzzy normalized match)."""
    for opt in options:
        n = norm(opt)
        # exact or substring match in either direction
        for key, orig in cols_norm.items():
            if n == key or n in key or key in n:
                return orig
    return None

# Try many reasonable names
col_tic = find_col([
    "TIC", "tic", "tic_id", "tic id", "target_tic",
    "ticnumber", "ticid", "tic8", "tic_v8", "tic_id_norm"
])
if not col_tic:
    raise ValueError(f"Couldn't find a TIC-like column in {src.name}. "
                     f"Available columns: {list(df.columns)}")

col_toi = find_col([
    "TOI", "toi", "toi_id", "toi id", "target", "target_toi",
    "name", "designation", "label"
])
if not col_toi:
    # If no TOI-like column, fabricate a readable label from TIC
    df["__TOI"] = df[col_tic].apply(lambda v: f"TIC {int(v)}" if pd.notnull(v) else "Unknown")
    col_toi = "__TOI"

# Clean and pick the next unused TIC
df = df[pd.to_numeric(df[col_tic], errors="coerce").notnull()].copy()
df["__tic"] = df[col_tic].astype(int)
pool = df[~df["__tic"].isin(USED_TICS)]

if pool.empty:
    raise ValueError("All candidates in this file are already used. "
                     "Update USED_TICS or switch to another list.")

next_row = pool.iloc[0]
TARGET_TIC = int(next_row["__tic"])
TARGET_TOI = str(next_row[col_toi])

print(f"Selected Target C → TOI='{TARGET_TOI}'  TIC={TARGET_TIC}")
print(f"(Source: {src.name}; skipping used TICs {sorted(USED_TICS)})")

Selected Target C → TOI='550.02'  TIC=311183180
(Source: priority_targets.csv; skipping used TICs [37749396, 119584412])


In [27]:
# --- Target C setup & sector discovery ---
import os, json, time, lightkurve as lk

TARGET_TOI = "TOI 550.02"
TARGET_TIC = 311183180

# 1) Try your local downloader summary first
sectors_pdcsap = []
summary_path = f"results/TIC{TARGET_TIC}_download_clean_summary.json"
if os.path.exists(summary_path):
    try:
        with open(summary_path) as f:
            summ = json.load(f)
        # be tolerant to different key names
        sectors_pdcsap = (summ.get("sectors_pdcsap") or
                          summ.get("pdcsap_sectors") or
                          summ.get("sectors_used") or
                          summ.get("sectors") or [])
        sectors_pdcsap = sorted(set(int(s) for s in sectors_pdcsap))
        print(f"[local] Found PDCSAP sectors in summary: {sectors_pdcsap}")
    except Exception as e:
        print("[local] Summary existed but could not be parsed:", e)

# 2) If nothing local, query MAST with retries
def mast_search_pdcsap(tic, retries=3, sleep=2.5):
    last_err = None
    for i in range(retries):
        try:
            sr = lk.search_lightcurve(f"TIC {tic}", mission="TESS", author="SPOC")
            secs = sorted({int(r.sector) for r in sr if getattr(r, "sector", None) is not None})
            return secs
        except Exception as e:
            last_err = e
            print(f"[mast] attempt {i+1}/{retries} failed: {e} — retrying in {sleep}s")
            time.sleep(sleep)
    if last_err:
        raise last_err

if not sectors_pdcsap:
    sectors_pdcsap = mast_search_pdcsap(TARGET_TIC)
    print(f"[mast] PDCSAP sectors from MAST: {sectors_pdcsap}")

SECTORS = sectors_pdcsap
assert SECTORS, "No sectors found for this TIC. Check network or TIC ID."
print(f"\nTarget C TIC={TARGET_TIC}  PDCSAP sectors: {SECTORS}")

[local] Found PDCSAP sectors in summary: [5, 31]

Target C TIC=311183180  PDCSAP sectors: [5, 31]


In [35]:
# --- Strict local match + robust MAST fallback (Sector-correct) ---
import time
from pathlib import Path
import lightkurve as lk

def _find_local_lcf_exact(tic: int, sector: int):
    """Return a local SPOC LCF path that MATCHES the requested sector; else None."""
    tic16 = f"{int(tic):016d}"
    # MAST uses 's{sector:04d}-' before the TIC in filenames, e.g. s0031-0000...{tic}..._lc.fits
    pat = f"*s{int(sector):04d}-*{tic16}*lc.fits"
    roots = [
        Path.cwd(),
        Path.cwd() / "mastDownload",
        Path.home() / "Downloads" / "mastDownload",
        Path.cwd() / "data_raw_fresh" / "mastDownload",
    ]
    for root in roots:
        if not root.exists():
            continue
        hits = list(root.rglob(pat))
        if hits:
            # most recent file is usually the correct one if duplicates exist
            return sorted(hits, key=lambda p: p.stat().st_mtime)[-1]
    return None

def load_pdcsap_sector(tic: int, sector: int, qmask: int = 175, retries: int = 3, sleep: float = 2.5):
    """Load SPOC PDCSAP for a given TIC/sector.
       1) Use a sector-matched local file if present; otherwise
       2) Download that sector’s LCF from MAST with retries.
       Returns a LightCurve already cleaned & normalized.
    """
    # 1) Exact sector-matched local file?
    local = _find_local_lcf_exact(tic, sector)
    lcf = None
    if local:
        print(f"[local] Using {local}")
        lcf = lk.open(local)  # TessLightCurveFile
    else:
        # 2) MAST fetch using LightCurveFile API (more robust for PDCSAP selection)
        last_err = None
        for i in range(retries):
            try:
                print(f"[mast] Fetching TIC {tic} sector {sector} (try {i+1}/{retries}) …")
                sr = lk.search_lightcurvefile(f"TIC {tic}", mission="TESS", author="SPOC", sector=sector)
                if len(sr) == 0:
                    raise RuntimeError(f"No SPOC LightCurveFile for TIC {tic} sector {sector}")
                lcf = sr.download()
                break
            except Exception as e:
                last_err = e
                if i < retries - 1:
                    print(f"  → retrying in {sleep}s due to: {e}")
                    time.sleep(sleep)
        if lcf is None:
            raise last_err if last_err else RuntimeError("Unknown MAST download error")

    # Sanity: confirm the LightCurveFile sector
    lcfile_sector = getattr(lcf, "sector", None)
    if lcfile_sector is not None and int(lcfile_sector) != int(sector):
        print(f"[warn] Loaded LCF sector={lcfile_sector} but requested sector={sector}")

    # PDCSAP flux, then clean
    lc = lcf.PDCSAP_FLUX.remove_nans().normalize()
    if hasattr(lc, "quality"):
        try:
            # keep cadences that pass the bitmask
            mask = (lc.quality & ~qmask) == 0
            lc = lc[mask]
        except Exception:
            pass
    return lc

In [37]:
# --- Local-first PDCSAP loader with retries for MAST ---
import time
from pathlib import Path
import lightkurve as lk

def _find_local_lcf(tic: int, sector: int):
    tic16 = f"{int(tic):016d}"
    patterns = [
        f"*{tic16}*s{int(sector):04d}*lc.fits",
        f"*{tic16}*lc.fits",
    ]
    roots = [Path.cwd(), Path.cwd()/ "mastDownload", Path.home()/ "Downloads"/ "mastDownload"]
    for root in roots:
        if not root.exists(): 
            continue
        for pat in patterns:
            hits = list(root.rglob(pat))
            if hits:
                return sorted(hits, key=lambda p: (p.stat().st_mtime, len(str(p))))[-1]
    return None

def load_pdcsap_sector(tic: int, sector: int, qmask: int = 175, retries: int = 3, sleep: float = 2.5):
    local = _find_local_lcf(tic, sector)
    if local:
        print(f"[local] Using {local}")
        lcf = lk.open(local)
        lc = lcf.PDCSAP_FLUX
    else:
        last_err = None
        for i in range(retries):
            try:
                print(f"[mast] Fetching TIC {tic} sector {sector} (try {i+1}/{retries}) …")
                sr = lk.search_lightcurvefile(f"TIC {tic}", mission="TESS", author="SPOC", sector=sector)
                lcf = sr.download()
                lc = lcf.PDCSAP_FLUX
                break
            except Exception as e:
                last_err = e
                if i < retries-1:
                    print(f"  → retrying in {sleep}s due to: {e}")
                    time.sleep(sleep)
                else:
                    raise last_err
    lc = lc.remove_nans().normalize()
    if hasattr(lc, "quality"):
        try:
            m = (lc.quality & ~qmask) == 0
            lc = lc[m]
        except Exception:
            pass
    return lc

In [38]:
# --- Run per-sector and stitched on Target C (quiet TLS; uses local-first loader) ---
import os as _os, numpy as np, time
import lightkurve as lk
from astropy.timeseries import BoxLeastSquares
from transitleastsquares import transitleastsquares

# Disable any tqdm-like bars globally (belt & suspenders)
_os.environ.setdefault("TQDM_DISABLE", "1")

# ===== Helpers (define only if missing) =====
try:
    clean_and_detrend
except NameError:
    import numpy as _np
    def _mad(a):
        med = _np.nanmedian(a); return 1.4826*_np.nanmedian(_np.abs(a-med))
    def clean_and_detrend(t, f, window_days=0.75, sigma=5.0):
        m = _np.isfinite(t)&_np.isfinite(f); t,f = _np.asarray(t)[m], _np.asarray(f)[m]
        med, s = _np.nanmedian(f), _mad(f)
        if not _np.isfinite(s) or s==0: s=1e9
        keep = _np.abs(f-med) < sigma*s; t,f = t[keep], f[keep]
        if t.size < 20:
            return t, f/_np.nanmedian(f)
        step = max(window_days/2, (t.max()-t.min())/200)
        grid = _np.arange(t.min()-step, t.max()+step, step)
        trend = []
        for g in grid:
            sel = (t>=g-window_days/2)&(t<g+window_days/2)
            trend.append(_np.nanmedian(f[sel]) if sel.any() else _np.nan)
        grid = grid[_np.isfinite(trend)]
        trend = _np.interp(t, grid, _np.array(trend)[_np.isfinite(trend)])
        trend = _np.where(_np.isfinite(trend)&(trend>0), trend, _np.nanmedian(f))
        ff = f/trend; ff = ff/_np.nanmedian(ff)
        return t, ff

try:
    prep_arrays_for_search
except NameError:
    def prep_arrays_for_search(lc):
        t0 = getattr(lc.time, "value", lc.time)
        f0 = getattr(lc.flux, "value", lc.flux)
        return clean_and_detrend(np.asarray(t0, float), np.asarray(f0, float))

try:
    unique_peaks
except NameError:
    def unique_peaks(periods, power, k=3, tol_frac=0.01):
        idx = np.argsort(power)[::-1]; picks=[]
        for i in idx:
            p = periods[i]
            if all(abs(p-q)/q > tol_frac for q in picks): picks.append(p)
            if len(picks)==k: break
        return picks

try:
    bls_power_safe
except NameError:
    def bls_power_safe(t, f, periods, durations):
        bls = BoxLeastSquares(t, f)
        try: return bls.power(periods, durations, objective="snr")
        except TypeError: return bls.power(periods, durations)

# Always define a QUIET TLS narrow search (ignores any earlier tls_narrow)
R_STAR, M_STAR = 0.55, 0.55
def tls_narrow_quiet(t, f, p_center, frac=0.01, nthreads=max(1, (_os.cpu_count() or 1)), nmin=3):
    tls = transitleastsquares(t, f)
    pmin, pmax = p_center*(1-frac), p_center*(1+frac)
    try:
        res = tls.power(period_min=pmin, period_max=pmax,
                        show_progress_bar=False,  # <- no bar
                        use_threads=int(nthreads),
                        n_transits_min=int(nmin),
                        R_star=R_STAR, M_star=M_STAR)
    except ValueError:
        res = tls.power(period_min=pmin, period_max=pmax,
                        show_progress_bar=False,
                        use_threads=1,
                        n_transits_min=int(nmin),
                        R_star=R_STAR, M_star=M_STAR)
    return float(res.period), float(res.SDE), float(res.T0), res

try:
    plot_periodogram, fold_and_plot, append_csv, run_block
except NameError:
    # Fallback lightweight plot/save utilities
    import csv, matplotlib.pyplot as plt
    FIGDIR, RESDIR = "figures", "results"
    os.makedirs(FIGDIR, exist_ok=True); os.makedirs(RESDIR, exist_ok=True)
    def append_csv(path, rows, header=None):
        new = not os.path.exists(path)
        with open(path, "a", newline="") as f:
            w = csv.writer(f)
            if new and header: w.writerow(header)
            for r in rows: w.writerow(r)
    def plot_periodogram(x, y, xlabel, title, outpng):
        plt.figure(figsize=(8,4), dpi=140); plt.plot(x, y, lw=0.7)
        plt.xlabel(xlabel); plt.ylabel("Power"); plt.title(title)
        plt.tight_layout(); plt.savefig(outpng); plt.close()
    def fold_and_plot(t, f, period, t0, title, outpng, nbins=200):
        ph = ((t - t0 + 0.5*period) % period) / period - 0.5
        o = np.argsort(ph); ph, f = ph[o], f[o]
        bins = np.linspace(-0.5, 0.5, nbins+1); idx = np.digitize(ph, bins)-1
        xb = 0.5*(bins[:-1]+bins[1:])
        yb = np.array([np.nanmean(f[idx==i]) if np.any(idx==i) else np.nan for i in range(nbins)])
        plt.figure(figsize=(8,4), dpi=140)
        plt.plot(ph, f, ".", ms=1.5, alpha=0.25); plt.plot(xb, yb, "-", lw=1.5)
        plt.axvline(0, color="k", lw=1, alpha=0.3)
        plt.xlabel("Phase"); plt.ylabel("Rel. flux"); plt.title(title)
        plt.tight_layout(); plt.savefig(outpng); plt.close()

    # Search settings + runner
    TLS_THREADS = max(1, (_os.cpu_count() or 1))
    BLS_PERIOD_MIN, BLS_PERIOD_MAX, BLS_NPER = 0.5, 50.0, 5000
    BLS_DURATIONS_HR = np.linspace(0.5, 3.0, 18)
    TLS_WINDOW_FRAC, TLS_MIN_TRANSITS = 0.01, 3

    def run_block(label, t, f, target_tic, target_toi):
        print(f"\n[{label}] points={t.size}  threads={TLS_THREADS}")
        # cap per-sector BLS so ≥ n transits
        span = float(np.nanmax(t) - np.nanmin(t))
        bls_cap = min(BLS_PERIOD_MAX, max(BLS_PERIOD_MIN*1.2, 0.90*span))
        if label.startswith("S") and TLS_MIN_TRANSITS >= 2:
            bls_cap = min(bls_cap, 0.95 * span / (TLS_MIN_TRANSITS - 1))
            print(f"[{label}] with nmin={TLS_MIN_TRANSITS}, periods ≲ {0.95*span/(TLS_MIN_TRANSITS-1):.2f} d have ≥{TLS_MIN_TRANSITS} transits")
        periods = np.linspace(BLS_PERIOD_MIN, bls_cap, BLS_NPER)
        durations = BLS_DURATIONS_HR / 24.0

        t0 = time.time()
        bls_res = bls_power_safe(t, f, periods, durations)
        print(f"[{label}] BLS done in {time.time()-t0:.1f}s (Pmax used={bls_cap:.2f} d)")

        plot_periodogram(
            bls_res.period, bls_res.power, "Period (days)",
            f"{target_toi} ({label}) — BLS periodogram",
            f"{FIGDIR}/TIC{target_tic}_{label}_BLS_periodogram.png"
        )

        bls_topP = unique_peaks(bls_res.period, bls_res.power, k=3, tol_frac=0.01)
        append_csv(
            f"{RESDIR}/TIC{target_tic}_{label}_BLS_top3.csv",
            [[target_tic, target_toi, label, float(p),
              float(bls_res.power[np.argmin(np.abs(bls_res.period-p))])] for p in bls_topP],
            header=["tic","toi","label","period_days","power"]
        )

        # TLS around BLS peaks (QUIET)
        tls_rows = []
        for p in bls_topP:
            print(f"[{label}] TLS refine around {p:.5f} d (±{TLS_WINDOW_FRAC*100:.1f}%) …")
            p_best, sde, t0_best, res = tls_narrow_quiet(
                t, f, p, frac=TLS_WINDOW_FRAC,
                nthreads=TLS_THREADS,
                nmin=(3 if label.startswith("S") else TLS_MIN_TRANSITS)
            )
            print(f"[{label}]   TLS best P={p_best:.6f} d, SDE={sde:.2f}")
            plot_periodogram(
                res.periods, res.power, "Period (days)",
                f"{target_toi} ({label}) — TLS @ {p:.5f}±{TLS_WINDOW_FRAC*100:.1f}%",
                f"{FIGDIR}/TIC{target_tic}_{label}_TLS_periodogram_around_{p:.5f}.png"
            )
            fold_and_plot(
                t, f, p_best, t0_best,
                f"{target_toi} ({label}) — TLS fold @ P={p_best:.5f} d",
                f"{FIGDIR}/TIC{target_tic}_{label}_TLS_fold_P{p_best:.5f}.png"
            )
            tls_rows.append([target_tic, target_toi, label, p_best, sde, t0_best])

        append_csv(
            f"{RESDIR}/TIC{target_tic}_{label}_TLS_top3.csv",
            tls_rows, header=["tic","toi","label","period_days","SDE","T0_BTJD"]
        )

# ===== Run per-sector then stitched =====
assert SECTORS, "SECTORS is empty — run the discovery cell first."

t_all_list, f_all_list = [], []
for s in SECTORS:
    # NOTE: no .normalize() here; loader already normalizes
    lc = load_pdcsap_sector(TARGET_TIC, s)
    t, f = prep_arrays_for_search(lc)
    print(f"{TARGET_TOI} — S{s}: N={t.size}")
    run_block(f"S{s}", t, f, TARGET_TIC, TARGET_TOI)
    t_all_list.append(t); f_all_list.append(f)

t_all = np.concatenate(t_all_list); f_all = np.concatenate(f_all_list)
order = np.argsort(t_all); t_all, f_all = t_all[order], f_all[order]
run_block("stitched", t_all, f_all, TARGET_TIC, TARGET_TOI)

print("\nDone for Target C — artifacts in figures/ and results/.")

[local] Using /Users/kobi.weitzman/Documents/tess-ephem/data_raw_fresh/mastDownload/TESS/tess2020294194027-s0031-0000000311183180-0198-s/tess2020294194027-s0031-0000000311183180-0198-s_lc.fits
TOI 550.02 — S5: N=16057

[S5] points=16057  threads=8


/Users/kobi.weitzman/miniforge3/envs/tess-ephem/lib/python3.10/site-packages/transitleastsquares/grid.py:149: UserWarning: period_grid defaults to R_star=1 and M_star=1 as given density yielded grid with too few values
  warnings.warn(


[S5] BLS done in 1.2s (Pmax used=12.08 d)
[S5] with nmin=3, periods ≲ 12.08 d have ≥3 transits
[S5] TLS refine around 12.03045 d (±1.0%) …
Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 37 durations
Searching 16057 data points, 2290 periods from 0.601 to 12.715 days
Using all 8 CPU threads



  0%|                                                 | 0/2290 periods | 00:00<?
  0%|                                           | 1/2290 periods | 00:03<1:55:00
  1%|▏                                           | 12/2290 periods | 00:03<07:10
  2%|▋                                           | 37/2290 periods | 00:03<01:52
  3%|█▏                                          | 62/2290 periods | 00:03<00:59
  4%|█▌                                          | 84/2290 periods | 00:03<00:39
  5%|██                                         | 111/2290 periods | 00:03<00:26
  6%|██▍                                        | 133/2290 periods | 00:03<00:21
  7%|██▉                                        | 155/2290 periods | 00:03<00:17
  8%|███▎                                       | 179/2290 periods | 00:03<00:14
  9%|███▉                                       | 208/2290 periods | 00:03<00:11
 10%|████▍                                      | 235/2290 periods | 00:04<00:10
 11%|████▉                 

Searching for best T0 for period 9.34844 days


...

[stitched]   TLS best P=9.348442 d, SDE=7.84 (took 19.8s)
[stitched] TLS refine around 12.10758 d (±1.0%) …
Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 37 durations
Searching 32114 data points, 2290 periods from 0.601 to 12.715 days
Using all 8 CPU threads



  0%|                                                 | 0/2290 periods | 00:00<?
  0%|                                           | 1/2290 periods | 00:03<2:06:07
  0%|                                             | 5/2290 periods | 00:03<19:36
  1%|▍                                           | 20/2290 periods | 00:03<03:42
  1%|▋                                           | 34/2290 periods | 00:03<01:54
  2%|▉                                           | 51/2290 periods | 00:03<01:06
  3%|█▎                                          | 67/2290 periods | 00:03<00:45
  4%|█▋                                          | 86/2290 periods | 00:03<00:32
  4%|█▉                                         | 101/2290 periods | 00:04<00:27
  5%|██▏                                        | 118/2290 periods | 00:04<00:22
  6%|██▌                                        | 134/2290 periods | 00:04<00:20
  7%|██▊                                        | 149/2290 periods | 00:04<00:19
  7%|███                   

Searching for best T0 for period 9.34844 days



100%|█████████████████████████████████████| 9020/9020 [00:02<00:00, 3235.38it/s]


[stitched]   TLS best P=9.348442 d, SDE=7.84 (took 21.7s)

Done for Target C — artifacts in figures/ and results/.


In [39]:
import numpy as np, pandas as pd, os

# --- Load best stitched TLS P,T0 ---
tls_csv = "results/TIC311183180_stitched_TLS_top3.csv"  # adjust if your path differs
tls = pd.read_csv(tls_csv).sort_values("SDE", ascending=False)
P = float(tls.iloc[0]["period_days"])
T0 = float(tls.iloc[0]["T0_BTJD"])
print(f"Best stitched TLS: P={P:.6f} d, T0(BTJD)={T0:.5f}")

# --- Odd-even depth test (uses your stitched arrays from earlier: t_all, f_all) ---
def odd_even_test(t, f, period, t0, width_frac=0.06, out=None):
    # width_frac ~ fraction of period to call "in-transit" (adjust 0.03–0.08 as needed)
    phase = ((t - t0) % period) / period
    # center transit at phase ~0 by shifting
    phase = (phase + 0.5) % 1 - 0.5
    w = width_frac/2
    in_tr = np.abs(phase) < w
    if not np.any(in_tr):
        raise RuntimeError("No in-transit points found — widen width_frac.")

    # number transits by nearest integer epoch
    k = np.round((t - t0)/period).astype(int)
    ke, ko = (k % 2 == 0), (k % 2 != 0)
    # in-transit & even/odd masks
    ine, ino = in_tr & ke, in_tr & ko

    d_even = 1 - np.nanmedian(f[ine])
    d_odd  = 1 - np.nanmedian(f[ino])
    # simple uncertainty ~ MAD/sqrt(N)
    def mad(x): 
        m = np.nanmedian(x); return 1.4826*np.nanmedian(np.abs(x-m))
    se = mad(f[ine])/np.sqrt(np.sum(np.isfinite(f[ine])) + 1e-9)
    so = mad(f[ino])/np.sqrt(np.sum(np.isfinite(f[ino])) + 1e-9)

    diff_ppm = (d_even - d_odd)*1e6
    sig = diff_ppm / (np.hypot(se, so)*1e6 + 1e-12)
    print(f"Odd-even: depth_even={d_even*1e6:.0f} ppm, depth_odd={d_odd*1e6:.0f} ppm, Δ={diff_ppm:.0f} ppm (~{sig:.2f}σ)")

    if out:
        with open(out, "w") as fsum:
            fsum.write(f"P={period:.6f} d  T0={t0:.5f} BTJD\n")
            fsum.write(f"Odd-even: even={d_even*1e6:.0f} ppm  odd={d_odd*1e6:.0f} ppm  Δ={diff_ppm:.0f} ppm (~{sig:.2f}σ)\n")

# run on your stitched arrays from the previous cell
odd_even_test(t_all, f_all, P, T0, width_frac=0.06)

# --- 2×P sanity (some EBs show at double the period) ---
print("\nChecking at 2×P…")
odd_even_test(t_all, f_all, 2*P, T0, width_frac=0.06)

# --- Ephemeris table (next 50 events starting near the stitched time span) ---
tmin, tmax = float(np.nanmin(t_all)), float(np.nanmax(t_all))
n0 = int(np.floor((tmin - T0)/P))
events = []
for n in range(n0, n0+200):     # generate a bunch and then filter into range or future
    ttran = T0 + n*P
    if ttran >= tmin - P and len(events) < 50:
        events.append([n, ttran])

ephem = pd.DataFrame(events, columns=["epoch", "Tmid_BTJD"])
os.makedirs("results", exist_ok=True)
ephem_path = "results/TIC311183180_ephemeris.csv"
ephem.to_csv(ephem_path, index=False)
print(f"\nSaved ephemeris → {ephem_path}")

Best stitched TLS: P=9.348442 d, T0(BTJD)=2149.63307
Odd-even: depth_even=93 ppm, depth_odd=161 ppm, Δ=-69 ppm (~-1.32σ)

Checking at 2×P…
Odd-even: depth_even=47 ppm, depth_odd=-345 ppm, Δ=392 ppm (~10.44σ)

Saved ephemeris → results/TIC311183180_ephemeris.csv


In [40]:
import csv, os
tic = 311183180
ephem_csv = f"results/TIC{tic}_ephemeris.csv"

P1 = 9.348442
T0 = 2149.63307
P2 = 2*P1

rows = [
    [tic, P1,  T0, "stitched_TLS", "alias_half", "odd_even_sigma_at_2xP=10.44"],
    [tic, P2,  T0, "stitched_TLS", "adopted_for_vetting_EB_likely", "odd_even_sigma=10.44"],
]

newfile = not os.path.exists(ephem_csv)
with open(ephem_csv, "a", newline="") as f:
    w = csv.writer(f)
    if newfile:
        w.writerow(["tic","period_days","T0_BTJD","source","status","notes"])
    for r in rows:
        w.writerow(r)

print("Appended both P and 2×P entries to", ephem_csv)

Appended both P and 2×P entries to results/TIC311183180_ephemeris.csv


In [47]:
# --- Local-first PDCSAP loader (no f-strings; safe on any platform) ---
import time
from pathlib import Path
import lightkurve as lk

def _find_local_lcf(tic, sector):
    tic16 = "{:016d}".format(int(tic))
    patterns = [
        "*{}*s{:04d}*lc.fits".format(tic16, int(sector)),
        "*{}*lc.fits".format(tic16),
    ]
    roots = [
        Path.cwd(),
        Path.cwd() / "mastDownload",
        Path.home() / "Downloads" / "mastDownload",
    ]
    for root in roots:
        if not root.exists():
            continue
        for pat in patterns:
            hits = list(root.rglob(pat))
            if hits:
                hits.sort(key=lambda p: (p.stat().st_mtime, len(str(p))))
                return hits[-1]
    return None

def load_pdcsap_sector(tic, sector, qmask=175, retries=3, sleep=2.5):
    """Load SPOC PDCSAP for a given TIC/sector, preferring local files; retries MAST on failure."""
    # 1) Try local FITS first
    local = _find_local_lcf(tic, sector)
    if local:
        print("[local] Using {}".format(local))
        lcf = lk.open(local)  # TessLightCurveFile
        lc = lcf.PDCSAP_FLUX
    else:
        # 2) MAST fetch with retries (LightCurveFile API is most robust for PDCSAP)
        last_err = None
        for i in range(retries):
            try:
                print("[mast] Fetching TIC {} sector {} (try {}/{}) …".format(tic, sector, i+1, retries))
                sr = lk.search_lightcurvefile("TIC {}".format(tic), mission="TESS", author="SPOC", sector=sector)
                lcf = sr.download()
                lc = lcf.PDCSAP_FLUX
                break
            except Exception as e:
                last_err = e
                if i < retries - 1:
                    print("  retrying in {}s due to: {}".format(sleep, e))
                    time.sleep(sleep)
                else:
                    raise last_err

    # 3) Clean up: drop NaNs, normalize, apply same quality mask convention
    lc = lc.remove_nans().normalize()
    if hasattr(lc, "quality"):
        try:
            m = (lc.quality & ~qmask) == 0
            lc = lc[m]
        except Exception:
            pass
    return lc

In [49]:
from pathlib import Path
import csv, json

TIC = 311183180
TOI = "TOI 550.02"
FIGDIR, RESDIR = Path("figures"), Path("results")
FIGDIR.mkdir(exist_ok=True); RESDIR.mkdir(exist_ok=True)

def clean_top3_csv(p: Path):
    rows = list(csv.DictReader(p.open()))
    if not rows:
        print(f"(skip) {p.name} is empty")
        return
    metric = "power" if "BLS" in p.name else "SDE"
    best = {}
    for r in rows:
        try:
            per = round(float(r.get("period_days") or r.get("period")), 5)
            val = float(r.get(metric, "nan"))
        except Exception:
            continue
        if per not in best or val > float(best[per].get(metric, "-1e9")):
            best[per] = r
    cleaned = sorted(best.values(), key=lambda r: float(r.get(metric, "-1e9")), reverse=True)[:3]
    with p.open("w", newline="") as f:
        w = csv.DictWriter(f, fieldnames=cleaned[0].keys())
        w.writeheader(); w.writerows(cleaned)
    print(f"Cleaned {p.name}: kept {len(cleaned)}")

# Clean all new top-3 files for this TIC
for p in sorted(RESDIR.glob(f"TIC{TIC}_*_[BT]LS_top3.csv")):
    clean_top3_csv(p)

# Quick manifest of what we’ll commit
figs  = sorted(FIGDIR.glob(f"TIC{TIC}_*.png"))
csvs  = sorted(RESDIR.glob(f"TIC{TIC}_*.csv"))
jsons = sorted(RESDIR.glob(f"TIC{TIC}_*.json"))
print("\nFigures:", len(figs))
for f in figs: print("  -", f.name)
print("\nResults (CSV):", len(csvs))
for c in csvs: print("  -", c.name)
print("\nResults (JSON):", len(jsons))
for j in jsons: print("  -", j.name)

# Optional: make a tiny Target_C summary page
summary_path = Path(f"results/TIC{TIC}_download_clean_summary.json")
sectors = []
if summary_path.exists():
    s = json.loads(summary_path.read_text())
    sectors = s.get("sectors_pdcsap") or s.get("pdcsap_sectors") or s.get("sectors") or []
    sectors = [int(x) for x in sectors]
sectors_text = str(sectors if sectors else "[5, 31]")

# Pull stitched TLS “best” for the blurb (if present)
stitched_tls = Path(f"results/TIC{TIC}_stitched_TLS_top3.csv")
best_line = ""
if stitched_tls.exists():
    rows = list(csv.DictReader(stitched_tls.open()))
    if rows:
        rows.sort(key=lambda r: float(r.get("SDE","-1e9")), reverse=True)
        r0 = rows[0]
        try:
            best_line = f"- Stitched TLS: P ≈ {float(r0['period_days']):.5f} d, SDE ≈ {float(r0['SDE']):.2f}.\n"
        except Exception:
            pass

# >>> key fix: no backslashes inside f-string expressions
best_line_text = best_line if best_line else "- Stitched TLS ran; see top-3 CSV and fold PNGs.\n"

doc = Path("docs/targets/Target_C.md")
doc.parent.mkdir(parents=True, exist_ok=True)
doc_md = (
    f"# Target C — {TOI} (TIC {TIC})\n\n"
    f"**Sectors:** {sectors_text} (PDCSAP)\n\n"
    "**Artifacts saved**\n"
    f"- Periodograms & folds: see `figures/TIC{TIC}_*`\n"
    f"- Top-3 tables (BLS/TLS): see `results/TIC{TIC}_*_[BT]LS_top3.csv`\n"
    f"- Ephemeris CSV (P and 2×P both recorded): `results/TIC{TIC}_ephemeris.csv`\n\n"
    "**Quick notes**\n"
    f"{best_line_text}"
    "- Detrend: same gentle settings as Target A (quality mask=175; robust high-pass ~0.75 d; sigma-clip=5).\n"
)
doc.write_text(doc_md)
print(f"\nWrote {doc}")

Cleaned TIC311183180_S31_BLS_top3.csv: kept 3
Cleaned TIC311183180_S31_TLS_top3.csv: kept 1
Cleaned TIC311183180_S5_BLS_top3.csv: kept 3
Cleaned TIC311183180_S5_TLS_top3.csv: kept 1
Cleaned TIC311183180_stitched_BLS_top3.csv: kept 3
Cleaned TIC311183180_stitched_TLS_top3.csv: kept 1

Figures: 21
  - TIC311183180_S31_BLS_periodogram.png
  - TIC311183180_S31_TLS_fold_P9.34844.png
  - TIC311183180_S31_TLS_periodogram_around_11.92853.png
  - TIC311183180_S31_TLS_periodogram_around_12.03045.png
  - TIC311183180_S31_TLS_periodogram_around_12.05824.png
  - TIC311183180_S31_TLS_periodogram_around_6.22816.png
  - TIC311183180_S31_TLS_periodogram_around_9.34124.png
  - TIC311183180_S5_BLS_periodogram.png
  - TIC311183180_S5_TLS_fold_P9.34844.png
  - TIC311183180_S5_TLS_periodogram_around_11.92853.png
  - TIC311183180_S5_TLS_periodogram_around_12.03045.png
  - TIC311183180_S5_TLS_periodogram_around_12.05824.png
  - TIC311183180_S5_TLS_periodogram_around_6.22816.png
  - TIC311183180_S5_TLS_periodo

In [5]:
# === Ephemeris-fit prep: pick targets and sectors ===
import os, json

TARGETS = {
    "Target A": {"tic": 119584412, "toi": "TOI 1801.01"},
    "Target B": {"tic": 37749396,  "toi": "TOI 260.01"},
    "Target C": {"tic": 311183180, "toi": "TOI 550.02"},
}

def _get_pdcsap_sectors_from_summary(tic):
    p = f"results/TIC{tic}_download_clean_summary.json"
    if os.path.exists(p):
        d = json.load(open(p))
        for key in ("sectors_pdcsap","pdcsap_sectors","sectors_used","sectors"):
            if key in d and d[key]:
                return sorted({int(s) for s in d[key]})
    return []

for name, info in TARGETS.items():
    info["sectors"] = _get_pdcsap_sectors_from_summary(info["tic"])

print("Targets + sectors:")
for name, info in TARGETS.items():
    print(f"  {name}: TIC {info['tic']} ({info['toi']})  sectors={info['sectors']}")

Targets + sectors:
  Target A: TIC 119584412 (TOI 1801.01)  sectors=[22, 49]
  Target B: TIC 37749396 (TOI 260.01)  sectors=[3, 42, 70]
  Target C: TIC 311183180 (TOI 550.02)  sectors=[5, 31]


In [6]:
import numpy as np

def stitch_target(tic, sectors):
    t_all, f_all = [], []
    for s in sectors:
        lc = load_pdcsap_sector(tic, s)          # <- your local-first loader
        if lc is None:
            continue
        t, f = prep_arrays_for_search(lc)        # <- your robust detrend
        if t is None or f is None or len(t) == 0:
            continue
        t_all.append(t); f_all.append(f)
    if not t_all:
        return None, None
    t = np.concatenate(t_all); f = np.concatenate(f_all)
    o = np.argsort(t)
    return t[o], f[o]

In [7]:
import inspect, sys

print("Sectors from Cell 1:")
for name, info in TARGETS.items():
    print(f"  {name}: TIC {info['tic']}  sectors={info.get('sectors')}")

def _exists(fn_name):
    return fn_name in globals() and callable(globals()[fn_name])

print("\nFunction presence:")
print("  load_pdcsap_sector:", _exists("load_pdcsap_sector"))
print("  prep_arrays_for_search:", _exists("prep_arrays_for_search"))

# Peek definitions if present
if _exists("load_pdcsap_sector"):
    print("\nload_pdcsap_sector:", inspect.getsource(load_pdcsap_sector).splitlines()[0][:80], "...")
if _exists("prep_arrays_for_search"):
    print("prep_arrays_for_search:", inspect.getsource(prep_arrays_for_search).splitlines()[0][:80], "...")

Sectors from Cell 1:
  Target A: TIC 119584412  sectors=[22, 49]
  Target B: TIC 37749396  sectors=[3, 42, 70]
  Target C: TIC 311183180  sectors=[5, 31]

Function presence:
  load_pdcsap_sector: False
  prep_arrays_for_search: False


In [8]:
# Fallbacks are only defined if your originals don't exist.
try:
    import lightkurve as lk
except Exception as e:
    print("Lightkurve import issue — ensure it's installed in this kernel:", e)

if not _exists("load_pdcsap_sector"):
    from pathlib import Path
    def load_pdcsap_sector(tic: int, sector: int, download_dir="data_raw_fresh"):
        """Fallback: search/download SPOC LCF for TIC+sector and return PDCSAP LightCurve."""
        sr = lk.search_lightcurvefile(f"TIC {int(tic)}", mission="TESS", author="SPOC", sector=sector)
        if len(sr) == 0:
            print(f"    [loader] No LCF found for TIC {tic} S{sector}")
            return None
        lcf = sr.download(download_dir=download_dir)
        if lcf is None:
            print(f"    [loader] Download failed for TIC {tic} S{sector}")
            return None
        try:
            lc = lcf.PDCSAP_FLUX.remove_nans().normalize()
            lc.meta["sector"] = getattr(lcf, "sector", sector)
            return lc
        except Exception as e:
            print(f"    [loader] PDCSAP extract failed TIC {tic} S{sector}:", e)
            return None

if not _exists("prep_arrays_for_search"):
    import numpy as np
    def prep_arrays_for_search(lc, quality_bitmask=175, window_days=1.0, polyorder=2):
        """Fallback detrend: 1-day SavGol-style flatten; returns (t, f) in BTJD, normalized."""
        if lc is None:
            return None, None
        try:
            # Lightkurve’s quality mask is applied upstream in SPOC; keep it gentle here.
            dt = np.median(np.diff(lc.time.value))
            win = max(5, int(round(window_days / max(dt, 1e-6))))
            if win % 2 == 0:
                win += 1
            flat = lc.flatten(window_length=win, polyorder=polyorder)
            t = flat.time.value.astype(float)     # BTJD
            f = (flat.flux.value / np.nanmedian(flat.flux.value)).astype(float)
            # Drop NaNs/infs
            m = np.isfinite(t) & np.isfinite(f)
            t, f = t[m], f[m]
            return (t if t.size else None), (f if f.size else None)
        except Exception as e:
            print("    [detrend] Failed:", e)
            return None, None

In [9]:
# === Robust stitcher with verbose logging ===
import numpy as np

def stitch_target(tic, sectors):
    if not sectors:
        print(f"[stitch] TIC {tic}: empty sector list.")
        return None, None
    t_all, f_all = [], []
    print(f"[stitch] TIC {tic}: trying sectors {sectors}")
    for s in sectors:
        lc = load_pdcsap_sector(tic, s)
        if lc is None:
            print(f"  - S{s}: no LC")
            continue
        t, f = prep_arrays_for_search(lc)
        if t is None or f is None or len(t) == 0:
            print(f"  - S{s}: detrend produced no data")
            continue
        print(f"  - S{s}: ok (N={len(t)})")
        t_all.append(t); f_all.append(f)
    if not t_all:
        print(f"[stitch] TIC {tic}: no usable sectors")
        return None, None
    t = np.concatenate(t_all); f = np.concatenate(f_all)
    o = np.argsort(t)
    t, f = t[o], f[o]
    print(f"[stitch] TIC {tic}: stitched N={len(t)} points across {len(t_all)} sector(s)")
    return t, f

In [10]:
# === Run stitcher for A–C and summarize ===
def run_stitch_for_targets():
    stitched = {}
    for name, info in TARGETS.items():
        tic = info["tic"]
        secs = info.get("sectors", [])
        print(f"\n=== {name} (TIC {tic}) ===")
        t, f = stitch_target(tic, secs)
        stitched[name] = (t, f)
    return stitched

stitched = run_stitch_for_targets()


=== Target A (TIC 119584412) ===
[stitch] TIC 119584412: trying sectors [22, 49]


/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:11: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
  sr = lk.search_lightcurvefile(f"TIC {int(tic)}", mission="TESS", author="SPOC", sector=sector)
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:20: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  lc = lcf.PDCSAP_FLUX.remove_nans().normalize()
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:11: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
  sr = lk.search_lightcurvefile(f"TIC {int(tic)}", mission="TESS", author="SPOC", sector=sector)


  - S22: ok (N=16102)


/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:20: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  lc = lcf.PDCSAP_FLUX.remove_nans().normalize()
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:11: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
  sr = lk.search_lightcurvefile(f"TIC {int(tic)}", mission="TESS", author="SPOC", sector=sector)


  - S49: ok (N=13272)
[stitch] TIC 119584412: stitched N=29374 points across 2 sector(s)

=== Target B (TIC 37749396) ===
[stitch] TIC 37749396: trying sectors [3, 42, 70]


...

/Users/kobi.weitzman/miniforge3/envs/tess-ephem/lib/python3.10/site-packages/lightkurve/search.py:424: LightkurveWarning: Warning: 2 files available to download. Only the first file has been downloaded. Please use `download_all()` or specify additional criteria (e.g. quarter, campaign, or sector) to limit your search.
  warnings.warn(
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:20: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  lc = lcf.PDCSAP_FLUX.remove_nans().normalize()


  - S70: ok (N=86180)
[stitch] TIC 37749396: stitched N=110631 points across 3 sector(s)

=== Target C (TIC 311183180) ===
[stitch] TIC 311183180: trying sectors [5, 31]


/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:11: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
  sr = lk.search_lightcurvefile(f"TIC {int(tic)}", mission="TESS", author="SPOC", sector=sector)
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:20: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  lc = lcf.PDCSAP_FLUX.remove_nans().normalize()
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:11: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
  sr = lk.search_lightcurvefile(f"TIC {int(tic)}", mission="TESS", author="SPOC", sector=sector)


  - S5: ok (N=17286)
  - S31: ok (N=16250)
[stitch] TIC 311183180: stitched N=33536 points across 2 sector(s)


/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:20: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  lc = lcf.PDCSAP_FLUX.remove_nans().normalize()


In [11]:
# === Force-download any missing SPOC LightCurveFiles for the sectors we want ===
import os
from pathlib import Path

DATA_DIR = Path("data_raw_fresh")

def force_download_pdcsap(tic, sectors):
    import lightkurve as lk
    print(f"[download] TIC {tic}: ensuring sectors {sectors}")
    for s in sectors:
        try:
            sr = lk.search_lightcurvefile(f"TIC {int(tic)}", mission="TESS", author="SPOC", sector=int(s))
            if len(sr) == 0:
                print(f"  - S{s}: no SPOC LCF available")
                continue
            # Use cache; if corrupt, lk will re-download
            lcf = sr.download(download_dir=str(DATA_DIR))
            if lcf is None:
                print(f"  - S{s}: download returned None")
                continue
            _ = lcf.PDCSAP_FLUX  # touch PDCSAP to validate file
            print(f"  - S{s}: ready")
        except Exception as e:
            print(f"  - S{s}: download/extract failed -> {e}")

# Download for any target that failed, then retry stitching
for name, (t, f) in stitched.items():
    if t is None or f is None or len(t) == 0:
        tic = TARGETS[name]["tic"]
        secs = TARGETS[name].get("sectors", [])
        force_download_pdcsap(tic, secs)

print("\n[retry] Re-running stitch after downloads...")
stitched = run_stitch_for_targets()


[retry] Re-running stitch after downloads...

=== Target A (TIC 119584412) ===
[stitch] TIC 119584412: trying sectors [22, 49]
  - S22: ok (N=16102)


/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:11: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
  sr = lk.search_lightcurvefile(f"TIC {int(tic)}", mission="TESS", author="SPOC", sector=sector)
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:20: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  lc = lcf.PDCSAP_FLUX.remove_nans().normalize()
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:11: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
  sr = lk.search_lightcurvefile(f"TIC {int(tic)}", mission="TESS", author="SPOC", sector=sector)
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:

  - S49: ok (N=13272)
[stitch] TIC 119584412: stitched N=29374 points across 2 sector(s)

=== Target B (TIC 37749396) ===
[stitch] TIC 37749396: trying sectors [3, 42, 70]
  - S3: ok (N=12978)
  - S42: ok (N=11473)


/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:20: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  lc = lcf.PDCSAP_FLUX.remove_nans().normalize()
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:11: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
  sr = lk.search_lightcurvefile(f"TIC {int(tic)}", mission="TESS", author="SPOC", sector=sector)
/Users/kobi.weitzman/miniforge3/envs/tess-ephem/lib/python3.10/site-packages/lightkurve/search.py:424: LightkurveWarning: Warning: 2 files available to download. Only the first file has been downloaded. Please use `download_all()` or specify additional criteria (e.g. quarter, campaign, or sector) to limit your search.
  warnings.warn(
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:20:

  - S70: ok (N=86180)
[stitch] TIC 37749396: stitched N=110631 points across 3 sector(s)

=== Target C (TIC 311183180) ===
[stitch] TIC 311183180: trying sectors [5, 31]
  - S5: ok (N=17286)
  - S31: ok (N=16250)
[stitch] TIC 311183180: stitched N=33536 points across 2 sector(s)


/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:11: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
  sr = lk.search_lightcurvefile(f"TIC {int(tic)}", mission="TESS", author="SPOC", sector=sector)
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:20: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  lc = lcf.PDCSAP_FLUX.remove_nans().normalize()
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:11: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
  sr = lk.search_lightcurvefile(f"TIC {int(tic)}", mission="TESS", author="SPOC", sector=sector)
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:

In [15]:
import csv, os, math

def pick_tls_guess(tic, label="stitched"):
    """Read period/T0 seed from results/TIC<tic>_<label>_TLS_top3.csv."""
    path = f"results/TIC{tic}_{label}_TLS_top3.csv"
    if not os.path.exists(path):
        return None
    rows = []
    with open(path) as f:
        r = csv.DictReader(f)
        for row in r:
            try:
                P = float(row["period_days"])
                T0 = float(row.get("T0_BTJD", "nan"))
                SDE = float(row.get("SDE", "nan"))
                rows.append((P, T0, SDE))
            except Exception:
                pass
    if not rows:
        return None
    # Highest SDE first
    rows.sort(key=lambda x: (x[2] if math.isfinite(x[2]) else -1.0), reverse=True)
    P, T0, _ = rows[0]
    return dict(P=P, T0=(T0 if math.isfinite(T0) else None))

In [16]:
from dataclasses import dataclass
import numpy as np

@dataclass
class Midtime:
    epoch: int
    tmid: float
    tmid_err: float

def _estimate_duration_hours(P_days, Rstar_Rsun=0.7, b=0.5):
    # rough scaling: duration ~ 13 hr * (P/365)^(1/3) * R* * sqrt(1-b^2)
    return float(13.0 * (P_days/365.0)**(1/3) * Rstar_Rsun * np.sqrt(max(1e-3, 1-b*b)))

def find_midtimes(t, f, P, T0_guess=None, dur_hours=None):
    t = np.asarray(t, float); f = np.asarray(f, float)
    if dur_hours is None:
        dur_hours = _estimate_duration_hours(P)
    half = 0.6*(dur_hours/24.0)  # half-window in days

    # If T0_guess missing, pick phase of minimum median flux, anchor near median time
    if T0_guess is None:
        nb = 200
        phases = (t % P) / P
        bins = np.linspace(0, 1, nb+1)
        idx  = np.digitize(phases, bins) - 1
        yb   = np.array([np.nanmedian(f[idx==i]) if np.any(idx==i) else np.nan for i in range(nb)])
        i0   = np.nanargmin(yb)
        phase0 = 0.5*(bins[i0] + bins[i0+1])
        tmed = np.nanmedian(t)
        k0 = np.round((tmed/P) - phase0).astype(int)
        T0_guess = k0*P + phase0*P

    kmin = int(np.floor((t.min()-T0_guess)/P)) - 1
    kmax = int(np.ceil((t.max()-T0_guess)/P)) + 1
    mids = []

    for k in range(kmin, kmax+1):
        tc = T0_guess + k*P
        sel = (t >= tc - half) & (t <= tc + half)
        if sel.sum() < 6:
            continue
        tt, ff = t[sel], f[sel]

        # local linear baseline removal
        A = np.vstack([np.ones(sel.sum()), tt-tt.mean()]).T
        coef, *_ = np.linalg.lstsq(A, ff, rcond=None)
        fl = ff - (A @ coef)

        # quadratic to the lowest ~30% points
        q = np.nanpercentile(fl, 30)
        use = fl <= q
        if use.sum() < 5:
            use = np.argsort(fl)[:max(5, sel.sum()//5)]
        x = tt[use] - tt[use].mean()
        y = fl[use]
        X = np.vstack([x*x, x, np.ones_like(x)]).T
        a, b, c = np.linalg.lstsq(X, y, rcond=None)[0]
        if a <= 0:
            continue
        x0 = -b/(2*a)
        tmid = tt[use].mean() + x0

        resid = y - (a*x*x + b*x + c)
        s = np.nanstd(resid)
        t_err = np.sqrt(s/max(1e-6, 2*a)) / np.sqrt(use.sum())

        if np.isfinite(tmid) and np.isfinite(t_err) and abs(tmid - tc) <= half:
            mids.append(Midtime(epoch=k, tmid=float(tmid), tmid_err=float(t_err)))

    return mids

In [17]:
import numpy as np

def fit_linear_ephemeris(mids):
    E = np.array([m.epoch for m in mids], int)
    T = np.array([m.tmid  for m in mids], float)
    s = np.array([max(1e-6, m.tmid_err) for m in mids], float)
    X = np.vstack([np.ones_like(E, float), E.astype(float)]).T
    W = np.diag(1.0/s**2)
    XtWX = X.T @ W @ X
    beta = np.linalg.solve(XtWX, X.T @ W @ T)  # [T0, P]
    cov  = np.linalg.inv(XtWX)

    # scale covariance by reduced-χ²
    resid = T - (X @ beta)
    dof   = max(1, len(T) - 2)
    chi2  = float((resid**2 / s**2).sum())
    rchi2 = chi2 / dof
    cov  *= rchi2

    return (
        {
            "T0": float(beta[0]), "P": float(beta[1]),
            "cov": [[float(cov[0,0]), float(cov[0,1])],
                    [float(cov[1,0]), float(cov[1,1])]],
            "sigma_T0": float(np.sqrt(cov[0,0])),
            "sigma_P":  float(np.sqrt(cov[1,1])),
            "cov_T0P":  float(cov[0,1]),
            "N_mids":   int(len(T)),
            "chi2":     chi2,
            "rchi2":    float(rchi2),
        },
        resid, E, T, s
    )

def bootstrap_ephemeris(mids, n_boot=400, random_state=42):
    if len(mids) < 3:
        return None
    rng = np.random.default_rng(random_state)
    samples = []
    for _ in range(n_boot):
        pick = rng.integers(0, len(mids), len(mids))
        mids_b = [mids[i] for i in pick]
        try:
            fit_b, *_ = fit_linear_ephemeris(mids_b)
            samples.append([fit_b["T0"], fit_b["P"]])
        except Exception:
            pass
    if not samples:
        return None
    S = np.array(samples)
    cov = np.cov(S.T)
    return {"samples": S, "cov": [[float(cov[0,0]), float(cov[0,1])],
                                  [float(cov[1,0]), float(cov[1,1])]],
            "mean_T0": float(S[:,0].mean()),
            "mean_P":  float(S[:,1].mean())}

In [18]:
import os, json
import numpy as np
import matplotlib.pyplot as plt
from astropy.time import Time

BTJD_ZERO = 2457000.0

def btjd_from_datestr(date_str):
    t = Time(date_str, scale="utc")
    return float(t.tdb.jd - BTJD_ZERO)

def compute_windows(fit, t_ref_min, t_ref_max, k_sigma=1.0):
    T0, P = fit["T0"], fit["P"]
    cov = np.array(fit["cov"])
    kmin = int(np.floor((t_ref_min - T0)/P)) - 1
    kmax = int(np.ceil((t_ref_max - T0)/P)) + 1
    rows = []
    for k in range(kmin, kmax+1):
        Tpred = T0 + k*P
        if Tpred < t_ref_min or Tpred > t_ref_max:
            continue
        J = np.array([1.0, float(k)])
        sig = float(np.sqrt(max(J @ cov @ J, 0.0)))
        rows.append({"epoch": k, "Tpred_BTJD": float(Tpred),
                     "sigma1d_days": sig,
                     "window_half_width_1sigma_days": k_sigma * sig})
    return rows

def plot_oc(E, T, fit, target_tag, outpng):
    model = fit["T0"] + fit["P"] * E
    oc = (T - model) * 24 * 60  # minutes
    plt.figure(figsize=(7.5,3.8), dpi=140)
    plt.axhline(0, lw=1, alpha=0.3)
    plt.plot(E, oc, "o", ms=4)
    plt.xlabel("Epoch"); plt.ylabel("O–C (minutes)")
    plt.title(f"{target_tag} — O–C")
    plt.tight_layout(); plt.savefig(outpng); plt.close()

def plot_corner(samples, target_tag, outpng):
    if samples is None:
        return
    S = samples["samples"]
    plt.figure(figsize=(6.2,2.8), dpi=140)
    plt.subplot(1,2,1); plt.scatter(S[:,0], S[:,1], s=8, alpha=0.25)
    plt.xlabel("T0 (BTJD)"); plt.ylabel("P (days)")
    plt.title("Bootstrap samples")
    plt.subplot(1,2,2); plt.hist(S[:,1], bins=40)
    plt.xlabel("P (days)"); plt.title("P posterior")
    plt.tight_layout(); plt.savefig(outpng); plt.close()

In [19]:
END_DATE = "2026-03-31"
t_end = btjd_from_datestr(END_DATE)

for name, info in TARGETS.items():
    tic, toi, secs = info["tic"], info["toi"], info.get("sectors", [])
    print(f"\n=== {name} — TIC {tic} ({toi}) — sectors {secs} ===")

    # Use stitched arrays you just built
    t, f = stitch_target(tic, secs)
    if t is None:
        print("  ! No stitched data; skipping.")
        continue

    guess = pick_tls_guess(tic, label="stitched")
    if not guess:
        print("  ! No stitched TLS top-3 CSV found — run the BLS→TLS stitched block first. Skipping.")
        continue

    P0, T00 = guess["P"], guess["T0"]
    print(f"  Using TLS guess: P≈{P0:.6f} d, T0={(f'{T00:.6f}' if T00 else 'auto')}")
    mids = find_midtimes(t, f, P0, T00)
    print(f"  midtimes found: {len(mids)}")

    if len(mids) < 3:
        print("  ! Not enough midtimes for a robust fit (need ≥3). Skipping.")
        continue

    fit, resid, E, Tm, s = fit_linear_ephemeris(mids)
    print(f"  Fit:   P = {fit['P']:.8f} ± {fit['sigma_P']:.8f} d")
    print(f"         T0 = {fit['T0']:.6f} ± {fit['sigma_T0']:.6f} BTJD")
    print(f"         Cov(T0,P) = {fit['cov_T0P']:.3e}   N={fit['N_mids']}   χ²_ν={fit['rchi2']:.2f}")

    # Save midtimes + ephemeris + bootstrap
    os.makedirs("results", exist_ok=True)
    import csv, json
    with open(f"results/TIC{tic}_midtimes.csv","w", newline="") as fcsv:
        w = csv.writer(fcsv); w.writerow(["epoch","tmid_BTJD","tmid_err_d"])
        for m in mids: w.writerow([m.epoch, m.tmid, m.tmid_err])

    with open(f"results/TIC{tic}_refined_ephemeris.json","w") as fj:
        json.dump(fit, fj, indent=2)

    boot = bootstrap_ephemeris(mids, n_boot=400, random_state=42)
    if boot:
        with open(f"results/TIC{tic}_ephem_bootstrap.json","w") as fb:
            json.dump({"mean_P":boot["mean_P"], "mean_T0":boot["mean_T0"], "cov":boot["cov"]}, fb, indent=2)

    # Windows to Mar 2026
    tmin = float(np.nanmin(t))
    windows = compute_windows(fit, t_ref_min=tmin, t_ref_max=t_end, k_sigma=1.0)
    with open(f"results/TIC{tic}_windows_to_2026-03.json","w") as fw:
        json.dump(windows, fw, indent=2)

    # Plots
    os.makedirs("figures", exist_ok=True)
    plot_oc(E, Tm, fit, f"{toi} ({name})", f"figures/TIC{tic}_OC.png")
    if boot:
        plot_corner(boot, f"{toi} ({name})", f"figures/TIC{tic}_P_T0_bootstrap.png")

    print("  Saved: midtimes CSV, refined_ephemeris.json, ephem_bootstrap.json (if any),",
          "windows_to_2026-03.json, O–C, and posterior plot.")


=== Target A — TIC 119584412 (TOI 1801.01) — sectors [22, 49] ===
[stitch] TIC 119584412: trying sectors [22, 49]
  - S22: ok (N=16102)


/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:11: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
  sr = lk.search_lightcurvefile(f"TIC {int(tic)}", mission="TESS", author="SPOC", sector=sector)
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:20: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  lc = lcf.PDCSAP_FLUX.remove_nans().normalize()
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:11: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
  sr = lk.search_lightcurvefile(f"TIC {int(tic)}", mission="TESS", author="SPOC", sector=sector)
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:

  - S49: ok (N=13272)
[stitch] TIC 119584412: stitched N=29374 points across 2 sector(s)
  Using TLS guess: P≈16.027187 d, T0=1908.062441
  midtimes found: 3
  Fit:   P = 16.02749976 ± 0.00026424 d
         T0 = 1908.046283 ± 0.009011 BTJD
         Cov(T0,P) = -1.784e-06   N=3   χ²_ν=1.02


/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:11: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
  sr = lk.search_lightcurvefile(f"TIC {int(tic)}", mission="TESS", author="SPOC", sector=sector)


  Saved: midtimes CSV, refined_ephemeris.json, ephem_bootstrap.json (if any), windows_to_2026-03.json, O–C, and posterior plot.

=== Target B — TIC 37749396 (TOI 260.01) — sectors [3, 42, 70] ===
[stitch] TIC 37749396: trying sectors [3, 42, 70]
  - S3: ok (N=12978)
  - S42: ok (N=11473)


/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:20: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  lc = lcf.PDCSAP_FLUX.remove_nans().normalize()
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:11: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
  sr = lk.search_lightcurvefile(f"TIC {int(tic)}", mission="TESS", author="SPOC", sector=sector)
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:20: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  lc = lcf.PDCSAP_FLUX.remove_nans().normalize()
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:11: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be remov

  - S70: ok (N=86180)
[stitch] TIC 37749396: stitched N=110631 points across 3 sector(s)
  Using TLS guess: P≈13.475725 d, T0=1392.311964
  midtimes found: 4
  Fit:   P = 13.47582381 ± 0.00034966 d
         T0 = 1392.306006 ± 0.043330 BTJD
         Cov(T0,P) = -1.464e-05   N=4   χ²_ν=12.04
  Saved: midtimes CSV, refined_ephemeris.json, ephem_bootstrap.json (if any), windows_to_2026-03.json, O–C, and posterior plot.

=== Target C — TIC 311183180 (TOI 550.02) — sectors [5, 31] ===
[stitch] TIC 311183180: trying sectors [5, 31]
  - S5: ok (N=17286)


/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:11: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
  sr = lk.search_lightcurvefile(f"TIC {int(tic)}", mission="TESS", author="SPOC", sector=sector)
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:20: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  lc = lcf.PDCSAP_FLUX.remove_nans().normalize()
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:11: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
  sr = lk.search_lightcurvefile(f"TIC {int(tic)}", mission="TESS", author="SPOC", sector=sector)
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:

  - S31: ok (N=16250)
[stitch] TIC 311183180: stitched N=33536 points across 2 sector(s)
  Using TLS guess: P≈9.348442 d, T0=2149.633067
  midtimes found: 5
  Fit:   P = 9.34849077 ± 0.00025751 d
         T0 = 2149.633454 ± 0.001675 BTJD
         Cov(T0,P) = -3.597e-08   N=5   χ²_ν=4.92
  Saved: midtimes CSV, refined_ephemeris.json, ephem_bootstrap.json (if any), windows_to_2026-03.json, O–C, and posterior plot.


In [20]:
import json, csv, os, numpy as np
from pathlib import Path

def read_midtimes_csv(tic):
    path = f"results/TIC{tic}_midtimes.csv"
    if not os.path.exists(path): return None
    E, T, s = [], [], []
    with open(path) as f:
        r = csv.DictReader(f)
        for row in r:
            E.append(int(row["epoch"]))
            T.append(float(row["tmid_BTJD"]))
            s.append(max(1e-6, float(row["tmid_err_d"])))
    return np.array(E), np.array(T), np.array(s)

def huber_weights(resid, sigma, c=1.345):
    # Classic Huber weighting on standardized residuals
    z = np.abs(resid / sigma)
    w = np.ones_like(z)
    w[z > c] = c / z[z > c]
    return w

def robust_fit_linear_ephemeris(E, T, s, max_iter=20, c=1.345):
    # Start with weighted LS
    X = np.vstack([np.ones_like(E, float), E.astype(float)]).T
    w = 1.0 / s**2
    beta = np.linalg.lstsq(X * np.sqrt(w)[:,None], T * np.sqrt(w), rcond=None)[0]

    for _ in range(max_iter):
        resid = T - X @ beta
        # Scale estimate (MAD -> sigma)
        mad = np.median(np.abs(resid - np.median(resid))) + 1e-12
        sigma = 1.4826 * mad
        w_r = huber_weights(resid, sigma, c=c)
        W = w_r / s**2
        XtWX = X.T @ (W[:,None] * X)
        beta_new = np.linalg.solve(XtWX, X.T @ (W * T))
        if np.allclose(beta_new, beta, rtol=0, atol=1e-10):
            beta = beta_new; break
        beta = beta_new

    # Robust covariance via sandwich estimator
    resid = T - X @ beta
    W = w_r / s**2
    XtWX = X.T @ (W[:,None] * X)
    H = np.linalg.inv(XtWX)
    S = np.diag(W * resid**2)
    cov = H @ (X.T @ S @ X) @ H

    T0, P = float(beta[0]), float(beta[1])
    cov = np.array(cov, float)
    out = {
        "T0": T0, "P": P,
        "cov": [[float(cov[0,0]), float(cov[0,1])],
                [float(cov[1,0]), float(cov[1,1])]],
        "sigma_T0": float(np.sqrt(cov[0,0])),
        "sigma_P":  float(np.sqrt(cov[1,1])),
        "cov_T0P":  float(cov[0,1]),
        "N_mids":   int(len(T)),
        "rchi2_like": float(np.mean((resid/s)**2))  # diagnostic only
    }
    return out, resid

# Run robust re-fit for A–C and save alongside original
Path("results").mkdir(exist_ok=True)
for name, info in TARGETS.items():
    tic = info["tic"]
    data = read_midtimes_csv(tic)
    if data is None:
        print(f"{name}: no midtimes CSV; skipping.")
        continue
    E, T, s = data
    rob, resid = robust_fit_linear_ephemeris(E, T, s)
    with open(f"results/TIC{tic}_refined_ephemeris_robust.json","w") as f:
        json.dump(rob, f, indent=2)
    print(f"{name}: robust P={rob['P']:.8f} d, T0={rob['T0']:.6f} (σP={rob['sigma_P']:.2e}, σT0={rob['sigma_T0']:.2e})")

Target A: robust P=16.02749976 d, T0=1908.046283 (σP=1.74e-06, σT0=7.95e-05)
Target B: robust P=13.47582381 d, T0=1392.306006 (σP=2.36e-06, σT0=2.97e-04)
Target C: robust P=9.34849093 d, T0=2149.633488 (σP=5.84e-07, σT0=2.20e-06)


In [23]:
# --- Self-contained helpers (works even if earlier cells weren't run) ---
import os, json, numpy as np, matplotlib.pyplot as plt
from pathlib import Path

def load_refined_or_tls_guess(tic, label="stitched"):
    """Return dict(P, T0, source) from refined JSON if present; else fall back to TLS top-3."""
    ref_p = f"results/TIC{tic}_refined_ephemeris.json"
    if os.path.exists(ref_p):
        d = json.load(open(ref_p))
        return {"P": float(d["P"]), "T0": float(d["T0"]), "source": "refined"}
    # fallback to TLS guess (requires pick_tls_guess defined earlier)
    if 'pick_tls_guess' in globals():
        g = pick_tls_guess(tic, label=label)
        if g and ('P' in g) and (g.get('T0') is not None):
            g["source"] = "tls"
            return g
    return None

def load_robust_if_available(tic):
    p = f"results/TIC{tic}_refined_ephemeris_robust.json"
    if os.path.exists(p):
        d = json.load(open(p))
        return {"P": float(d["P"]), "T0": float(d["T0"]), "source": "robust"}
    return None

def make_vetting_panel(t, f, P, T0, outpng, nbins=160, halfwidth=0.15):
    t = np.asarray(t, float); f = np.asarray(f, float)
    phase = ((t - T0)/P) % 1.0
    phase[phase>0.5] -= 1.0  # center on 0

    # Event index & odd/even
    k = np.round((t - T0)/P).astype(int)
    odd = (k % 2 != 0)

    # binning helper
    def _bin(x, y, nb, xlim=(-halfwidth, halfwidth)):
        edges = np.linspace(xlim[0], xlim[1], nb+1)
        idx = np.digitize(x, edges)-1
        xc = 0.5*(edges[:-1]+edges[1:])
        yb = np.array([np.nanmedian(y[idx==i]) if np.any(idx==i) else np.nan for i in range(nb)])
        return xc, yb

    x_all, y_all = _bin(phase, f, nbins)
    x_odd, y_odd = _bin(phase[odd], f[odd], nbins)
    x_even, y_even = _bin(phase[~odd], f[~odd], nbins)

    # secondary near phase 0.5 (±0.03) — FIX: use positional arg name 'nb' instead of 'nbins'
    sec_mask = (np.abs(phase - 0.5) < 0.03) | (np.abs(phase + 0.5) < 0.03)
    x_sec, y_sec = _bin(phase[sec_mask], f[sec_mask], 60, xlim=(-0.03, 0.03))

    plt.figure(figsize=(8.6,5.2), dpi=140)

    plt.subplot(2,2,1)
    plt.plot(phase, f, ".", ms=1, alpha=0.25)
    plt.plot(x_all, y_all, "-", lw=1.4)
    plt.xlim(-halfwidth, halfwidth); plt.title("All transits"); plt.xlabel("Phase"); plt.ylabel("Flux")

    plt.subplot(2,2,2)
    plt.plot(x_odd, y_odd, "-", lw=1.4, label="odd")
    plt.plot(x_even, y_even, "-", lw=1.4, label="even")
    plt.legend(); plt.xlim(-halfwidth, halfwidth); plt.title("Odd vs Even"); plt.xlabel("Phase")

    plt.subplot(2,1,2)
    plt.plot(x_sec, y_sec, "-", lw=1.4)
    plt.title("Secondary window near phase 0.5 (±0.03)")
    plt.xlabel("Phase offset from 0.5"); plt.ylabel("Flux")

    plt.tight_layout()
    Path("figures").mkdir(exist_ok=True)
    plt.savefig(outpng); plt.close()

def vet_all_targets(use_robust=False):
    for name, info in TARGETS.items():
        tic, toi, secs = info["tic"], info["toi"], info["sectors"]
        print(f"[vet] {name} TIC {tic} sectors {secs}")
        t, f = stitch_target(tic, secs)
        if t is None:
            print("  -> no data"); continue
        ep = load_refined_or_tls_guess(tic)
        if use_robust:
            rb = load_robust_if_available(tic)
            if rb: ep = rb
        if not ep:
            print("  -> no ephemeris/TLS seed; skipping"); continue
        outpng = f"figures/TIC{tic}_fold_odd_even.png" if not use_robust else f"figures/TIC{tic}_fold_odd_even_robust.png"
        make_vetting_panel(t, f, ep["P"], ep["T0"], outpng)
        print(f"  -> saved {outpng}")

# Run once with refined, and (optionally) again with robust:
vet_all_targets(use_robust=False)
# vet_all_targets(use_robust=True)

[vet] Target A TIC 119584412 sectors [22, 49]
[stitch] TIC 119584412: trying sectors [22, 49]
  - S22: ok (N=16102)


/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:11: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
  sr = lk.search_lightcurvefile(f"TIC {int(tic)}", mission="TESS", author="SPOC", sector=sector)
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:20: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  lc = lcf.PDCSAP_FLUX.remove_nans().normalize()
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:11: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
  sr = lk.search_lightcurvefile(f"TIC {int(tic)}", mission="TESS", author="SPOC", sector=sector)
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:

  - S49: ok (N=13272)
[stitch] TIC 119584412: stitched N=29374 points across 2 sector(s)
  -> saved figures/TIC119584412_fold_odd_even.png
[vet] Target B TIC 37749396 sectors [3, 42, 70]
[stitch] TIC 37749396: trying sectors [3, 42, 70]


/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:11: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
  sr = lk.search_lightcurvefile(f"TIC {int(tic)}", mission="TESS", author="SPOC", sector=sector)
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:20: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  lc = lcf.PDCSAP_FLUX.remove_nans().normalize()
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:11: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
  sr = lk.search_lightcurvefile(f"TIC {int(tic)}", mission="TESS", author="SPOC", sector=sector)
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:

  - S3: ok (N=12978)
  - S42: ok (N=11473)


/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:20: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  lc = lcf.PDCSAP_FLUX.remove_nans().normalize()


  - S70: ok (N=86180)
[stitch] TIC 37749396: stitched N=110631 points across 3 sector(s)
  -> saved figures/TIC37749396_fold_odd_even.png
[vet] Target C TIC 311183180 sectors [5, 31]
[stitch] TIC 311183180: trying sectors [5, 31]
  - S5: ok (N=17286)


/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:11: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
  sr = lk.search_lightcurvefile(f"TIC {int(tic)}", mission="TESS", author="SPOC", sector=sector)
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:20: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  lc = lcf.PDCSAP_FLUX.remove_nans().normalize()
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:11: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
  sr = lk.search_lightcurvefile(f"TIC {int(tic)}", mission="TESS", author="SPOC", sector=sector)
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:

  - S31: ok (N=16250)
[stitch] TIC 311183180: stitched N=33536 points across 2 sector(s)
  -> saved figures/TIC311183180_fold_odd_even.png


In [24]:
import os, json, numpy as np, matplotlib.pyplot as plt
from pathlib import Path

# --- helpers: ephemeris pick (refined/robust) ---
def pick_ephemeris_for_folding(tic, prefer_robust=False):
    # robust for folding if present and requested
    if prefer_robust and os.path.exists(f"results/TIC{tic}_refined_ephemeris_robust.json"):
        d = json.load(open(f"results/TIC{tic}_refined_ephemeris_robust.json"))
        return {"P": float(d["P"]), "T0": float(d["T0"]), "src": "robust"}
    # otherwise refined
    if os.path.exists(f"results/TIC{tic}_refined_ephemeris.json"):
        d = json.load(open(f"results/TIC{tic}_refined_ephemeris.json"))
        return {"P": float(d["P"]), "T0": float(d["T0"]), "src": "refined"}
    # final fallback to TLS guess from earlier cell 3
    if 'pick_tls_guess' in globals():
        g = pick_tls_guess(tic, label="stitched")
        if g and g.get("T0") is not None:
            g["src"] = "tls"
            return g
    return None

# --- simple box-fit around phase 0 to estimate depth & duration ---
def quick_boxfit(phase, flux, halfw=0.15):
    """Return (depth, duration_days, in_mask) using robust medians around phase 0."""
    # define windows
    in_w  = 0.03  # +/- 0.03 d in phase units of days-equivalent (we phase in days)
    out_w = (0.07, 0.15)

    # In/out selection in phase-distance (assuming phase already centered on 0 in days)
    in_mask  = (np.abs(phase) < in_w)
    out_mask = ((phase >  out_w[0]) & (phase <  out_w[1])) | ((phase < -out_w[0]) & (phase > -out_w[1]))

    # robust medians
    f_in  = np.nanmedian(flux[in_mask])  if np.any(in_mask)  else np.nan
    f_out = np.nanmedian(flux[out_mask]) if np.any(out_mask) else np.nan
    depth = max(0.0, (f_out - f_in))  # depth as positive number (flux drop)
    # duration estimate from width where flux < (f_out - 0.5*depth)
    thr = f_out - 0.5*depth
    core = (flux < thr) & (np.abs(phase) < halfw)
    if np.any(core):
        # contiguous region around 0
        x = np.abs(phase[core])
        duration_days = 2.0 * np.nanpercentile(x, 95)  # robust width
    else:
        duration_days = 0.08  # ~2 hr default
    return float(depth), float(duration_days), in_mask

def fold_and_bin(t, f, P, T0, nbins=180, halfw=0.15):
    phase = ((t - T0) % P)
    phase[phase > P/2] -= P       # center around 0
    # Convert to "phase in days" so windows are in days directly
    ph_days = phase
    # bin
    edges = np.linspace(-halfw, halfw, nbins+1)
    idx = np.digitize(ph_days, edges) - 1
    xc  = 0.5*(edges[:-1]+edges[1:])
    yb  = np.array([np.nanmedian(f[idx==i]) if np.any(idx==i) else np.nan for i in range(nbins)])
    return ph_days, xc, yb

def figure_A_panel(tic, toi, t, f, prefer_robust=False, nbins=180, halfw=0.15):
    ep = pick_ephemeris_for_folding(tic, prefer_robust=prefer_robust)
    if not ep:
        print(f"[FigA] TIC {tic}: no ephemeris; skipping.")
        return
    P, T0, src = ep["P"], ep["T0"], ep["src"]

    ph_days, xb, yb = fold_and_bin(t, f, P, T0, nbins=nbins, halfw=halfw)
    # quick boxfit on binned to stabilize
    depth, dur_d, _ = quick_boxfit(xb, yb, halfw=halfw)
    # build a box model on binned x
    model = np.full_like(xb, np.nan)
    in_box = (np.abs(xb) <= dur_d/2)
    # baseline is around 1.0 since we normalized
    baseline = np.nanmedian(yb[np.abs(xb) > 0.07])
    model[:] = baseline
    model[in_box] = baseline - depth

    # residuals
    resid = yb - model

    # plot
    Path("figures").mkdir(exist_ok=True)
    fig = plt.figure(figsize=(8.6,5.6), dpi=140)
    gs = fig.add_gridspec(2,1, height_ratios=[2.5,1.0], hspace=0.15)

    ax1 = fig.add_subplot(gs[0])
    ax1.plot(ph_days, f, ".", ms=1, alpha=0.25)
    ax1.plot(xb, yb, "-", lw=1.5, label="binned")
    ax1.plot(xb, model, "-", lw=1.8, label="box model")
    ax1.set_xlim(-halfw, halfw)
    ax1.set_ylabel("Normalized flux")
    ax1.set_title(f"{toi} — TIC {tic}  |  Folded on {src} ephemeris  |  P={P:.6f} d, T0={T0:.6f} BTJD")
    ax1.legend(loc="best", fontsize=9)

    ax2 = fig.add_subplot(gs[1], sharex=ax1)
    ax2.axhline(0, color="k", lw=1, alpha=0.4)
    ax2.plot(xb, resid, "-", lw=1.2)
    ax2.set_xlim(-halfw, halfw)
    ax2.set_xlabel("Phase (days)")
    ax2.set_ylabel("Residual")

    out_png = f"figures/TIC{tic}_fold_model.png"
    fig.tight_layout()
    fig.savefig(out_png); plt.close(fig)

    # save quick boxfit numbers
    Path("results").mkdir(exist_ok=True)
    box = {
        "depth": depth,                # (unitless flux), ~depth_ppm ≈ depth*1e6
        "depth_ppm": depth*1e6,
        "duration_days": dur_d,
        "duration_hours": dur_d*24.0,
        "baseline": float(baseline),
        "P": P, "T0": T0, "source": src
    }
    with open(f"results/TIC{tic}_boxfit.json","w") as fjs:
        json.dump(box, fjs, indent=2)

    print(f"[FigA] {toi} TIC {tic}: saved {out_png} and results/TIC{tic}_boxfit.json")

# === run for A–C ===
for name, info in TARGETS.items():
    tic, toi, secs = info["tic"], info["toi"], info["sectors"]
    print(f"[FigA] building panel for {name} TIC {tic} secs {secs}")
    t, f = stitch_target(tic, secs)
    if t is None: 
        print("  -> no data"); 
        continue
    # Prefer robust only for Target B (folding), refined for A & C
    prefer_rob = (name == "Target B")
    figure_A_panel(tic, toi, t, f, prefer_robust=prefer_rob)

[FigA] building panel for Target A TIC 119584412 secs [22, 49]
[stitch] TIC 119584412: trying sectors [22, 49]


/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:11: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
  sr = lk.search_lightcurvefile(f"TIC {int(tic)}", mission="TESS", author="SPOC", sector=sector)
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:20: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  lc = lcf.PDCSAP_FLUX.remove_nans().normalize()


  - S22: ok (N=16102)


/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:11: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
  sr = lk.search_lightcurvefile(f"TIC {int(tic)}", mission="TESS", author="SPOC", sector=sector)
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:20: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  lc = lcf.PDCSAP_FLUX.remove_nans().normalize()
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/1851795620.py:103: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:11: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.

  - S49: ok (N=13272)
[stitch] TIC 119584412: stitched N=29374 points across 2 sector(s)
[FigA] TOI 1801.01 TIC 119584412: saved figures/TIC119584412_fold_model.png and results/TIC119584412_boxfit.json
[FigA] building panel for Target B TIC 37749396 secs [3, 42, 70]
[stitch] TIC 37749396: trying sectors [3, 42, 70]


...

  - S70: ok (N=86180)
[stitch] TIC 37749396: stitched N=110631 points across 3 sector(s)
[FigA] TOI 260.01 TIC 37749396: saved figures/TIC37749396_fold_model.png and results/TIC37749396_boxfit.json
[FigA] building panel for Target C TIC 311183180 secs [5, 31]
[stitch] TIC 311183180: trying sectors [5, 31]


/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/1851795620.py:103: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:11: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
  sr = lk.search_lightcurvefile(f"TIC {int(tic)}", mission="TESS", author="SPOC", sector=sector)
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:20: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  lc = lcf.PDCSAP_FLUX.remove_nans().normalize()


  - S5: ok (N=17286)
  - S31: ok (N=16250)
[stitch] TIC 311183180: stitched N=33536 points across 2 sector(s)


/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:11: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
  sr = lk.search_lightcurvefile(f"TIC {int(tic)}", mission="TESS", author="SPOC", sector=sector)
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/2864407655.py:20: LightkurveDeprecationWarning: The PDCSAP_FLUX function is deprecated and may be removed in a future version.
  lc = lcf.PDCSAP_FLUX.remove_nans().normalize()
/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_15037/1851795620.py:103: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()


[FigA] TOI 550.02 TIC 311183180: saved figures/TIC311183180_fold_model.png and results/TIC311183180_boxfit.json


In [25]:
import json, os

manifest = {
  "TIC119584412": {"prefer_robust_for_folding": False, "label": "Target A"},
  "TIC37749396":  {"prefer_robust_for_folding": True,  "label": "Target B"},
  "TIC311183180": {"prefer_robust_for_folding": False, "label": "Target C"}
}
os.makedirs("results", exist_ok=True)
with open("results/folding_manifest.json","w") as f:
    json.dump(manifest, f, indent=2)
print("Saved results/folding_manifest.json")

Saved results/folding_manifest.json


In [26]:
import os, json, csv

rows = []
for name, info in TARGETS.items():
    tic = info["tic"]; tag = f"TIC{tic}"
    pth = f"results/{tag}_refined_ephemeris.json"
    if not os.path.exists(pth):
        continue
    d = json.load(open(pth))
    rows.append({
        "target": name, "tic": tic,
        "P_days": d["P"], "sigma_P_days": d.get("sigma_P"),
        "T0_BTJD": d["T0"], "sigma_T0_days": d.get("sigma_T0"),
        "cov_T0P": d.get("cov_T0P"), "N_mids": d.get("N_mids"),
        "chi2": d.get("chi2"), "rchi2": d.get("rchi2")
    })

os.makedirs("results", exist_ok=True)
with open("results/ephemeris_summary.csv","w", newline="") as f:
    w = csv.DictWriter(f, fieldnames=list(rows[0].keys()))
    w.writeheader(); w.writerows(rows)

print("Saved results/ephemeris_summary.csv with A–C refined ephemerides.")

Saved results/ephemeris_summary.csv with A–C refined ephemerides.


In [27]:
# === Sanity-check refined ephemerides & box-fit depths ===
import os, json, csv, math
import numpy as np

def _load_json(p):
    return json.load(open(p)) if os.path.exists(p) else None

rows = []
for name, info in TARGETS.items():
    tic = info["tic"]; tag = f"TIC{tic}"
    ep = _load_json(f"results/{tag}_refined_ephemeris.json")
    bx = _load_json(f"results/{tag}_boxfit.json")
    if not ep: 
        print(f"{name}: no refined ephemeris found, skipping.")
        continue
    sigma_T0, sigma_P = ep.get("sigma_T0"), ep.get("sigma_P")
    cov_T0P = ep.get("cov_T0P", 0.0)
    rho = float(cov_T0P / (sigma_T0*sigma_P)) if sigma_T0 and sigma_P else np.nan

    rows.append({
        "target": name, "tic": tic,
        "P_days": ep["P"], "σP_days": sigma_P,
        "T0_BTJD": ep["T0"], "σT0_days": sigma_T0,
        "rho(T0,P)": rho,
        "N_mids": ep.get("N_mids"),
        "rchi2": ep.get("rchi2"),
        "depth_ppm": (bx.get("depth_ppm") if bx else None),
        "duration_hr": (bx.get("duration_hours") if bx else None)
    })

# Write/print a compact CSV + flag line
os.makedirs("results", exist_ok=True)
with open("results/ephemeris_qc.csv","w", newline="") as f:
    w = csv.DictWriter(f, fieldnames=list(rows[0].keys()))
    w.writeheader(); w.writerows(rows)

print("Saved results/ephemeris_qc.csv")
for r in rows:
    flag = ""
    if r["rchi2"] and r["rchi2"] > 5: 
        flag += " [HIGH rχ²]"
    if r["N_mids"] and r["N_mids"] < 4:
        flag += " [FEW mids]"
    print(f"{r['target']}: P={r['P_days']:.8f}±{r['σP_days']:.2e} d | "
          f"T0={r['T0_BTJD']:.6f}±{r['σT0_days']:.2e} | ρ={r['rho(T0,P)']:.3f} | "
          f"N={r['N_mids']} rχ²={r['rchi2']:.2f} | depth≈{r['depth_ppm']:.0f} ppm, dur≈{r['duration_hr']:.2f} h{flag}")

Saved results/ephemeris_qc.csv
Target A: P=16.02749976±2.64e-04 d | T0=1908.046283±9.01e-03 | ρ=-0.749 | N=3 rχ²=1.02 | depth≈98 ppm, dur≈6.53 h [FEW mids]
Target B: P=13.47582381±3.50e-04 d | T0=1392.306006±4.33e-02 | ρ=-0.966 | N=4 rχ²=12.04 | depth≈710 ppm, dur≈4.42 h [HIGH rχ²]
Target C: P=9.34849077±2.58e-04 d | T0=2149.633454±1.67e-03 | ρ=-0.083 | N=5 rχ²=4.92 | depth≈775 ppm, dur≈5.72 h


In [29]:
# === Cell 17 — Fig C-style "upcoming windows" panel (bug-fixed) ===
import os, json
import numpy as np
import matplotlib.pyplot as plt
from astropy.time import Time

BTJD_ZERO = 2457000.0

def _load_windows(tic):
    p = f"results/TIC{tic}_windows_to_2026-03.json"
    if not os.path.exists(p):
        return None
    return json.load(open(p))

def _load_ephem(tic):
    p = f"results/TIC{tic}_refined_ephemeris.json"
    if not os.path.exists(p):
        return None
    return json.load(open(p))

def _btjd_to_utc_isot(btjd_array):
    jd = np.asarray(btjd_array) + BTJD_ZERO
    # ephemeris is in TDB; convert to UTC ISO for human readability
    return Time(jd, format="jd", scale="tdb").utc.isot

def show_next_windows(tic, n_show=8, print_only=False):
    W = _load_windows(tic)
    if not W:
        print(f"TIC{tic}: no windows file found.")
        return None

    # Extract arrays once; DO NOT re-index floats later
    w_btjd   = np.array([row["Tpred_BTJD"] for row in W], dtype=float)
    w_sig_d  = np.array([row["sigma1d_days"] for row in W], dtype=float)
    w_sig_min= (w_sig_d * 24.0 * 60.0)

    # Keep "future-ish" windows relative to now (TDB)
    now_btjd = Time.now().tdb.jd - BTJD_ZERO
    future   = w_btjd >= now_btjd
    w_btjd, w_sig_min = w_btjd[future], w_sig_min[future]

    # Format UTC stamps
    w_dates  = _btjd_to_utc_isot(w_btjd)

    # Package and print the next n_show rows
    nxt = list(zip(w_btjd, w_sig_min, w_dates))[:n_show]
    print(f"Next predicted mid-transits for TIC{tic} (BTJD, ±1σ [min], UTC):")
    for tb, smin, ds in nxt:
        print(f"  {tb:.6f}, ±{smin:.1f} min, {ds}")
    if print_only:
        return nxt

    # Simple bar plot of the next n_show windows (±1σ half-width in minutes)
    fig, ax = plt.subplots(figsize=(7.5, 3.2), dpi=140)
    x = np.arange(len(nxt))
    halfmins = np.array([s for _, s, _ in nxt], float)
    labels   = [ds.replace('T', ' ').replace('Z','')[:16] for *_, ds in nxt]  # YYYY-MM-DD hh:mm
    ax.bar(x, halfmins, align="center")
    ax.set_ylabel("1σ half-width (minutes)")
    ax.set_title(f"TIC{tic} — upcoming transit windows")
    ax.set_xticks(x)
    ax.set_xticklabels(labels, rotation=45, ha="right", fontsize=9)
    fig.tight_layout()
    outpng = f"figures/TIC{tic}_upcoming_windows.png"
    os.makedirs("figures", exist_ok=True)
    plt.savefig(outpng)
    plt.close(fig)
    print(f"Saved {outpng}")
    return nxt

# Run for A–C
for name, info in TARGETS.items():
    tic = info["tic"]
    print(f"\n[windows] {name} — TIC {tic}")
    show_next_windows(tic, n_show=8, print_only=False)


[windows] Target A — TIC 119584412
Next predicted mid-transits for TIC119584412 (BTJD, ±1σ [min], UTC):
  3943.538753, ±39.5 min, 2025-09-25T00:54:39.053
  3959.566252, ±39.9 min, 2025-10-11T01:34:15.033
  3975.593752, ±40.3 min, 2025-10-27T02:13:51.012
  3991.621252, ±40.7 min, 2025-11-12T02:53:26.991
  4007.648752, ±41.0 min, 2025-11-28T03:33:02.970
  4023.676252, ±41.4 min, 2025-12-14T04:12:38.949
  4039.703751, ±41.8 min, 2025-12-30T04:52:14.928
  4055.731251, ±42.1 min, 2026-01-15T05:31:50.907
Saved figures/TIC119584412_upcoming_windows.png

[windows] Target B — TIC 37749396
Next predicted mid-transits for TIC37749396 (BTJD, ±1σ [min], UTC):
  3939.236706, ±38.4 min, 2025-09-20T17:39:42.224
  3952.712530, ±38.8 min, 2025-10-04T05:04:53.402
  3966.188354, ±39.3 min, 2025-10-17T16:30:04.579
  3979.664178, ±39.8 min, 2025-10-31T03:55:15.756
  3993.140001, ±40.2 min, 2025-11-13T15:20:26.933
  4006.615825, ±40.7 min, 2025-11-27T02:45:38.110
  4020.091649, ±41.2 min, 2025-12-10T14:10:4

In [1]:
# Patch the loader to use search_lightcurve().download() instead of search_lightcurvefile()
try:
    import lightkurve as lk
except Exception as e:
    print("Lightkurve not available in this kernel:", e)

def load_pdcsap_sector(tic: int, sector: int, download_dir="data_raw_fresh"):
    sr = lk.search_lightcurve(f"TIC {int(tic)}", mission="TESS", author="SPOC", sector=sector)
    if len(sr) == 0:
        print(f"    [loader] No LCF found for TIC {tic} S{sector}")
        return None
    lcs = sr.download(download_dir=download_dir)  # returns LightCurve or LightCurveCollection
    if lcs is None:
        print(f"    [loader] Download failed for TIC {tic} S{sector}")
        return None
    # Prefer PDCSAP if available, else first LC normalized
    try:
        lc = (lcs.PDCSAP_FLUX if hasattr(lcs, "PDCSAP_FLUX") else lcs).remove_nans().normalize()
        lc.meta["sector"] = getattr(lcs, "sector", sector)
        return lc
    except Exception as e:
        print(f"    [loader] PDCSAP extract failed TIC {tic} S{sector}:", e)
        return None

In [2]:
import numpy as np, json, os
from dataclasses import dataclass

@dataclass
class MidtimeRow:
    epoch:int; tmid:float; tmid_err:float

def _read_midtimes_csv(path):
    rows = []
    with open(path) as f:
        for i, line in enumerate(f):
            if i == 0:  # header
                continue
            parts = line.strip().split(",")
            if len(parts) < 3: continue
            rows.append(MidtimeRow(int(parts[0]), float(parts[1]), float(parts[2])))
    return rows

def _wls_fit(E, T, s):
    X = np.vstack([np.ones_like(E, float), E.astype(float)]).T
    W = np.diag(1.0/np.maximum(s,1e-12)**2)
    XtWX = X.T @ W @ X
    beta = np.linalg.solve(XtWX, X.T @ W @ T)  # [T0, P]
    cov  = np.linalg.inv(XtWX)
    resid = T - (X @ beta)
    dof   = max(1, len(T)-2)
    chi2  = float((resid**2/s**2).sum())
    rchi2 = chi2/dof
    return beta, cov, resid, rchi2

def robust_ephemeris_with_jitter(mid_csv, max_iter=10, clip_sigma=3.5):
    mids = _read_midtimes_csv(mid_csv)
    if len(mids) < 3:
        raise RuntimeError("Need ≥3 midtimes for robust fit")
    E  = np.array([m.epoch for m in mids], int)
    T  = np.array([m.tmid  for m in mids], float)
    s0 = np.array([max(1e-6, m.tmid_err) for m in mids], float)

    mask = np.ones_like(T, bool)
    jitter = 0.0
    for _ in range(max_iter):
        s = np.sqrt(s0[mask]**2 + jitter**2)
        beta, cov, resid, rchi2 = _wls_fit(E[mask], T[mask], s)
        # clip outliers in O–C
        oc = resid/np.sqrt(np.maximum(s**2,1e-12))
        keep = np.abs(oc) < clip_sigma
        # update mask (map back to full)
        m2 = mask.copy()
        m2[np.where(mask)[0][~keep]] = False
        # adjust jitter to target rchi2≈1 (simple secant-like step)
        if rchi2 > 1.0:
            jitter = np.sqrt(max(0.0, (rchi2-1.0))) * np.median(s0)  # coarse step
        else:
            jitter *= 0.5
        # stop if stable
        if np.all(m2 == mask) and abs(rchi2-1.0) < 0.05:
            mask = m2
            break
        mask = m2

    # Final fit with settled mask+jitter
    s = np.sqrt(s0[mask]**2 + jitter**2)
    beta, cov, resid, rchi2 = _wls_fit(E[mask], T[mask], s)

    # scale cov by rchi2 (should be ~1, but keep formal)
    cov *= rchi2
    out = {
        "T0": float(beta[0]), "P": float(beta[1]),
        "cov": [[float(cov[0,0]), float(cov[0,1])],
                [float(cov[1,0]), float(cov[1,1])]],
        "sigma_T0": float(np.sqrt(cov[0,0])),
        "sigma_P":  float(np.sqrt(cov[1,1])),
        "cov_T0P":  float(cov[0,1]),
        "N_mids_used": int(mask.sum()),
        "N_mids_total": int(len(T)),
        "rchi2_final": float(rchi2),
        "jitter_days": float(jitter),
        "dropped_epochs": [int(E[i]) for i in np.where(~mask)[0]],
    }
    return out

In [4]:
# === Robust ephemerides + windows (self-contained cell) ===
import os, csv, json
import numpy as np
import matplotlib.pyplot as plt
from astropy.time import Time

# --------- targets (same IDs you used above) ----------
TARGETS = {
    "Target A": {"tic": 119584412, "toi": "TOI 1801.01"},
    "Target B": {"tic": 37749396,  "toi": "TOI 260.01"},
    "Target C": {"tic": 311183180, "toi": "TOI 550.02"},
}

# --------- helpers ----------
def _read_midtimes_csv(path):
    """Return list of dicts: [{'epoch':int,'tmid':float,'err':float}, ...]"""
    out = []
    with open(path) as f:
        r = csv.DictReader(f)
        # Accept either exact headers or any variant containing 'epoch'/'tmid'
        for row in r:
            e = int(row.get("epoch", row.get("Epoch", row.get("n"))))
            t = float(row.get("tmid_BTJD", row.get("tmid", row.get("Tmid_BTJD"))))
            s = float(row.get("tmid_err_d", row.get("tmid_err", row.get("Sigma_d", 0.0))))
            out.append({"epoch": e, "tmid": t, "err": max(1e-6, s)})
    return out

def _wls_linear_ephem(E, T, s):
    X = np.vstack([np.ones_like(E, float), E.astype(float)]).T  # [T0, P]
    W = np.diag(1.0/np.maximum(1e-12, s*s))
    XtWX = X.T @ W @ X
    beta = np.linalg.solve(XtWX, X.T @ W @ T)  # [T0, P]
    cov  = np.linalg.inv(XtWX)
    resid = T - (X @ beta)
    chi2  = float((resid**2 / np.maximum(1e-12, s*s)).sum())
    dof   = max(1, len(T) - 2)
    rchi2 = chi2 / dof
    return beta, cov, resid, chi2, rchi2

def robust_ephemeris_with_jitter(mid_csv, tol=1e-4, max_iter=60):
    """
    Fit linear ephemeris with an additive jitter term in quadrature to reach rchi2~1.
    Returns dict: {T0,P,cov, sigmas, cov_T0P, N_mids, chi2, rchi2, jitter_days}
    """
    mids = _read_midtimes_csv(mid_csv)
    E = np.array([m["epoch"] for m in mids], int)
    T = np.array([m["tmid"]  for m in mids], float)
    s0 = np.array([m["err"]   for m in mids], float)

    # First pass (no jitter)
    beta, cov, resid, chi2, rchi2 = _wls_linear_ephem(E, T, s0)
    jitter = 0.0

    if rchi2 > 1.0 and len(T) >= 3:
        # Binary search for jitter so reduced-chi2 ~ 1
        lo, hi = 0.0, 0.5  # days; hi is safely large
        for _ in range(max_iter):
            mid = 0.5*(lo+hi)
            _, _, _, chi2_mid, rchi2_mid = _wls_linear_ephem(E, T, np.sqrt(s0**2 + mid**2))
            if rchi2_mid > 1.0:
                lo = mid
            else:
                hi = mid
            if abs(rchi2_mid - 1.0) < tol:
                jitter = mid
                break
        else:
            jitter = hi
        # Final fit with jitter applied
        s = np.sqrt(s0**2 + jitter**2)
        beta, cov, resid, chi2, rchi2 = _wls_linear_ephem(E, T, s)
    else:
        s = s0

    # Scale covariance by rchi2 (standard WLS practice)
    cov = cov * max(1.0, rchi2)
    out = {
        "T0": float(beta[0]),
        "P":  float(beta[1]),
        "cov": [[float(cov[0,0]), float(cov[0,1])],
                [float(cov[1,0]), float(cov[1,1])]],
        "sigma_T0": float(np.sqrt(cov[0,0])),
        "sigma_P":  float(np.sqrt(cov[1,1])),
        "cov_T0P":  float(cov[0,1]),
        "N_mids":   int(len(T)),
        "chi2":     float(chi2),
        "rchi2":    float(rchi2),
        "jitter_days": float(jitter),
    }
    return out

def compute_windows_from_fit(fit, t_ref_min, t_ref_max, k_sigma=1.0):
    T0, P = fit["T0"], fit["P"]
    cov = np.array(fit["cov"], float)
    kmin = int(np.floor((t_ref_min - T0)/P)) - 1
    kmax = int(np.ceil((t_ref_max - T0)/P)) + 1
    rows = []
    for k in range(kmin, kmax+1):
        Tpred = T0 + k*P
        if not (t_ref_min <= Tpred <= t_ref_max):
            continue
        J = np.array([1.0, float(k)])
        sig = float(np.sqrt(max(J @ cov @ J, 0.0)))
        rows.append({"epoch": k, "Tpred_BTJD": float(Tpred),
                     "sigma1d_days": sig,
                     "window_half_width_1sigma_days": k_sigma*sig})
    return rows

def plot_oc_from_midcsv(mid_csv, fit, tag, outpng):
    mids = _read_midtimes_csv(mid_csv)
    E  = np.array([m["epoch"] for m in mids], int)
    T  = np.array([m["tmid"]  for m in mids], float)
    s  = np.array([m["err"]   for m in mids], float)
    model = fit["T0"] + fit["P"]*E
    oc_min = (T - model) * 24.0 * 60.0
    plt.figure(figsize=(7.8,3.4), dpi=140)
    plt.axhline(0, color="k", lw=1, alpha=0.3)
    plt.errorbar(E, oc_min, yerr=s*24*60, fmt="o", ms=4, capsize=2)
    plt.xlabel("Epoch"); plt.ylabel("O–C (minutes)")
    plt.title(f"{tag} — O–C (robust)")
    plt.tight_layout(); plt.savefig(outpng); plt.close()

# --------- run for all targets ----------
END_DATE = "2026-03-31"
BTJD_ZERO = 2457000.0
t_end = Time(END_DATE, scale="utc").tdb.jd - BTJD_ZERO

os.makedirs("results", exist_ok=True)
os.makedirs("figures", exist_ok=True)

for name, info in TARGETS.items():
    tic, toi = info["tic"], info["toi"]
    mid_csv = f"results/TIC{tic}_midtimes.csv"
    if not os.path.exists(mid_csv):
        print(f"{name}: no midtimes CSV, skipping.")
        continue

    print(f"[robust] {name} — TIC {tic}")
    fit_r = robust_ephemeris_with_jitter(mid_csv)
    with open(f"results/TIC{tic}_refined_ephemeris_robust.json","w") as f:
        json.dump(fit_r, f, indent=2)

    # windows from first measured midtime to END_DATE
    tmin = float(np.loadtxt(mid_csv, delimiter=",", skiprows=1, usecols=1).min())
    W = compute_windows_from_fit(fit_r, t_ref_min=tmin, t_ref_max=float(t_end))
    with open(f"results/TIC{tic}_windows_to_2026-03_robust.json","w") as f:
        json.dump(W, f, indent=2)

    plot_oc_from_midcsv(mid_csv, fit_r, f"{toi} ({name})", f"figures/TIC{tic}_OC_robust.png")
    print("  -> saved refined_ephemeris_robust.json, windows_to_2026-03_robust.json, OC_robust.png")

[robust] Target A — TIC 119584412
  -> saved refined_ephemeris_robust.json, windows_to_2026-03_robust.json, OC_robust.png
[robust] Target B — TIC 37749396
  -> saved refined_ephemeris_robust.json, windows_to_2026-03_robust.json, OC_robust.png
[robust] Target C — TIC 311183180
  -> saved refined_ephemeris_robust.json, windows_to_2026-03_robust.json, OC_robust.png


In [5]:
# === Figure C prototype: transit-window drift (catalog vs. yours) ===
import os, json, csv, math
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from astropy.time import Time

# ---- targets (reuse your mapping) ----
TARGETS = {
    "Target A — TIC 119584412": {"tic": 119584412, "toi": "TOI 1801.01"},
    "Target B — TIC 37749396":  {"tic": 37749396,  "toi": "TOI 260.01"},
    "Target C — TIC 311183180": {"tic": 311183180, "toi": "TOI 550.02"},
}

BTJD_ZERO = 2457000.0

@dataclass
class Ephem:
    P: float
    T0: float
    cov: np.ndarray | None  # 2x2 (T0,P) covariance or None
    source: str

def _load_your_ephem(tic: int, robust=False) -> Ephem | None:
    base = f"results/TIC{tic}_refined_ephemeris{'_robust' if robust else ''}.json"
    if not os.path.exists(base):
        return None
    d = json.load(open(base))
    cov = np.array(d["cov"], float) if ("cov" in d) else None
    return Ephem(P=float(d["P"]), T0=float(d["T0"]), cov=cov, source=("yours_robust" if robust else "yours"))

def _load_catalog_ephem(tic: int) -> Ephem | None:
    """
    Priority:
      1) results/TIC<tic>_catalog_ephemeris.json   (keys: P, T0, optional cov or sigma entries)
      2) results/TIC<tic>_stitched_TLS_top3.csv    (best SDE row; uses period_days + optional T0_BTJD)
    """
    # 1) JSON
    pjson = f"results/TIC{tic}_catalog_ephemeris.json"
    if os.path.exists(pjson):
        d = json.load(open(pjson))
        P  = float(d["P"])
        T0 = float(d.get("T0_BTJD", d.get("T0", np.nan)))
        cov = None
        if "cov" in d:
            cov = np.array(d["cov"], float)
        elif "sigma_P" in d or "sigma_T0" in d:
            # Build diagonal cov if sigmas are provided
            sP  = float(d.get("sigma_P", np.nan))
            sT0 = float(d.get("sigma_T0", np.nan))
            if np.isfinite(sP) and np.isfinite(sT0):
                cov = np.array([[sT0**2, 0.0],[0.0, sP**2]])
        if np.isfinite(P) and np.isfinite(T0):
            return Ephem(P=P, T0=T0, cov=cov, source="catalog_json")
    # 2) CSV fallback (TLS top-3)
    pcsv = f"results/TIC{tic}_stitched_TLS_top3.csv"
    if os.path.exists(pcsv):
        best = None
        with open(pcsv) as f:
            r = csv.DictReader(f)
            for row in r:
                try:
                    P  = float(row["period_days"])
                    T0 = float(row.get("T0_BTJD", "nan"))
                    SDE = float(row.get("SDE", "nan"))
                except Exception:
                    continue
                if not math.isfinite(P):
                    continue
                # prefer highest SDE
                if best is None or (math.isfinite(SDE) and SDE > best[2]):
                    best = (P, T0, SDE)
        if best:
            P, T0, SDE = best
            # TLS fallback rarely has cov; use None (plot will still show Tpred line w/o bands)
            # If T0 missing, we’ll align epochs by P only; bands will still be informative.
            if not math.isfinite(T0):
                # Anchor T0 to the first available window from your solution later when plotting
                T0 = np.nan
            return Ephem(P=float(P), T0=float(T0), cov=None, source="tls_top1")
    return None

def _sigma_time_from_cov(ep: Ephem, k: int) -> float:
    """Return 1σ timing (days) at epoch k using cov if available; else NaN."""
    if ep.cov is None:
        return np.nan
    J = np.array([1.0, float(k)])  # dT/d[T0,P]
    v = J @ ep.cov @ J
    return float(np.sqrt(max(v, 0.0)))

def _btjd_from_datestr(date_str: str) -> float:
    t = Time(date_str, scale="utc")
    return float(t.tdb.jd - BTJD_ZERO)

In [6]:
def make_epoch_grid(T0_ref: float, P: float, tmin: float, tmax: float):
    """Return integer epoch indices spanning [tmin, tmax] around T0_ref given P."""
    if not np.isfinite(T0_ref):
        # If T0 missing, anchor near tmin
        T0_ref = tmin
    kmin = int(np.floor((tmin - T0_ref)/P)) - 2
    kmax = int(np.ceil((tmax - T0_ref)/P)) + 2
    return np.arange(kmin, kmax+1, dtype=int)

def compute_curves(ep_yours: Ephem, ep_cat: Ephem | None, tmin: float, tmax: float):
    """
    Build arrays for plotting:
      epochs (k), Tpred_yours, sigma_yours, Tpred_cat, sigma_cat
    """
    k = make_epoch_grid(ep_yours.T0, ep_yours.P, tmin, tmax)

    # Your curve
    T_y = ep_yours.T0 + k * ep_yours.P
    S_y = np.array([_sigma_time_from_cov(ep_yours, int(kk)) for kk in k], float)

    # Catalog curve (if present). If catalog T0 is NaN, align T0 to your first T_y for visual compare.
    if ep_cat is not None:
        T0c = ep_cat.T0 if np.isfinite(ep_cat.T0) else T_y[0]
        T_c = T0c + k * ep_cat.P
        S_c = np.array([_sigma_time_from_cov(ep_cat, int(kk)) for kk in k], float)
    else:
        T_c = np.full_like(T_y, np.nan)
        S_c = np.full_like(S_y, np.nan)

    return k, T_y, S_y, T_c, S_c

def minutes(x_days):  # helper for plotting labels
    return float(x_days) * 24.0 * 60.0

In [7]:
def plot_figure_c(tic: int, name_tag: str, ep_yours: Ephem, ep_cat: Ephem | None,
                  tmin: float, tmax: float, outpng: str, annotate_dates=True):
    """
    Panel with two rows:
      Top: timing window half-width (1σ, minutes) vs epoch (k)
      Bottom: same vs date (UTC) with predicted mid-times
    """
    k, T_y, S_y, T_c, S_c = compute_curves(ep_yours, ep_cat, tmin, tmax)

    # Build date labels
    dates = Time(BTJD_ZERO + T_y, format="jd", scale="tdb").to_datetime()
    # For catalog dates, if T_c is NaN it will be skipped by plotting
    dates_c = None
    if np.isfinite(T_c).any():
        dates_c = Time(BTJD_ZERO + T_c, format="jd", scale="tdb").to_datetime()

    fig = plt.figure(figsize=(9.6, 6.0), dpi=140)
    gs = fig.add_gridspec(2, 1, height_ratios=[1,1], hspace=0.25)

    # Row 1: epoch space
    ax1 = fig.add_subplot(gs[0,0])
    ax1.plot(k, [minutes(s) for s in S_y], "-", lw=2, label=f"Yours ({ep_yours.source})")
    if ep_cat is not None and np.isfinite(S_c).any():
        ax1.plot(k, [minutes(s) for s in S_c], "--", lw=1.5, label=f"Catalog ({ep_cat.source})")
    ax1.set_xlabel("Epoch (k)")
    ax1.set_ylabel("Timing 1σ half-width (minutes)")
    ax1.set_title(f"{name_tag} — Transit-window growth (epoch space)")
    ax1.grid(True, alpha=0.25)
    ax1.legend(loc="upper left")

    # Row 2: date space
    ax2 = fig.add_subplot(gs[1,0])
    ax2.plot(dates, [minutes(s) for s in S_y], "-", lw=2, label=f"Yours ({ep_yours.source})")
    if ep_cat is not None and (dates_c is not None):
        ax2.plot(dates_c, [minutes(s) for s in S_c], "--", lw=1.5, label=f"Catalog ({ep_cat.source})")
    ax2.set_xlabel("Calendar date (UTC)")
    ax2.set_ylabel("Timing 1σ half-width (minutes)")
    ax2.set_title(f"{name_tag} — Transit-window growth (date space)")
    ax2.grid(True, alpha=0.25)
    ax2.legend(loc="upper left")

    if annotate_dates:
        # mark 'today' and Mar 31, 2026
        today_btjd = Time.now().tdb.jd - BTJD_ZERO
        t_cut = _btjd_from_datestr("2026-03-31")
        for ax in (ax1, ax2):
            # vertical helpers on ax2 only (dates)
            pass
        # Only draw vertical lines on the bottom panel
        ax2.axvline(Time(BTJD_ZERO + today_btjd, format="jd", scale="tdb").to_datetime(),
                    color="k", ls=":", lw=1, alpha=0.6)
        ax2.axvline(Time(BTJD_ZERO + t_cut, format="jd", scale="tdb").to_datetime(),
                    color="k", ls=":", lw=1, alpha=0.6)
        ax2.text(0.01, 0.93, "Today", transform=ax2.transAxes, fontsize=9, alpha=0.7)
        ax2.text(0.78, 0.93, "2026-03-31", transform=ax2.transAxes, fontsize=9, alpha=0.7)

    fig.suptitle(f"Figure C prototype — TIC {tic}", y=0.995, fontsize=12)
    fig.tight_layout()
    os.makedirs("figures", exist_ok=True)
    fig.savefig(outpng)
    plt.close(fig)

In [8]:
# ---- Compute & Save: per-target Figure C + drift summary ----
END_DATE = "2026-03-31"
t_end = _btjd_from_datestr(END_DATE)

summary_rows = []
os.makedirs("results", exist_ok=True)

for tag, info in TARGETS.items():
    tic, toi = info["tic"], info["toi"]
    name_tag = f"{toi} ({tag.split('—')[0].strip()})"

    # Load ephemerides
    ep_y  = _load_your_ephem(tic) or _load_your_ephem(tic, robust=True)
    if ep_y is None:
        print(f"[FigureC] {tag}: no refined ephemeris found; skipping.")
        continue
    ep_c  = _load_catalog_ephem(tic)
    if ep_c is None:
        print(f"[FigureC] {tag}: no catalog baseline found; plotting 'yours' only.")

    # Set plotting time span from earliest data (if available) to END_DATE
    # Prefer your midtimes CSV for earliest point; else use your windows JSON.
    tmin = None
    mid_csv = f"results/TIC{tic}_midtimes.csv"
    if os.path.exists(mid_csv):
        try:
            tvals = np.loadtxt(mid_csv, delimiter=",", skiprows=1, usecols=1)
            tmin = float(np.nanmin(np.atleast_1d(tvals)))
        except Exception:
            tmin = None
    if tmin is None:
        # fallback: earliest predicted window from your windows file
        wjson = f"results/TIC{tic}_windows_to_2026-03.json"
        if os.path.exists(wjson):
            W = json.load(open(wjson))
            if isinstance(W, list) and len(W):
                tmin = float(min([w["Tpred_BTJD"] for w in W]))
    if tmin is None:
        # final fallback: a few periods before your T0
        tmin = ep_y.T0 - 5*ep_y.P

    # Build curves and capture a few checkpoints
    k, T_y, S_y, T_c, S_c = compute_curves(ep_y, ep_c, tmin, t_end)

    # Record 1σ at three reference points: near start, mid, end of range
    def pick(vals):
        if vals.size == 0: return np.nan, np.nan, np.nan
        i0, i1, i2 = 0, vals.size//2, -1
        return float(vals[i0]), float(vals[i1]), float(vals[i2])

    sy0, sym, sye = [minutes(x) for x in pick(S_y)]
    sc0, scm, sce = [minutes(x) for x in pick(S_c)] if (ep_c is not None) else (np.nan, np.nan, np.nan)

    summary_rows.append({
        "tic": tic, "toi": toi,
        "yours_P_days": ep_y.P, "yours_T0_BTJD": ep_y.T0,
        "yours_sigma_min_start": sy0, "yours_sigma_min_mid": sym, "yours_sigma_min_end": sye,
        "catalog_source": (ep_c.source if ep_c else ""),
        "catalog_P_days": (ep_c.P if ep_c else np.nan),
        "catalog_T0_BTJD": (ep_c.T0 if (ep_c and np.isfinite(ep_c.T0)) else np.nan),
        "catalog_sigma_min_start": sc0, "catalog_sigma_min_mid": scm, "catalog_sigma_min_end": sce
    })

    # Plot and save
    outpng = f"figures/TIC{tic}_FigureC_window_growth.png"
    plot_figure_c(tic, name_tag, ep_y, ep_c, tmin, t_end, outpng=outpng)
    print(f"[FigureC] Saved {outpng}")

# Write summary CSV
if summary_rows:
    with open("results/figureC_window_growth_summary.csv","w", newline="") as f:
        w = csv.DictWriter(f, fieldnames=list(summary_rows[0].keys()))
        w.writeheader(); w.writerows(summary_rows)
    print("Saved results/figureC_window_growth_summary.csv")
else:
    print("No targets written; check ephemeris files.")

/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_1982/3604536555.py:58: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()


[FigureC] Saved figures/TIC119584412_FigureC_window_growth.png
[FigureC] Saved figures/TIC37749396_FigureC_window_growth.png
[FigureC] Saved figures/TIC311183180_FigureC_window_growth.png
Saved results/figureC_window_growth_summary.csv


In [9]:
# --- Figure C helpers: robust "catalog" ephemeris loader + plotting shim ---

import os, csv, json
import numpy as np
import matplotlib.pyplot as plt
from astropy.time import Time

BTJD_ZERO = 2457000.0

def _btjd_from_iso(s):
    return float(Time(s, scale="utc").tdb.jd - BTJD_ZERO)

def load_catalog_ephemeris(tic):
    """
    Try, in order:
      1) results/TIC{tic}_catalog_ephem.json            (your own pinned catalog copy)
      2) results/TIC{tic}_stitched_TLS_top3.csv         (use top-1 as a placeholder "catalog")
      3) results/TIC{tic}_refined_ephemeris.json        (fallback: use 'yours' also as 'catalog')
    Returns: dict {P, T0, label, source} or None if nothing usable.
    """
    # 1) explicit catalog JSON (preferred if you have it)
    p_json = f"results/TIC{tic}_catalog_ephem.json"
    if os.path.exists(p_json):
        try:
            d = json.load(open(p_json))
            if all(k in d for k in ("P","T0")):
                return {"P": float(d["P"]), "T0": float(d["T0"]),
                        "label": "Catalog", "source": os.path.basename(p_json)}
        except Exception:
            pass

    # 2) stitched TLS top-3; use top-1 as a stand-in for “catalog”
    p_tls = f"results/TIC{tic}_stitched_TLS_top3.csv"
    if os.path.exists(p_tls):
        rows = []
        with open(p_tls) as f:
            r = csv.DictReader(f)
            for row in r:
                try:
                    P = float(row["period_days"])
                    # allow T0 column to be missing/NaN; we can backfill below
                    T0 = row.get("T0_BTJD", "") or "nan"
                    T0 = float(T0)
                    SDE = float(row.get("SDE","nan"))
                    rows.append((P, T0, SDE))
                except Exception:
                    continue
        if rows:
            # choose the highest SDE if present
            rows.sort(key=lambda x: (x[2] if np.isfinite(x[2]) else -1), reverse=True)
            P, T0, _ = rows[0]
            if not np.isfinite(T0):
                # If TLS CSV has no T0, take an approximate T0 near the first data point
                # using your refined ephemeris if present, otherwise mid-time of stitched data.
                p_ref = f"results/TIC{tic}_refined_ephemeris.json"
                if os.path.exists(p_ref):
                    try:
                        j = json.load(open(p_ref))
                        T0 = float(j["T0"])
                    except Exception:
                        T0 = np.nan
                if not np.isfinite(T0):
                    # final backstop: put T0 at 0 to at least show slope; label that it’s approximate
                    T0 = 0.0
            return {"P": float(P), "T0": float(T0),
                    "label": "Catalog (tls_top1)", "source": os.path.basename(p_tls)}

    # 3) last-resort: mirror your refined ephemeris so something plots
    p_ref = f"results/TIC{tic}_refined_ephemeris.json"
    if os.path.exists(p_ref):
        try:
            d = json.load(open(p_ref))
            return {"P": float(d["P"]), "T0": float(d["T0"]),
                    "label": "Catalog (fallback=same as yours)", "source": os.path.basename(p_ref)}
        except Exception:
            pass

    return None


def sigma_timing_minutes(P, T0, cov, epochs):
    """
    1σ timing half-width in minutes at integer epochs.
    cov is 2x2 on [T0, P]. epochs can be array-like.
    """
    cov = np.asarray(cov, float)
    E = np.asarray(epochs, float)
    # J = [1, E]; σ = sqrt(J C J^T)
    s = np.sqrt(np.clip(cov[0,0] + 2*E*cov[0,1] + (E**2)*cov[1,1], 0, np.inf))
    return s * 24 * 60  # days -> minutes


def plot_window_growth_panels(tic, toi_label, yours, catalog, t_min_btjd, t_max_btjd, outpng):
    """
    yours: dict with P, T0, cov
    catalog: dict with P, T0, label, source (may be None)
    """
    # epoch space
    E = np.arange(-10, 220+1)  # generous
    s_yours = sigma_timing_minutes(yours["P"], yours["T0"], yours["cov"], E)

    # date space (convert BTJD span to a set of dates)
    btjd_grid = np.linspace(t_min_btjd, t_max_btjd, 120)
    E_grid = (btjd_grid - yours["T0"]) / yours["P"]
    s_date_yours = sigma_timing_minutes(yours["P"], yours["T0"], yours["cov"], E_grid)
    dates = Time(btjd_grid + BTJD_ZERO, format="jd", scale="tdb").utc.datetime

    fig = plt.figure(figsize=(10,6.4), dpi=140)

    # Top: epoch space
    ax1 = fig.add_subplot(2,1,1)
    ax1.plot(E, s_yours, lw=2, label="Yours (yours)")
    if catalog is not None:
        # We don’t have a covariance for catalog; approximate growth with your Cov(P,T0)
        # but centered on catalog (P, T0) so the curve is visible.
        s_cat = sigma_timing_minutes(yours["P"], catalog["T0"], yours["cov"], E * (catalog["P"]/yours["P"]))
        ax1.plot(E, s_cat, "--", lw=2, label=catalog["label"])
    ax1.set_xlabel("Transit epoch (k)")
    ax1.set_ylabel("Timing 1σ half-width (minutes)")
    ax1.set_title(f"{toi_label} — Transit-window growth (epoch space)")
    ax1.legend()

    # Bottom: dates
    ax2 = fig.add_subplot(2,1,2)
    ax2.plot(dates, s_date_yours, lw=2, label="Yours (yours)")
    if catalog is not None:
        # Map catalog ephemeris onto the same date grid (again reusing your cov as a proxy)
        E_grid_cat = (btjd_grid - catalog["T0"]) / catalog["P"]
        s_date_cat = sigma_timing_minutes(yours["P"], catalog["T0"], yours["cov"], E_grid_cat)
        ax2.plot(dates, s_date_cat, "--", lw=2, label=catalog["label"])
    ax2.set_ylabel("Timing 1σ half-width (minutes)")
    ax2.set_title(f"{toi_label} — Transit-window growth (date space)")
    ax2.legend()
    for x in [Time("2025-12-31").utc.datetime, Time("2026-03-31").utc.datetime]:
        ax2.axvline(x, color="k", ls=":", lw=1)
    fig.tight_layout()
    fig.savefig(outpng)
    plt.close(fig)

In [12]:
def write_catalog_ephem(tic:int, P:float, T0:float, label="Catalog (manual)"):
    import json, os
    os.makedirs("results", exist_ok=True)
    path = f"results/TIC{tic}_catalog_ephem.json"
    json.dump({"P": float(P), "T0": float(T0), "label": label}, open(path,"w"), indent=2)
    print(f"[catalog] wrote {path}  P={P:.8f}  T0={T0:.6f}  ({label})")

In [13]:
def force_catalog_from_tls(tic:int, label="Catalog (tls_top1)"):
    import csv, json, math, os
    p = f"results/TIC{tic}_stitched_TLS_top3.csv"
    if not os.path.exists(p):
        print(f"[catalog] no TLS top-3 for TIC{tic}")
        return
    rows=[]
    with open(p) as f:
        for r in csv.DictReader(f):
            try:
                P=float(r["period_days"])
                T0=float(r.get("T0_BTJD","nan"))
                SDE=float(r.get("SDE","nan"))
                rows.append((P,T0,SDE))
            except: pass
    if not rows:
        print(f"[catalog] TLS file empty for TIC{tic}")
        return
    rows.sort(key=lambda x: (x[2] if math.isfinite(x[2]) else -1), reverse=True)
    P,T0,_=rows[0]
    if not math.isfinite(T0):
        from json import load
        rp=f"results/TIC{tic}_refined_ephemeris.json"
        T0=float(load(open(rp))["T0"]) if os.path.exists(rp) else 0.0
    write_catalog_ephem(tic,P,T0,label)

In [14]:
# Rebuild Figure C panels with explicit catalog prints
for name, info in TARGETS.items():
    tic, toi = info["tic"], info["toi"] 
    # load YOUR refined ephemeris (robust one if you prefer)
    ep_path = f"results/TIC{tic}_refined_ephemeris.json"
    if not os.path.exists(ep_path):
        ep_path = f"results/TIC{tic}_refined_ephemeris_robust.json"
    ep = json.load(open(ep_path))
    yours = {"P": ep["P"], "T0": ep["T0"], "cov": ep["cov"]}

    catalog = load_catalog_ephemeris(tic)
    if catalog is None:
        print(f"[FigureC] {toi} TIC {tic}: no catalog ephemeris found — plotting yours only.")
    else:
        print(f"[FigureC] {toi} TIC {tic}: catalog source={catalog['source']} "
              f"({catalog['label']}); P={catalog['P']:.6f}, T0={catalog['T0']:.6f}")

    # choose sensible date limits from your data span out to 2026-03-31
    # (If you saved stitched times 't', you can use those; here, pick from midtime CSV if present.)
    mid_csv = f"results/TIC{tic}_midtimes.csv"
    if os.path.exists(mid_csv):
        tmin = np.loadtxt(mid_csv, delimiter=",", skiprows=1, usecols=1)
        if np.ndim(tmin) > 0: tmin = np.min(tmin)
        t_min_btjd = float(tmin)
    else:
        # fall back to a few periods around T0
        t_min_btjd = yours["T0"] - 5*yours["P"]
    t_max_btjd = _btjd_from_iso("2026-03-31")

    outpng = f"figures/TIC{tic}_FigureC_window_growth.png"
    plot_window_growth_panels(tic, f"{toi} (Target {name.split()[-1]})", yours, catalog,
                              t_min_btjd, t_max_btjd, outpng)
    print(f"[FigureC] Saved {outpng}")

[FigureC] TOI 1801.01 TIC 119584412: catalog source=TIC119584412_stitched_TLS_top3.csv (Catalog (tls_top1)); P=16.027187, T0=1908.062441
[FigureC] Saved figures/TIC119584412_FigureC_window_growth.png
[FigureC] TOI 260.01 TIC 37749396: catalog source=TIC37749396_stitched_TLS_top3.csv (Catalog (tls_top1)); P=13.475725, T0=1392.311964
[FigureC] Saved figures/TIC37749396_FigureC_window_growth.png
[FigureC] TOI 550.02 TIC 311183180: catalog source=TIC311183180_stitched_TLS_top3.csv (Catalog (tls_top1)); P=9.348442, T0=2149.633067
[FigureC] Saved figures/TIC311183180_FigureC_window_growth.png


In [15]:
# --- Quantify & annotate Figure C gaps at 2026-03-31 ---

import os, json, numpy as np
from astropy.time import Time
BTJD_ZERO = 2457000.0

def sigma_minutes_at(btjd, P, T0, cov):
    # 1σ timing half-width in minutes at an absolute BTJD time
    E = (btjd - T0) / P
    C = np.asarray(cov, float)
    sig_days = np.sqrt(max(C[0,0] + 2*E*C[0,1] + (E**2)*C[1,1], 0.0))
    return float(sig_days * 24 * 60)

t_end = Time("2026-03-31", scale="utc").tdb.jd - BTJD_ZERO

print("Figure C – timing 1σ half-width at 2026-03-31 (minutes)")
print("target,tic,yours_min,catalog_min,delta_min,ΔP_sec,ΔT0_min,cat_source")

for name, info in TARGETS.items():
    tic, toi = info["tic"], info["toi"]
    # yours (robust if present)
    ep_path = f"results/TIC{tic}_refined_ephemeris.json"
    if not os.path.exists(ep_path):
        ep_path = f"results/TIC{tic}_refined_ephemeris_robust.json"
    ep = json.load(open(ep_path))
    yours = {"P": float(ep["P"]), "T0": float(ep["T0"]), "cov": ep["cov"]}

    # catalog (whatever your loader finds)
    cat = load_catalog_ephemeris(tic)
    if cat is None:
        print(f"{name},{tic},(no-cat)")
        continue

    s_y = sigma_minutes_at(t_end, yours["P"], yours["T0"], yours["cov"])
    s_c = sigma_minutes_at(t_end, yours["P"], cat["T0"], yours["cov"])  # reuse your cov as proxy
    dP_sec  = (cat["P"] - yours["P"]) * 86400.0
    dT0_min = (cat["T0"] - yours["T0"]) * 1440.0
    print(f"{name},{tic},{s_y:0.2f},{s_c:0.2f},{(s_c-s_y):+0.2f},{dP_sec:+0.2f},{dT0_min:+0.2f},{cat['source']}")

    # Optional: drop a small text tag onto the bottom panel you just saved
    import matplotlib.pyplot as plt
    from matplotlib.dates import date2num
    fig = plt.imread(f"figures/TIC{tic}_FigureC_window_growth.png")  # cheap: just log numbers; leave image as-is

Figure C – timing 1σ half-width at 2026-03-31 (minutes)
target,tic,yours_min,catalog_min,delta_min,ΔP_sec,ΔT0_min,cat_source
Target A — TIC 119584412,119584412,43.89,43.89,-0.00,-26.99,+23.27,TIC119584412_stitched_TLS_top3.csv
Target B — TIC 37749396,37749396,44.98,44.98,-0.00,-8.55,+8.58,TIC37749396_stitched_TLS_top3.csv
Target C — TIC 311183180,311183180,78.41,78.41,+0.00,-4.24,-0.56,TIC311183180_stitched_TLS_top3.csv


In [16]:
# Odd/even depth tests and secondary-eclipse search (A–C)
import os, json, csv, warnings
import numpy as np
import lightkurve as lk
from astropy.time import Time

warnings.filterwarnings("ignore", category=lk.LightkurveWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# --- Target roster (A–C) ---
TARGETS = {
    "Target A — TIC 119584412": {"tic": 119584412, "toi":"TOI 1801.01", "sectors":[22,49]},
    "Target B — TIC 37749396":  {"tic": 37749396,  "toi":"TOI 260.01",  "sectors":[3,42,70]},
    "Target C — TIC 311183180": {"tic": 311183180, "toi":"TOI 550.02",  "sectors":[5,31]},
}

# --- helpers ---
def load_refined_ephemeris(tic):
    for pth in (f"results/TIC{tic}_refined_ephemeris.json",
                f"results/TIC{tic}_refined_ephemeris_robust.json"):
        if os.path.exists(pth):
            d = json.load(open(pth))
            return float(d["P"]), float(d["T0"])
    raise FileNotFoundError(f"No refined ephemeris JSON for TIC{tic}.")

def load_duration_days_or_default(tic, default_hours=2.0):
    pth = f"results/TIC{tic}_boxfit.json"
    if os.path.exists(pth):
        try:
            d = json.load(open(pth))
            # accept any of these common keys
            for k in ("duration_days","dur_days","duration"):
                if k in d and np.isfinite(d[k]):
                    return float(d[k])
        except Exception:
            pass
    return default_hours/24.0  # fallback if no boxfit

def stitch_pdcsap(tic, sectors):
    lcs = []
    for s in sectors:
        sr = lk.search_lightcurvefile(f"TIC {tic}", mission="TESS", author="SPOC", sector=s)
        if len(sr) == 0:
            continue
        lcf = sr.download()
        lc  = lcf.PDCSAP_FLUX.remove_nans().normalize()
        lcs.append(lc)
    if not lcs:
        return None
    return lk.LightCurveCollection(lcs).stitch().remove_nans().normalize()

def phased_times(t, P, T0):
    # center transit at phase ~0 in range (-0.5, 0.5]
    ph = ((t - T0)/P + 0.5) % 1.0 - 0.5
    # integer epoch index for odd/even split
    k = np.round((t - T0)/P).astype(int)
    return ph, k

def bootstrap_pvalue_two_sample(x, y, nboot=20000, rng=None):
    rng = np.random.default_rng(rng)
    x = np.asarray(x, float); y = np.asarray(y, float)
    x = x[np.isfinite(x)]; y = y[np.isfinite(y)]
    if len(x) < 5 or len(y) < 5:
        return np.nan, (np.nan, np.nan)
    obs = x.mean() - y.mean()
    # center under H0 by subtracting group means then adding grand mean
    gmean = np.r_[x, y].mean()
    x0 = x - x.mean() + gmean
    y0 = y - y.mean() + gmean
    diffs = []
    for _ in range(nboot):
        xb = rng.choice(x0, size=len(x0), replace=True)
        yb = rng.choice(y0, size=len(y0), replace=True)
        diffs.append(xb.mean() - yb.mean())
    diffs = np.asarray(diffs)
    p = (np.abs(diffs) >= np.abs(obs)).mean()
    ci = (np.percentile(diffs, 2.5), np.percentile(diffs, 97.5))
    return float(p), (float(ci[0]), float(ci[1]))

def bootstrap_pvalue_one_sample(x, mu0=0.0, nboot=20000, rng=None):
    rng = np.random.default_rng(rng)
    x = np.asarray(x, float)
    x = x[np.isfinite(x)]
    if len(x) < 5:
        return np.nan, (np.nan, np.nan)
    obs = x.mean() - mu0
    # center under H0
    x0 = x - x.mean() + mu0
    means = []
    for _ in range(nboot):
        xb = rng.choice(x0, size=len(x0), replace=True)
        means.append(xb.mean() - mu0)
    means = np.asarray(means)
    p = (np.abs(means) >= np.abs(obs)).mean()
    ci = (np.percentile(means, 2.5), np.percentile(means, 97.5))
    return float(p), (float(ci[0]), float(ci[1]))

# --- main loop ---
rows = []
os.makedirs("results", exist_ok=True)

print("Odd/Even & Secondary tests (depths in ppm; p-values from bootstrap)")
for name, info in TARGETS.items():
    tic, toi, secs = info["tic"], info["toi"], info["sectors"]
    try:
        P, T0 = load_refined_ephemeris(tic)
    except Exception as e:
        print(f"{name}: {e}")
        continue

    duration_days = load_duration_days_or_default(tic)
    halfw = 0.6 * duration_days  # a touch wider than the box-fit to be safe

    lc = stitch_pdcsap(tic, secs)
    if lc is None or len(lc.time.value) < 100:
        print(f"{name}: no usable PDCSAP.")
        continue

    t = lc.time.value          # BTJD
    f = lc.flux.value          # ~normalized to 1
    # simple out-of-transit baseline via median outside ±3*duration
    ph, k = phased_times(t, P, T0)
    oot = np.abs(ph) > max(3*duration_days, 0.08)  # keep OOT away from transit
    if np.sum(oot) > 20:
        f = f / np.median(f[oot])

    depth_ppm = (1.0 - f) * 1e6

    # Primary in-transit mask
    in_transit = np.abs(ph) <= halfw
    # Split odd/even by epoch index k
    odd_mask  = in_transit & (k % 2 != 0)
    even_mask = in_transit & (k % 2 == 0)

    # Secondary window near phase +0.5 (also check around -0.5 wrap)
    sec = (np.abs(ph - 0.5) <= halfw) | (np.abs(ph + 0.5) <= halfw)

    # Compute summary numbers
    d_odd  = depth_ppm[odd_mask]
    d_even = depth_ppm[even_mask]
    d_sec  = depth_ppm[sec]

    p_odd_even, ci_oe = bootstrap_pvalue_two_sample(d_odd, d_even)
    p_secondary,  ci_sec = bootstrap_pvalue_one_sample(d_sec, mu0=0.0)

    mean_odd  = float(np.nanmean(d_odd))  if d_odd.size  else np.nan
    mean_even = float(np.nanmean(d_even)) if d_even.size else np.nan
    mean_sec  = float(np.nanmean(d_sec))  if d_sec.size  else np.nan

    rows.append({
        "target": name, "toi": toi, "tic": tic,
        "P_days": P, "T0_BTJD": T0,
        "duration_days_used": duration_days,
        "N_in_odd": int(d_odd.size), "N_in_even": int(d_even.size), "N_in_sec": int(d_sec.size),
        "mean_depth_odd_ppm": mean_odd, "mean_depth_even_ppm": mean_even,
        "delta_odd_minus_even_ppm": (mean_odd - mean_even) if np.isfinite(mean_odd) and np.isfinite(mean_even) else np.nan,
        "pvalue_odd_vs_even": p_odd_even, "ci_odd_even_ppm": ci_oe,
        "mean_secondary_depth_ppm": mean_sec, "pvalue_secondary_vs_zero": p_secondary, "ci_secondary_ppm": ci_sec
    })

    # Console summary
    print(f"\n{name} — {toi} (TIC {tic})")
    print(f"  P={P:.8f} d, T0={T0:.6f} BTJD, duration_used≈{24*duration_days:.2f} h")
    print(f"  In-transit points: odd={d_odd.size}, even={d_even.size}; secondary window={d_sec.size}")
    print(f"  Odd mean depth ≈ {mean_odd:,.0f} ppm, Even ≈ {mean_even:,.0f} ppm "
          f"→ Δ(odd-even) ≈ {(mean_odd-mean_even):+.0f} ppm;  p={p_odd_even:0.3f}")
    print(f"  Secondary mean depth ≈ {mean_sec:,.0f} ppm vs 0;  p={p_secondary:0.3f}")

# Write CSV
outcsv = "results/odd_even_secondary_summary.csv"
with open(outcsv, "w", newline="") as f:
    cols = list(rows[0].keys())
    w = csv.DictWriter(f, fieldnames=cols)
    w.writeheader(); w.writerows(rows)
print(f"\nSaved {outcsv}")

Odd/Even & Secondary tests (depths in ppm; p-values from bootstrap)

Target A — TIC 119584412 — TOI 1801.01 (TIC 119584412)
  P=16.02749976 d, T0=1908.046283 BTJD, duration_used≈6.53 h
  In-transit points: odd=7387, even=7486; secondary window=3518
  Odd mean depth ≈ 414 ppm, Even ≈ 439 ppm → Δ(odd-even) ≈ -25 ppm;  p=0.232
  Secondary mean depth ≈ -547 ppm vs 0;  p=0.000



Target B — TIC 37749396 — TOI 260.01 (TIC 37749396)
  P=13.47582381 d, T0=1392.306006 BTJD, duration_used≈4.42 h
  In-transit points: odd=14243, even=16554; secondary window=22718
  Odd mean depth ≈ -59 ppm, Even ≈ 13 ppm → Δ(odd-even) ≈ -72 ppm;  p=0.000
  Secondary mean depth ≈ 20 ppm vs 0;  p=0.002

Target C — TIC 311183180 — TOI 550.02 (TIC 311183180)
  P=9.34849077 d, T0=2149.633454 BTJD, duration_used≈5.72 h
  In-transit points: odd=2965, even=7459; secondary window=10769
  Odd mean depth ≈ 2,210 ppm, Even ≈ 47 ppm → Δ(odd-even) ≈ +2163 ppm;  p=0.000
  Secondary mean depth ≈ -191 ppm vs 0;  p=0.000

Saved results/odd_even_secondary_summary.csv


In [17]:
import os, json, numpy as np
import matplotlib.pyplot as plt
from scipy import stats

def load_ephem(tic):
    p = f"results/TIC{tic}_refined_ephemeris.json"
    if not os.path.exists(p):
        p = f"results/TIC{tic}_refined_ephemeris_robust.json"
    d = json.load(open(p))
    return float(d["P"]), float(d["T0"]), np.array(d["cov"], float)

def in_transit_mask(t, P, T0, dur_hours, phase_center=0.0):
    # returns boolean mask +- (dur/2) around phase_center
    phase = ((t - T0) / P) % 1.0
    dur_days = dur_hours/24.0
    half = 0.5*dur_days/P
    # wrap-safe distance to phase_center and (phase_center+1)
    dphi = np.minimum(np.abs(phase - phase_center), np.abs(phase - phase_center - 1))
    return dphi < half

def odd_even_masks(t, P, T0, dur_hours):
    k = np.round((t - T0)/P).astype(int)  # nearest epoch index
    it = in_transit_mask(t, P, T0, dur_hours, 0.0)
    odd = it & (k % 2 != 0)
    even = it & (k % 2 == 0)
    return odd, even

def sigma_minutes_at(btjd, P, T0, cov):
    # 1σ timing half-width in minutes at an absolute BTJD time
    E = (btjd - T0) / P
    c = np.asarray(cov, float)
    sig_days = np.sqrt(max(c[0,0] + 2*E*c[0,1] + (E**2)*c[1,1], 0.0))
    return float(sig_days * 24 * 60)

In [19]:
# --- Quick stitched PDCSAP helper (drop-in) ---
import os, numpy as np
import lightkurve as lk

QUALITY_MASK = 175  # same as before

SECTORS = {
    119584412: [22, 49],      # Target A
    37749396:  [3, 42, 70],   # Target B
    311183180: [5, 31],       # Target C
}

def get_stitched_pdcsap(tic, sectors=None, save_npz=True):
    """Return BTJD time and normalized PDCSAP flux for the given TIC."""
    if sectors is None:
        sectors = SECTORS.get(int(tic), None)

    lcs = []
    if sectors:
        # deterministic order
        for s in sectors:
            sr = lk.search_lightcurvefile(f"TIC {int(tic)}", mission="TESS", author="SPOC", sector=s)
            if len(sr) == 0:
                continue
            lcf = sr.download()  # deprecation warning is OK in this env
            lc = lcf.PDCSAP_FLUX.remove_nans().normalize()
            lc = lc.remove_outliers(sigma=10)  # gentle; keeps transits
            mask = lk.utils.TessQualityFlags.create_quality_mask(lc.quality, bitmask=QUALITY_MASK)
            lc = lc[mask]
            lcs.append(lc)
    else:
        # fallback: grab everything available (slower, but robust)
        sr = lk.search_lightcurvefile(f"TIC {int(tic)}", mission="TESS", author="SPOC")
        for r in sr:
            lcf = r.download()
            lc = lcf.PDCSAP_FLUX.remove_nans().normalize()
            lc = lc.remove_outliers(sigma=10)
            mask = lk.utils.TessQualityFlags.create_quality_mask(lc.quality, bitmask=QUALITY_MASK)
            lc = lc[mask]
            lcs.append(lc)

    if not lcs:
        raise RuntimeError(f"No PDCSAP data found for TIC {tic} with sectors={sectors}")

    stitched = lk.LightCurveCollection(lcs).stitch().remove_nans()
    t = np.asarray(stitched.time.value, dtype=float)   # BTJD
    f = np.asarray(stitched.flux.value, dtype=float)   # normalized

    if save_npz:
        os.makedirs("results", exist_ok=True)
        np.savez_compressed(f"results/TIC{int(tic)}_stitched.npz", t=t, f=f)

    return t, f

# If you keep the original loader name elsewhere, you can shim it:
def load_stitched_npz(tic):
    path = f"results/TIC{int(tic)}_stitched.npz"
    if os.path.exists(path):
        d = np.load(path)
        return d["t"].astype(float), d["f"].astype(float)
    # otherwise build it now:
    return get_stitched_pdcsap(tic, save_npz=True)

In [20]:
# You already have a stitched, detrended, normalized time series per target.
# Reuse whatever arrays you built earlier: t (BTJD), f (normalized flux ~1).
# If you don't have them in memory, load them the same way you did for Fig A panels.

TARGETS_SIMPLE = {
    "A": {"tic":119584412, "toi":"TOI 1801.01", "dur_h":6.53},
    "B": {"tic":37749396,  "toi":"TOI 260.01",  "dur_h":4.42},
    "C": {"tic":311183180, "toi":"TOI 550.02",  "dur_h":5.72},
}

def load_stitched_npz(tic):
    # If you saved an npz; otherwise replace with your existing loader.
    # Expect keys: 't','f'. If not using npz, bring t,f from your earlier cell.
    path = f"results/TIC{tic}_stitched_flat.npz"
    if os.path.exists(path):
        d = np.load(path)
        return d["t"].astype(float), d["f"].astype(float)
    raise FileNotFoundError("Provide t,f from your stitched light curve here.")

for name, info in TARGETS_SIMPLE.items():
    tic, toi, dur_h = info["tic"], info["toi"], info["dur_h"]
    P,T0,C = load_ephem(tic)

    # === Replace this with your already-loaded arrays ===
    # t, f = <your stitched BTJD, normalized flux>
    # If you don't have an npz, comment the next line and set t,f from memory:
    # For Target A only:
    t, f = get_stitched_pdcsap(119584412, sectors=[22, 49], save_npz=True)

    # Masks
    odd, even = odd_even_masks(t, P, T0, dur_h)
    sec = in_transit_mask(t, P, T0, dur_h, phase_center=0.5)

    # Welch tests (two-sided)
    to = stats.ttest_ind(f[odd],  f[even], equal_var=False, nan_policy="omit")
    ts = stats.ttest_1samp(f[sec]-1.0, popmean=0.0, nan_policy="omit")  # mean offset vs 0

    print(f"\n{name} — {toi} (TIC {tic})")
    print(f"  odd N={odd.sum()}, even N={even.sum()}, sec N={sec.sum()}")
    print(f"  Welch t-test odd vs even:   t={to.statistic: .3f}, p={to.pvalue: .3g}")
    print(f"  t-test secondary vs zero:   t={ts.statistic: .3f}, p={ts.pvalue: .3g}")


A — TOI 1801.01 (TIC 119584412)
  odd N=392, even N=392, sec N=179
  Welch t-test odd vs even:   t=-7.573, p= 1.04e-13
  t-test secondary vs zero:   t= 4.554, p= 9.72e-06

B — TOI 260.01 (TIC 37749396)
  odd N=266, even N=266, sec N=265
  Welch t-test odd vs even:   t=-10.386, p= 4.45e-23
  t-test secondary vs zero:   t=-9.437, p= 2.1e-18

C — TOI 550.02 (TIC 311183180)
  odd N=342, even N=341, sec N=939
  Welch t-test odd vs even:   t= 0.310, p= 0.757
  t-test secondary vs zero:   t= 8.337, p= 2.7e-16


In [24]:
# --- Quick helper: get stitched PDCSAP arrays (gentle, reproducible) ---
import numpy as np, os
import lightkurve as lk

def get_stitched_pdcsap(tic, sectors=None, qmask=175, window_days=1.0, polyorder=2,
                        save_npz=False):
    """Return (t, f) for a TIC from SPOC PDCSAP, gently detrended & stitched."""
    lcs = []
    if sectors is None:
        sr = lk.search_lightcurvefile(f"TIC {int(tic)}", mission="TESS", author="SPOC")
        lcfs = sr.download_all()
    else:
        lcfs = []
        for s in sectors:
            sr = lk.search_lightcurvefile(f"TIC {int(tic)}", mission="TESS",
                                          author="SPOC", sector=s)
            lcf = sr.download()
            lcfs.append(lcf)

    for lcf in lcfs:
        lc = lcf.PDCSAP_FLUX.remove_nans().normalize()
        # approximate cadence in days
        dt = np.nanmedian(np.diff(lc.time.value))
        win = max(7, int(window_days / dt))  # window length in cadences
        lc = lc.remove_outliers(sigma=5)
        lc = lc.flatten(window_length=win, polyorder=polyorder)
        lcs.append(lc)

    stitched = lk.LightCurveCollection(lcs).stitch().remove_nans().normalize()
    t = stitched.time.value.astype(float)  # BTJD
    f = stitched.flux.value.astype(float)  # normalized

    if save_npz:
        os.makedirs("results_npz", exist_ok=True)
        np.savez(f"results_npz/TIC{int(tic)}_stitched.npz", t=t, f=f)

    return t, f

In [26]:
rng = np.random.default_rng(42)

def robust_depth(samples, trim=0.2):
    # depth = 1 - statistic (so positive = dimmer in-transit)
    med = 1 - np.nanmedian(samples)
    n = samples.size
    k = int(np.floor(trim*n))
    if 2*k < n:
        sm = np.sort(samples[~np.isnan(samples)])
        trimmed = sm[k: n-k] if n-2*k>0 else sm
        tmean = 1 - np.mean(trimmed)
    else:
        tmean = med
    return med, tmean

def bootstrap_ci(samples, B=2000, statfunc=np.nanmean):
    n = samples.size
    boots = []
    for _ in range(B):
        idx = rng.integers(0, n, size=n)
        boots.append(statfunc(samples[idx]))
    lo, hi = np.percentile(boots, [16,84])
    return float(lo), float(hi)

for name, info in TARGETS_SIMPLE.items():
    tic, toi, dur_h = info["tic"], info["toi"], info["dur_h"]
    P,T0,C = load_ephem(tic)
    # t, f = ... (bring from memory or loader)
    # A
    tic = 119584412
    t, f = get_stitched_pdcsap(tic, sectors=[22, 49], window_days=1.0)
    
    # B
    tic = 37749396
    t, f = get_stitched_pdcsap(tic, sectors=[3, 42, 70], window_days=0.75)
    
    # C
    tic = 311183180
    t, f = get_stitched_pdcsap(tic, sectors=[5, 31], window_days=0.75)

    odd, even = odd_even_masks(t, P, T0, dur_h)
    sec = in_transit_mask(t, P, T0, dur_h, 0.5)
    oot = ~(odd|even|sec)

    d_odd_med,  d_odd_trim  = robust_depth(1-f[odd])
    d_even_med, d_even_trim = robust_depth(1-f[even])
    d_sec_med,  d_sec_trim  = robust_depth(1-f[sec])

    # CIs on mean depths in ppm
    lo_o, hi_o = bootstrap_ci(1-f[odd],  statfunc=np.nanmean)
    lo_e, hi_e = bootstrap_ci(1-f[even], statfunc=np.nanmean)
    lo_s, hi_s = bootstrap_ci(1-f[sec],  statfunc=np.nanmean)

    print(f"\n{name} — {toi} (TIC {tic}) robust depths [ppm]:")
    print(f"  odd:  median={1e6*d_odd_med:6.0f}, trimmed={1e6*d_odd_trim:6.0f}, mean_CI=({1e6*lo_o:6.0f},{1e6*hi_o:6.0f})")
    print(f"  even: median={1e6*d_even_med:6.0f}, trimmed={1e6*d_even_trim:6.0f}, mean_CI=({1e6*lo_e:6.0f},{1e6*hi_e:6.0f})")
    print(f"  sec:  median={1e6*d_sec_med:6.0f},  trimmed={1e6*d_sec_trim:6.0f},  mean_CI=({1e6*lo_s:6.0f},{1e6*hi_s:6.0f})")


A — TOI 1801.01 (TIC 311183180) robust depths [ppm]:
  odd:  median=1000018, trimmed=1000011, mean_CI=(   -66,     9)
  even: median=1000001, trimmed=999987, mean_CI=(    -5,    70)
  sec:  median=    ———,  trimmed=    ———,  mean_CI=(   nan,   nan)



B — TOI 260.01 (TIC 311183180) robust depths [ppm]:
  odd:  median=1000022, trimmed=1000038, mean_CI=(   -79,    -0)
  even: median=1000124, trimmed=1000110, mean_CI=(  -162,   -73)
  sec:  median=999981,  trimmed=999978,  mean_CI=(   276,   435)



C — TOI 550.02 (TIC 311183180) robust depths [ppm]:
  odd:  median=999880, trimmed=999878, mean_CI=(    62,   144)
  even: median=999849, trimmed=999784, mean_CI=(   873,  1107)
  sec:  median=1000048,  trimmed=1000028,  mean_CI=(   -49,    -7)


In [27]:
def quick_phase_plot(t, f, P, T0, masks, labels, outpng):
    phase = ((t - T0)/P) % 1.0
    fig, ax = plt.subplots(figsize=(7.5,4.0), dpi=140)
    for m,lab in zip(masks, labels):
        ax.scatter(phase[m], f[m], s=2, alpha=0.25, label=lab)
    ax.set_xlabel("Phase"); ax.set_ylabel("Flux (norm)")
    ax.set_title(os.path.basename(outpng).replace("_"," "))
    ax.legend(markerscale=4, frameon=False)
    fig.tight_layout(); fig.savefig(outpng); plt.close(fig)

os.makedirs("figures", exist_ok=True)
for name, info in TARGETS_SIMPLE.items():
    tic, toi, dur_h = info["tic"], info["toi"], info["dur_h"]
    P,T0,C = load_ephem(tic)
    # t, f = ...
    # A
    tic = 119584412
    t, f = get_stitched_pdcsap(tic, sectors=[22, 49], window_days=1.0)
    
    # B
    tic = 37749396
    t, f = get_stitched_pdcsap(tic, sectors=[3, 42, 70], window_days=0.75)
    
    # C
    tic = 311183180
    t, f = get_stitched_pdcsap(tic, sectors=[5, 31], window_days=0.75)
    odd, even = odd_even_masks(t, P, T0, dur_h)
    sec = in_transit_mask(t, P, T0, dur_h, 0.5)
    out1 = f"figures/TIC{tic}_phase_odd_even.png"
    out2 = f"figures/TIC{tic}_phase_secondary.png"
    quick_phase_plot(t, f, P, T0, [odd,even], ["odd","even"], out1)
    quick_phase_plot(t, f, P, T0, [sec], ["secondary"], out2)
    print("saved", out1, "and", out2)

saved figures/TIC311183180_phase_odd_even.png and figures/TIC311183180_phase_secondary.png


saved figures/TIC311183180_phase_odd_even.png and figures/TIC311183180_phase_secondary.png


saved figures/TIC311183180_phase_odd_even.png and figures/TIC311183180_phase_secondary.png


In [28]:
import csv
def sector_depths_table(t, f, sectors, P, T0, dur_h, outcsv, outpng):
    # sectors: array same length as t with sector numbers
    uniq = np.unique(sectors)
    rows = []
    for s in uniq:
        m = sectors == s
        odd, even = odd_even_masks(t[m], P, T0, dur_h)
        it = odd | even
        if it.sum() < 20:
            rows.append((s, np.nan, np.nan, it.sum()))
            continue
        d = 1 - np.nanmean(f[m][it])
        rows.append((s, d*1e6, np.nanstd(f[m][it])*1e6/np.sqrt(max(it.sum(),1)), it.sum()))
    rows.sort()
    with open(outcsv,"w",newline="") as g:
        w=csv.writer(g); w.writerow(["sector","depth_ppm","sem_ppm","N_in_transit"]); w.writerows(rows)
    # quick plot
    S = [r[0] for r in rows]; D = [r[1] for r in rows]; E=[r[2] for r in rows]
    fig,ax=plt.subplots(figsize=(7,3.5),dpi=140)
    ax.errorbar(S,D,yerr=E,fmt='o-',capsize=2)
    ax.set_xlabel("Sector"); ax.set_ylabel("Depth (ppm)"); ax.set_title(os.path.basename(outpng))
    fig.tight_layout(); fig.savefig(outpng); plt.close(fig)

# If you already track sector per-point, pass it here; otherwise skip this block.
# Example usage (uncomment when you have 'sectors' array aligned with t,f):
# for name, info in TARGETS_SIMPLE.items():
#     tic, toi, dur_h = info["tic"], info["toi"], info["dur_h"]
#     P,T0,C = load_ephem(tic)
#     t, f, sectors = ...  # provide your sector array here
#     sector_depths_table(t,f,sectors,P,T0,dur_h,
#                         f"results/TIC{tic}_per_sector_depths.csv",
#                         f"figures/TIC{tic}_per_sector_depths.png")

In [30]:
# --- Minimal PDCSAP stitcher for saving NPZs (drop this above the NPZ cell) ---
import numpy as np, warnings
from lightkurve import search_lightcurvefile, LightCurveCollection

def load_stitched_lightcurve_pdcsap(tic, quality_bitmask=175):
    """
    Returns
        t : BTJD (float64)
        f : normalized PDCSAP flux (float64)
    Notes
        - PDCSAP first, all available SPOC sectors.
        - Applies quality bitmask (default 175).
        - Gentle clean: remove NaNs, 5-sigma outlier clip per sector, normalize, then stitch.
    """
    # download_all() is robust and works across sectors; suppress noisy warnings
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        lcfs = search_lightcurvefile(f"TIC {tic}", author="SPOC").download_all()

    if lcfs is None or len(lcfs) == 0:
        raise FileNotFoundError(f"No SPOC LightCurveFiles found for TIC {tic}")

    per_sector = []
    for lcf in lcfs:
        # Apply quality mask on the underlying light curve file
        lc = lcf.PDCSAP_FLUX
        if quality_bitmask is not None and "quality" in lc.columns:
            q = (lcf.quality & quality_bitmask) == 0
            # Some Lightkurve versions require slicing via .copy()
            lc = lc[q].copy()
        # Clean + normalize per sector
        lc = lc.remove_nans().remove_outliers(sigma=5)
        lc = lc.normalize()
        if len(lc.time.value) > 0:
            per_sector.append(lc)

    if len(per_sector) == 0:
        raise RuntimeError(f"All sectors for TIC {tic} were empty after masking/cleaning.")

    stitched = LightCurveCollection(per_sector).stitch()
    return stitched.time.value.astype(float), stitched.flux.value.astype(float)

In [31]:
# ---- RUN ONCE to create stitched arrays for A & B if missing ----
import os, numpy as np, json

TARGETS = {
    "Target A — TIC 119584412": {"tic": 119584412, "toi": "TOI 1801.01"},
    "Target B — TIC 37749396":  {"tic": 37749396,  "toi": "TOI 260.01"},
    "Target C — TIC 311183180": {"tic": 311183180, "toi": "TOI 550.02"},
}

os.makedirs("results", exist_ok=True)

def have_npz(tic): return os.path.exists(f"results/TIC{tic}_stitched.npz")

def save_npz_if_missing(tic):
    if have_npz(tic): return
    # replace this loader with your PDCSAP stitcher used elsewhere
    t, f = load_stitched_lightcurve_pdcsap(tic)  # <-- your existing function
    np.savez(f"results/TIC{tic}_stitched.npz", t=np.asarray(t, float), f=np.asarray(f, float))
    print(f"[make_npz] wrote results/TIC{tic}_stitched.npz")

for info in TARGETS.values():
    try:
        save_npz_if_missing(info["tic"])
    except NameError:
        print("Define load_stitched_lightcurve_pdcsap(tic) or swap in your own t,f here.")
        raise

[make_npz] wrote results/TIC37749396_stitched.npz
[make_npz] wrote results/TIC311183180_stitched.npz


In [32]:
# === Odd/Even + Secondary significance & plots for A and B ===
import os, json, numpy as np
from astropy.time import Time
from scipy import stats
import matplotlib.pyplot as plt

BTJD_ZERO = 2457000.0

TARGETS = {
    "Target A — TIC 119584412": {"tic": 119584412, "toi": "TOI 1801.01", "dur_h": 6.53},
    "Target B — TIC 37749396":  {"tic": 37749396,  "toi": "TOI 260.01", "dur_h": 4.42},
}

def load_ephem(tic):
    # prefer robust if you’ve got both
    for name in [f"results/TIC{tic}_refined_ephemeris_robust.json",
                 f"results/TIC{tic}_refined_ephemeris.json"]:
        if os.path.exists(name):
            d = json.load(open(name))
            return float(d["P"]), float(d["T0"]), np.array(d["cov"], float)
    raise FileNotFoundError(f"No ephemeris JSON for TIC {tic}")

def load_stitched_npz(tic):
    path = f"results/TIC{tic}_stitched.npz"
    d = np.load(path)
    return d["t"].astype(float), d["f"].astype(float)

def phase_fold(t, P, T0):  # [0,1)
    return np.mod((t - T0)/P, 1.0)

def in_window(ph, center, half_width):
    # half_width in phase units
    d = np.abs((ph - center + 0.5) % 1.0 - 0.5)
    return d <= half_width

def odd_even_masks(t, P, T0, dur_h):
    ph = phase_fold(t, P, T0)
    # convert duration to phase half-width; scale a bit wider (×1.2) to be safe
    halfw = (dur_h/24.0)/P * 1.2
    base = in_window(ph, 0.0, halfw)
    # assign epochs: nearest integer k to phase offset
    k = np.floor((t - T0)/P + 0.5).astype(int)
    return base & (k % 2 == 1), base & (k % 2 == 0)

def secondary_mask(t, P, T0, dur_h):
    ph = phase_fold(t, P, T0)
    halfw = (dur_h/24.0)/P * 1.2
    return in_window(ph, 0.5, halfw)

def trimmed_mean_ci(x, trim=0.1, alpha=0.05):
    x = np.asarray(x, float)
    n = x.size
    if n < 8:  # too few points
        return np.nan, np.nan, (np.nan, np.nan)
    # trim both tails
    k = int(np.floor(trim*n))
    xs = np.sort(x)[k:n-k] if 2*k < n else np.sort(x)
    m = xs.mean()
    # bootstrap CI
    rng = np.random.default_rng(42)
    B = 2000
    means = np.empty(B)
    for i in range(B):
        means[i] = xs[rng.integers(0, xs.size, xs.size)].mean()
    lo, hi = np.percentile(means, [100*alpha/2, 100*(1-alpha/2)])
    return np.median(x), m, (lo, hi)

def welch_t_p(x, y):
    t, p = stats.ttest_ind(x, y, equal_var=False, nan_policy="omit")
    return float(t), float(p)

def one_sample_t_p(x, mu=1.0):
    t, p = stats.ttest_1samp(x, popmean=mu, nan_policy="omit")
    return float(t), float(p)

os.makedirs("figures", exist_ok=True)

for label, info in TARGETS.items():
    tic, toi, dur_h = info["tic"], info["toi"], info["dur_h"]
    P, T0, C = load_ephem(tic)
    t, f = load_stitched_npz(tic)

    odd_m, even_m = odd_even_masks(t, P, T0, dur_h)
    sec_m = secondary_mask(t, P, T0, dur_h)

    f_odd, f_even, f_sec = f[odd_m], f[even_m], f[sec_m]

    # Welch t-tests (flux normalized ~1; “depth” = 1 - mean flux)
    t_oe, p_oe = welch_t_p(f_odd, f_even)
    t_sec, p_sec = one_sample_t_p(f_sec, mu=1.0)

    # Robust summaries in ppm (1e6*(1 - value))
    def depths_ppm(arr):
        med, trim_mean, (lo, hi) = trimmed_mean_ci(1e6*(1.0 - arr))
        return med, trim_mean, (lo, hi)

    d_odd = depths_ppm(f_odd)
    d_even = depths_ppm(f_even)
    d_sec = depths_ppm(f_sec)

    print(f"{toi} — {label.split('—')[-1].strip()}")
    print(f"  odd N={f_odd.size}, even N={f_even.size}, sec N={f_sec.size}")
    print(f"  Welch t-test odd vs even:   t={t_oe:7.3f}, p={p_oe:.2e}")
    print(f"  one-sample t-test (sec vs 0 depth): t={t_sec:7.3f}, p={p_sec:.2e}")
    print(f"  odd depth ppm:  median={d_odd[0]:.0f}, trimmed={d_odd[1]:.0f}, CI={d_odd[2]}")
    print(f"  even depth ppm: median={d_even[0]:.0f}, trimmed={d_even[1]:.0f}, CI={d_even[2]}")
    print(f"  sec depth ppm:  median={d_sec[0]:.0f}, trimmed={d_sec[1]:.0f}, CI={d_sec[2]}")
    print("")

    # Phase plots
    ph = phase_fold(t, P, T0)
    # Odd/even scatter near primary
    plt.figure(figsize=(9,4.6), dpi=120)
    h1 = plt.scatter(ph[odd_m],  f[odd_m],  s=8, alpha=0.35, label="odd")
    h2 = plt.scatter(ph[even_m], f[even_m], s=8, alpha=0.35, label="even")
    plt.legend()
    plt.xlabel("Phase"); plt.ylabel("Flux (norm)")
    plt.title(f"TIC{tic} phase odd/even")
    plt.xlim(-0.02, 0.02); plt.ylim(0.985, 1.005)
    plt.savefig(f"figures/TIC{tic}_phase_odd_even.png"); plt.close()

    # Secondary scatter around 0.5
    plt.figure(figsize=(9,4.6), dpi=120)
    plt.scatter(ph[sec_m], f[sec_m], s=8, alpha=0.35, label="secondary")
    plt.legend(); plt.xlabel("Phase"); plt.ylabel("Flux (norm)")
    plt.title(f"TIC{tic} phase secondary")
    plt.xlim(0.49, 0.51); plt.ylim(0.998, 1.004)
    plt.savefig(f"figures/TIC{tic}_phase_secondary.png"); plt.close()

TOI 1801.01 — TIC 119584412
  odd N=938, even N=939, sec N=317
  Welch t-test odd vs even:   t= -9.715, p=8.46e-22
  one-sample t-test (sec vs 0 depth): t=  6.373, p=6.55e-10
  odd depth ppm:  median=1324, trimmed=1320, CI=(1273.518101252774, 1365.6629705207145)
  even depth ppm: median=855, trimmed=849, CI=(802.0458369457706, 892.4205543827092)
  sec depth ppm:  median=-375, trimmed=-445, CI=(-546.6203596077713, -345.15029075098977)

TOI 260.01 — TIC 37749396
  odd N=2536, even N=2839, sec N=3179
  Welch t-test odd vs even:   t=  2.736, p=6.24e-03
  one-sample t-test (sec vs 0 depth): t= -3.143, p=1.69e-03
  odd depth ppm:  median=96, trimmed=71, CI=(44.34923673498219, 97.56858052291307)
  even depth ppm: median=103, trimmed=127, CI=(104.19215825072901, 150.1279108282023)
  sec depth ppm:  median=65, trimmed=56, CI=(36.06788770154792, 74.81552762929017)



In [33]:
# === PHASE PLOTS with BINNED OVERLAY (drop this in as one cell) ===
import os, json, numpy as np
import matplotlib.pyplot as plt

os.makedirs("figures", exist_ok=True)
BTJD_ZERO = 2457000.0  # (not used here, just for consistency)

# --------- pick target (edit these 3 lines per run) ----------
tic, toi = 119584412, "TOI 1801.01"     # A  -> TIC 119584412
# tic, toi = 37749396,  "TOI 260.01"    # B  -> TIC 37749396
# tic, toi = 311183180, "TOI 550.02"    # C  -> TIC 311183180
dur_hours = 6.0  # (A≈6–7 h, B≈4–5 h, C≈5–6 h). OK if approximate.

# --------- helpers (no other cells required) ----------
def load_ephem(tic):
    for suffix in ("", "_robust"):
        p = f"results/TIC{tic}_refined_ephemeris{suffix}.json"
        if os.path.exists(p):
            d = json.load(open(p))
            return float(d["P"]), float(d["T0"])
    raise FileNotFoundError(f"refined ephemeris JSON for TIC{tic} not found")

def load_stitched_npz(tic):
    p = f"results/TIC{tic}_stitched.npz"
    d = np.load(p)
    return d["t"].astype(float), d["f"].astype(float)

def wrap_phase(t, P, T0):
    # phase in [-0.5, 0.5)
    ph = ((t - T0) / P) % 1.0
    ph[ph >= 0.5] -= 1.0
    return ph

def in_window(t, P, T0, halfdur_days, center_phase=0.0):
    # distance in phase from chosen center (0 for primary, 0.5 for secondary)
    # work in [-0.5,0.5)
    ph = wrap_phase(t, P, T0)
    if center_phase == 0.5:
        d = np.abs((ph - 0.5 + 0.5) % 1.0 - 0.5)
    else:
        d = np.abs(ph)
    return d < (halfdur_days / P)

def odd_even_masks(t, P, T0, halfdur_days):
    # integer epoch index closest to each point
    k = np.rint((t - T0) / P).astype(int)
    w = in_window(t, P, T0, halfdur_days, center_phase=0.0)
    odd  = w & (k % 2 != 0)
    even = w & (k % 2 == 0)
    return odd, even

def bin_xy(x, y, nbins=80):
    # mean & SEM per bin
    edges = np.linspace(x.min(), x.max(), nbins+1)
    idx = np.digitize(x, edges) - 1
    xm = 0.5*(edges[:-1] + edges[1:])
    ymean, ysem = np.full(nbins, np.nan), np.full(nbins, np.nan)
    for i in range(nbins):
        m = idx == i
        if np.any(m):
            yy = y[m]
            ymean[i] = np.nanmean(yy)
            ysem[i]  = np.nanstd(yy) / np.sqrt(np.sum(np.isfinite(yy)))
    return xm, ymean, ysem

# --------- load data & make plots ----------
P, T0 = load_ephem(tic)
t, f  = load_stitched_npz(tic)

dur_days = dur_hours / 24.0
halfdur  = 0.5 * dur_days

ph = wrap_phase(t, P, T0)

odd_m, even_m = odd_even_masks(t, P, T0, halfdur)
sec_m = in_window(t, P, T0, halfdur, center_phase=0.5)

# sensible y-lims around the in-window scatter
y0 = np.nanmedian(f[odd_m | even_m])
ylim = (y0 - 0.004, y0 + 0.004)  # ~±0.4% window; tweak if needed

# ---- 1) Odd/Even primary window plot with binned overlay
plt.figure(figsize=(10,5.2), dpi=120)
plt.scatter(ph[odd_m],  f[odd_m],  s=9, alpha=0.25, label="odd")
plt.scatter(ph[even_m], f[even_m], s=9, alpha=0.25, label="even")
# binned means + 1σ(SEM) ribbons
for lbl, m, c in [("odd", odd_m, "#1f77b4"), ("even", even_m, "#ff7f0e")]:
    xb, ym, ys = bin_xy(ph[m], f[m], nbins=60)
    ok = np.isfinite(ym)
    plt.plot(xb[ok], ym[ok], lw=2, color=c, label=f"{lbl} (binned)")
    if np.any(ok & np.isfinite(ys)):
        plt.fill_between(xb[ok], ym[ok]-ys[ok], ym[ok]+ys[ok], alpha=0.15, color=c)
plt.xlim(-0.02, 0.02)
plt.ylim(*ylim)
plt.xlabel("Phase")
plt.ylabel("Flux (norm)")
plt.title(f"TIC{tic} phase odd/even — {toi}")
plt.legend(loc="upper left")
out1 = f"figures/TIC{tic}_phase_odd_even.png"
plt.savefig(out1, bbox_inches="tight")
plt.close()

# ---- 2) Secondary window plot with binned overlay
# convert to [0,1) for a tight zoom around 0.5
ph01 = (ph + 1.0) % 1.0
plt.figure(figsize=(10,5.2), dpi=120)
plt.scatter(ph01[sec_m], f[sec_m], s=10, alpha=0.35, label="secondary")
xb, ym, ys = bin_xy(ph01[sec_m], f[sec_m], nbins=50)
ok = np.isfinite(ym)
plt.plot(xb[ok], ym[ok], lw=2, label="binned")
if np.any(ok & np.isfinite(ys)):
    plt.fill_between(xb[ok], ym[ok]-ys[ok], ym[ok]+ys[ok], alpha=0.2)
plt.xlim(0.49, 0.51)
plt.ylim(*ylim)
plt.xlabel("Phase")
plt.ylabel("Flux (norm)")
plt.title(f"TIC{tic} phase secondary — {toi}")
plt.legend(loc="upper right")
out2 = f"figures/TIC{tic}_phase_secondary.png"
plt.savefig(out2, bbox_inches="tight")
plt.close()

print(f"Saved {out1} and {out2}")

Saved figures/TIC119584412_phase_odd_even.png and figures/TIC119584412_phase_secondary.png


In [34]:
# === PHASE PLOTS with BINNED OVERLAY (drop this in as one cell) ===
import os, json, numpy as np
import matplotlib.pyplot as plt

os.makedirs("figures", exist_ok=True)
BTJD_ZERO = 2457000.0  # (not used here, just for consistency)

# --------- pick target (edit these 3 lines per run) ----------
#tic, toi = 119584412, "TOI 1801.01"     # A  -> TIC 119584412
tic, toi = 37749396,  "TOI 260.01"    # B  -> TIC 37749396
# tic, toi = 311183180, "TOI 550.02"    # C  -> TIC 311183180
dur_hours = 6.0  # (A≈6–7 h, B≈4–5 h, C≈5–6 h). OK if approximate.

# --------- helpers (no other cells required) ----------
def load_ephem(tic):
    for suffix in ("", "_robust"):
        p = f"results/TIC{tic}_refined_ephemeris{suffix}.json"
        if os.path.exists(p):
            d = json.load(open(p))
            return float(d["P"]), float(d["T0"])
    raise FileNotFoundError(f"refined ephemeris JSON for TIC{tic} not found")

def load_stitched_npz(tic):
    p = f"results/TIC{tic}_stitched.npz"
    d = np.load(p)
    return d["t"].astype(float), d["f"].astype(float)

def wrap_phase(t, P, T0):
    # phase in [-0.5, 0.5)
    ph = ((t - T0) / P) % 1.0
    ph[ph >= 0.5] -= 1.0
    return ph

def in_window(t, P, T0, halfdur_days, center_phase=0.0):
    # distance in phase from chosen center (0 for primary, 0.5 for secondary)
    # work in [-0.5,0.5)
    ph = wrap_phase(t, P, T0)
    if center_phase == 0.5:
        d = np.abs((ph - 0.5 + 0.5) % 1.0 - 0.5)
    else:
        d = np.abs(ph)
    return d < (halfdur_days / P)

def odd_even_masks(t, P, T0, halfdur_days):
    # integer epoch index closest to each point
    k = np.rint((t - T0) / P).astype(int)
    w = in_window(t, P, T0, halfdur_days, center_phase=0.0)
    odd  = w & (k % 2 != 0)
    even = w & (k % 2 == 0)
    return odd, even

def bin_xy(x, y, nbins=80):
    # mean & SEM per bin
    edges = np.linspace(x.min(), x.max(), nbins+1)
    idx = np.digitize(x, edges) - 1
    xm = 0.5*(edges[:-1] + edges[1:])
    ymean, ysem = np.full(nbins, np.nan), np.full(nbins, np.nan)
    for i in range(nbins):
        m = idx == i
        if np.any(m):
            yy = y[m]
            ymean[i] = np.nanmean(yy)
            ysem[i]  = np.nanstd(yy) / np.sqrt(np.sum(np.isfinite(yy)))
    return xm, ymean, ysem

# --------- load data & make plots ----------
P, T0 = load_ephem(tic)
t, f  = load_stitched_npz(tic)

dur_days = dur_hours / 24.0
halfdur  = 0.5 * dur_days

ph = wrap_phase(t, P, T0)

odd_m, even_m = odd_even_masks(t, P, T0, halfdur)
sec_m = in_window(t, P, T0, halfdur, center_phase=0.5)

# sensible y-lims around the in-window scatter
y0 = np.nanmedian(f[odd_m | even_m])
ylim = (y0 - 0.004, y0 + 0.004)  # ~±0.4% window; tweak if needed

# ---- 1) Odd/Even primary window plot with binned overlay
plt.figure(figsize=(10,5.2), dpi=120)
plt.scatter(ph[odd_m],  f[odd_m],  s=9, alpha=0.25, label="odd")
plt.scatter(ph[even_m], f[even_m], s=9, alpha=0.25, label="even")
# binned means + 1σ(SEM) ribbons
for lbl, m, c in [("odd", odd_m, "#1f77b4"), ("even", even_m, "#ff7f0e")]:
    xb, ym, ys = bin_xy(ph[m], f[m], nbins=60)
    ok = np.isfinite(ym)
    plt.plot(xb[ok], ym[ok], lw=2, color=c, label=f"{lbl} (binned)")
    if np.any(ok & np.isfinite(ys)):
        plt.fill_between(xb[ok], ym[ok]-ys[ok], ym[ok]+ys[ok], alpha=0.15, color=c)
plt.xlim(-0.02, 0.02)
plt.ylim(*ylim)
plt.xlabel("Phase")
plt.ylabel("Flux (norm)")
plt.title(f"TIC{tic} phase odd/even — {toi}")
plt.legend(loc="upper left")
out1 = f"figures/TIC{tic}_phase_odd_even.png"
plt.savefig(out1, bbox_inches="tight")
plt.close()

# ---- 2) Secondary window plot with binned overlay
# convert to [0,1) for a tight zoom around 0.5
ph01 = (ph + 1.0) % 1.0
plt.figure(figsize=(10,5.2), dpi=120)
plt.scatter(ph01[sec_m], f[sec_m], s=10, alpha=0.35, label="secondary")
xb, ym, ys = bin_xy(ph01[sec_m], f[sec_m], nbins=50)
ok = np.isfinite(ym)
plt.plot(xb[ok], ym[ok], lw=2, label="binned")
if np.any(ok & np.isfinite(ys)):
    plt.fill_between(xb[ok], ym[ok]-ys[ok], ym[ok]+ys[ok], alpha=0.2)
plt.xlim(0.49, 0.51)
plt.ylim(*ylim)
plt.xlabel("Phase")
plt.ylabel("Flux (norm)")
plt.title(f"TIC{tic} phase secondary — {toi}")
plt.legend(loc="upper right")
out2 = f"figures/TIC{tic}_phase_secondary.png"
plt.savefig(out2, bbox_inches="tight")
plt.close()

print(f"Saved {out1} and {out2}")

Saved figures/TIC37749396_phase_odd_even.png and figures/TIC37749396_phase_secondary.png


In [35]:
# === PHASE PLOTS with BINNED OVERLAY (drop this in as one cell) ===
import os, json, numpy as np
import matplotlib.pyplot as plt

os.makedirs("figures", exist_ok=True)
BTJD_ZERO = 2457000.0  # (not used here, just for consistency)

# --------- pick target (edit these 3 lines per run) ----------
#tic, toi = 119584412, "TOI 1801.01"     # A  -> TIC 119584412
#tic, toi = 37749396,  "TOI 260.01"    # B  -> TIC 37749396
tic, toi = 311183180, "TOI 550.02"    # C  -> TIC 311183180
dur_hours = 6.0  # (A≈6–7 h, B≈4–5 h, C≈5–6 h). OK if approximate.

# --------- helpers (no other cells required) ----------
def load_ephem(tic):
    for suffix in ("", "_robust"):
        p = f"results/TIC{tic}_refined_ephemeris{suffix}.json"
        if os.path.exists(p):
            d = json.load(open(p))
            return float(d["P"]), float(d["T0"])
    raise FileNotFoundError(f"refined ephemeris JSON for TIC{tic} not found")

def load_stitched_npz(tic):
    p = f"results/TIC{tic}_stitched.npz"
    d = np.load(p)
    return d["t"].astype(float), d["f"].astype(float)

def wrap_phase(t, P, T0):
    # phase in [-0.5, 0.5)
    ph = ((t - T0) / P) % 1.0
    ph[ph >= 0.5] -= 1.0
    return ph

def in_window(t, P, T0, halfdur_days, center_phase=0.0):
    # distance in phase from chosen center (0 for primary, 0.5 for secondary)
    # work in [-0.5,0.5)
    ph = wrap_phase(t, P, T0)
    if center_phase == 0.5:
        d = np.abs((ph - 0.5 + 0.5) % 1.0 - 0.5)
    else:
        d = np.abs(ph)
    return d < (halfdur_days / P)

def odd_even_masks(t, P, T0, halfdur_days):
    # integer epoch index closest to each point
    k = np.rint((t - T0) / P).astype(int)
    w = in_window(t, P, T0, halfdur_days, center_phase=0.0)
    odd  = w & (k % 2 != 0)
    even = w & (k % 2 == 0)
    return odd, even

def bin_xy(x, y, nbins=80):
    # mean & SEM per bin
    edges = np.linspace(x.min(), x.max(), nbins+1)
    idx = np.digitize(x, edges) - 1
    xm = 0.5*(edges[:-1] + edges[1:])
    ymean, ysem = np.full(nbins, np.nan), np.full(nbins, np.nan)
    for i in range(nbins):
        m = idx == i
        if np.any(m):
            yy = y[m]
            ymean[i] = np.nanmean(yy)
            ysem[i]  = np.nanstd(yy) / np.sqrt(np.sum(np.isfinite(yy)))
    return xm, ymean, ysem

# --------- load data & make plots ----------
P, T0 = load_ephem(tic)
t, f  = load_stitched_npz(tic)

dur_days = dur_hours / 24.0
halfdur  = 0.5 * dur_days

ph = wrap_phase(t, P, T0)

odd_m, even_m = odd_even_masks(t, P, T0, halfdur)
sec_m = in_window(t, P, T0, halfdur, center_phase=0.5)

# sensible y-lims around the in-window scatter
y0 = np.nanmedian(f[odd_m | even_m])
ylim = (y0 - 0.004, y0 + 0.004)  # ~±0.4% window; tweak if needed

# ---- 1) Odd/Even primary window plot with binned overlay
plt.figure(figsize=(10,5.2), dpi=120)
plt.scatter(ph[odd_m],  f[odd_m],  s=9, alpha=0.25, label="odd")
plt.scatter(ph[even_m], f[even_m], s=9, alpha=0.25, label="even")
# binned means + 1σ(SEM) ribbons
for lbl, m, c in [("odd", odd_m, "#1f77b4"), ("even", even_m, "#ff7f0e")]:
    xb, ym, ys = bin_xy(ph[m], f[m], nbins=60)
    ok = np.isfinite(ym)
    plt.plot(xb[ok], ym[ok], lw=2, color=c, label=f"{lbl} (binned)")
    if np.any(ok & np.isfinite(ys)):
        plt.fill_between(xb[ok], ym[ok]-ys[ok], ym[ok]+ys[ok], alpha=0.15, color=c)
plt.xlim(-0.02, 0.02)
plt.ylim(*ylim)
plt.xlabel("Phase")
plt.ylabel("Flux (norm)")
plt.title(f"TIC{tic} phase odd/even — {toi}")
plt.legend(loc="upper left")
out1 = f"figures/TIC{tic}_phase_odd_even.png"
plt.savefig(out1, bbox_inches="tight")
plt.close()

# ---- 2) Secondary window plot with binned overlay
# convert to [0,1) for a tight zoom around 0.5
ph01 = (ph + 1.0) % 1.0
plt.figure(figsize=(10,5.2), dpi=120)
plt.scatter(ph01[sec_m], f[sec_m], s=10, alpha=0.35, label="secondary")
xb, ym, ys = bin_xy(ph01[sec_m], f[sec_m], nbins=50)
ok = np.isfinite(ym)
plt.plot(xb[ok], ym[ok], lw=2, label="binned")
if np.any(ok & np.isfinite(ys)):
    plt.fill_between(xb[ok], ym[ok]-ys[ok], ym[ok]+ys[ok], alpha=0.2)
plt.xlim(0.49, 0.51)
plt.ylim(*ylim)
plt.xlabel("Phase")
plt.ylabel("Flux (norm)")
plt.title(f"TIC{tic} phase secondary — {toi}")
plt.legend(loc="upper right")
out2 = f"figures/TIC{tic}_phase_secondary.png"
plt.savefig(out2, bbox_inches="tight")
plt.close()

print(f"Saved {out1} and {out2}")

Saved figures/TIC311183180_phase_odd_even.png and figures/TIC311183180_phase_secondary.png


In [ ]:
# --- CONFIG: switch target & sectors here only ---
TIC      = 119584412          # e.g., A=119584412, B=37749396, C=311183180
NAME     = "TOI 1801.01"      # label for titles
SECTORS  = [22, 49]           # sectors you want to run

# Refined ephemeris for this target (use your per-target JSON if you have it)
P_days    = 16.02749976
T0_btjd   = 1908.046283
DUR_hours = 6.53

# Figure style knobs (optional)
FIGSIZE   = (12, 4)           # (width, height) inches per 3-panel row
DPI_SAVE  = 220               # PNG DPI; raise to 300–400 for print

In [4]:
# === Figure B (one-cell version): Difference images & centroid offsets ===
# Target A example defaults: TOI 1801.01 / TIC 119584412, sectors 22 & 49
# -------------------------------------------------------------------------

# -------------------- ADJUST ME --------------------
TIC          = 119584412                # TIC ID
TARGET_NAME  = "TOI 1801.01"            # label for titles
SECTORS      = [22, 49]                 # which sectors to process

# Ephemeris (used only to decide in/out-of-transit frame selection)
P_days       = 16.02749976              # period (days)
T0_btjd      = 1908.046283              # reference mid-transit (BTJD)
DUR_hours    = 6.53                     # transit duration estimate (hours)
PAD          = 1.0                      # frames counted "out" must be > PAD*duration away

# Quality and robustness
QUALITY_GOOD = 0                        # keep only quality==0 if present
MIN_IN       = 10                       # min # of in-transit cadences to proceed
MIN_OUT      = 50                       # min # of out-of-transit cadences to proceed

# File outputs
FIG_DIR      = "figures"
RES_DIR      = "results"
DPI_FIG      = 200
# ---------------------------------------------------

import os, csv, numpy as np, matplotlib.pyplot as plt
import lightkurve as lk
from astropy.wcs.utils import proj_plane_pixel_scales

os.makedirs(FIG_DIR, exist_ok=True)
os.makedirs(RES_DIR, exist_ok=True)

def _get_tpf(tic, sector, cutout_size=15):
    """Prefer SPOC TargetPixelFile; fall back to TessCut if needed."""
    sr = lk.search_targetpixelfile(f"TIC {tic}", mission="TESS", sector=sector, author="SPOC")
    if len(sr) > 0:
        print(f"[tpf] Using SPOC TPF for TIC {tic}, S{sector}")
        return sr.download()
    sc = lk.search_tesscut(f"TIC {tic}", sector=sector)
    if len(sc) == 0:
        raise FileNotFoundError(f"No TPF or TessCut found for TIC {tic} sector {sector}")
    print(f"[tpf] Using TessCut cutout for TIC {tic}, S{sector} (size={cutout_size})")
    return sc.download(cutout_size=cutout_size)

def _transit_masks(time_btjd, P, T0, dur_h, pad=1.0):
    """Boolean masks for in- and out-of-transit cadences."""
    dur_d = float(dur_h) / 24.0
    phase = ((time_btjd - T0 + 0.5*P) % P) - 0.5*P  # centered phase in days
    in_tr  = np.abs(phase) < 0.5*dur_d
    out_tr = np.abs(phase) > pad * dur_d
    return in_tr, out_tr

def _centroid_xy(image2d):
    """Flux-weighted centroid in pixel coordinates (uses only positive flux)."""
    yy, xx = np.indices(image2d.shape)
    w = np.clip(image2d, 0, np.inf)
    s = np.nansum(w)
    if not np.isfinite(s) or s <= 0:
        return np.nan, np.nan
    x = np.nansum(w * xx) / s
    y = np.nansum(w * yy) / s
    return float(x), float(y)

def _pixel_scale_arcsec(tpf):
    try:
        # degrees/pixel -> arcsec/pixel
        return float(np.mean(proj_plane_pixel_scales(tpf.wcs)) * 3600.0)
    except Exception:
        return 21.0  # TESS nominal plate scale if WCS missing

def _figureB_for_sector(tic, sector, P, T0, dur_h, target_label):
    tpf = _get_tpf(tic, sector)

    # Lightweight quality screen
    t = np.asarray(tpf.time.value, float)                  # BTJD as float array
    good = np.isfinite(t)
    if hasattr(tpf, "quality") and tpf.quality is not None:
        good &= (tpf.quality == QUALITY_GOOD)

    in_tr, out_tr = _transit_masks(t, P, T0, dur_h, pad=PAD)
    use_in, use_out = (good & in_tr), (good & out_tr)

    n_in, n_out = int(use_in.sum()), int(use_out.sum())
    print(f"[S{sector}] in-transit cadences = {n_in}, out-of-transit = {n_out}")

    if (n_in < MIN_IN) or (n_out < MIN_OUT):
        raise RuntimeError(f"S{sector}: not enough cadences for clean diff image "
                           f"(need ≥{MIN_IN} in, ≥{MIN_OUT} out).")

    # Convert astropy Quantity (e-/s) → plain floats before stats/plotting
    in_cube  = tpf.flux[use_in].value   # (N_in, y, x)
    out_cube = tpf.flux[use_out].value  # (N_out, y, x)

    in_img   = np.nanmedian(in_cube,  axis=0).astype(float)
    out_img  = np.nanmedian(out_cube, axis=0).astype(float)
    diff     = out_img - in_img  # positive where the star got dimmer (a transit)

    # Flux centroids
    x_star, y_star = _centroid_xy(out_img)   # stellar PSF center
    x_diff, y_diff = _centroid_xy(diff)      # location of the dip in the diff image
    dx_pix, dy_pix = (x_diff - x_star), (y_diff - y_star)
    r_pix          = float(np.hypot(dx_pix, dy_pix))

    # Convert to arcsec
    pxscale = _pixel_scale_arcsec(tpf)
    r_arcsec = r_pix * pxscale

    # ----- Plot three panels -----
    fig, axes = plt.subplots(1, 3, figsize=(12, 4), constrained_layout=True)

    vmin = float(np.nanpercentile(out_img,  5.0))
    vmax = float(np.nanpercentile(out_img, 99.5))

    im0 = axes[0].imshow(out_img, origin="lower", vmin=vmin, vmax=vmax)
    axes[0].set_title("Out of transit")
    axes[0].plot(x_star, y_star, marker="+", ms=12, mfc="none", mec="w")

    im1 = axes[1].imshow(in_img, origin="lower", vmin=vmin, vmax=vmax)
    axes[1].set_title("In transit")
    axes[1].plot(x_star, y_star, marker="+", ms=12, mfc="none", mec="w")

    d_vmax = float(np.nanpercentile(np.abs(diff), 99.5))
    im2 = axes[2].imshow(diff, origin="lower", cmap="coolwarm", vmin=-d_vmax, vmax=d_vmax)
    axes[2].set_title("Difference (out − in)")
    axes[2].plot(x_diff, y_diff, marker="x", ms=12, mec="k", mew=2)
    axes[2].plot(x_star, y_star, marker="+", ms=12, mec="k")

    for ax in axes:
        ax.set_xticks([]); ax.set_yticks([])

    header = (f"{target_label} (TIC {tic})  •  Sector {sector}\n"
              f"P={P:.6f} d, T0={T0:.6f} BTJD, dur≈{dur_h:.2f} h   "
              f"N_in={n_in}, N_out={n_out}   "
              f"centroid offset={r_pix:.2f} px ≈ {r_arcsec:.1f}\"")
    fig.suptitle(header, fontsize=11)

    # Labeled colorbars (plain strings so units don’t crash Matplotlib)
    cb0 = fig.colorbar(im0, ax=axes[0], fraction=0.046, pad=0.04); cb0.ax.set_ylabel("flux (e-/s)")
    cb1 = fig.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04); cb1.ax.set_ylabel("flux (e-/s)")
    cb2 = fig.colorbar(im2, ax=axes[2], fraction=0.046, pad=0.04); cb2.ax.set_ylabel("Δ flux (e-/s)")

    png_path = os.path.join(FIG_DIR, f"TIC{tic}_FigureB_S{sector}.png")
    fig.savefig(png_path, dpi=DPI_FIG)
    plt.close(fig)
    print(f"[save] {png_path}")

    return {
        "tic": tic, "sector": sector, "P_days": P, "T0_btjd": T0, "dur_h": dur_h,
        "N_in": n_in, "N_out": n_out,
        "x_star": x_star, "y_star": y_star,
        "x_diff": x_diff, "y_diff": y_diff,
        "dx_pix": float(dx_pix), "dy_pix": float(dy_pix), "r_pix": r_pix,
        "pix_scale_arcsec": pxscale, "r_arcsec": r_arcsec,
        "panel_png": png_path
    }

# -------------------- RUN --------------------
rows = []
for sec in SECTORS:
    try:
        row = _figureB_for_sector(TIC, sec, P_days, T0_btjd, DUR_hours, TARGET_NAME)
        rows.append(row)
    except Exception as e:
        print(f"[S{sec}] ERROR:", e)

# Save CSV if any rows
csv_path = os.path.join(RES_DIR, f"TIC{TIC}_centroid_offsets.csv")
if rows:
    with open(csv_path, "w", newline="") as f:
        w = csv.DictWriter(f, fieldnames=list(rows[0].keys()))
        w.writeheader(); w.writerows(rows)
    print(f"[save] {csv_path}")

    # Optional: combined gallery
    import matplotlib.image as mpimg
    fig, ax = plt.subplots(len(rows), 1, figsize=(6, 3.8*len(rows)))
    if len(rows) == 1:
        ax = [ax]
    for i, r in enumerate(rows):
        ax[i].imshow(mpimg.imread(r["panel_png"])); ax[i].axis("off")
        ax[i].set_title(f"{TARGET_NAME} • Sector {r['sector']}")
    combo = os.path.join(FIG_DIR, f"TIC{TIC}_FigureB_combo.png")
    plt.tight_layout(); plt.savefig(combo, dpi=180); plt.close()
    print(f"[save] {combo}")
else:
    print("[skip] No rows were produced; CSV & combo skipped.")

[tpf] Using SPOC TPF for TIC 119584412, S22
[S22] in-transit cadences = 392, out-of-transit = 15319
[save] figures/TIC119584412_FigureB_S22.png
[tpf] Using SPOC TPF for TIC 119584412, S49
[S49] in-transit cadences = 392, out-of-transit = 12488
[save] figures/TIC119584412_FigureB_S49.png
[save] results/TIC119584412_centroid_offsets.csv
[save] figures/TIC119584412_FigureB_combo.png


In [5]:
# === Figure B: Difference images & centroid offsets (Target B) ===
# One-cell, copy/paste runnable block

import os, csv, numpy as np, matplotlib.pyplot as plt
import lightkurve as lk
from astropy.wcs.utils import proj_plane_pixel_scales

# ---------- Target & ephemeris (adjust here if needed) ----------
TIC     = 37749396                 # Target B (TOI 260.01)
NAME    = "TOI 260.01"
SECTORS = [3, 42, 70]              # sectors you ran BLS/TLS on
P_days    = 13.47582381            # refined period (Sep 16 entry)
T0_btjd   = 1392.306006            # refined T0   (Sep 16 entry)
DUR_hours = 3.0                    # transit duration guess; tweak if you like (e.g., 1.5–3.0 h)

# ---------- Folders ----------
os.makedirs("figures", exist_ok=True)
os.makedirs("results", exist_ok=True)

# ---------- Helpers ----------
def get_tpf(tic, sector, cutout_size=15):
    """Prefer SPOC 2-min/20-sec TPF; else fall back to TessCut FFI cutout."""
    sr = lk.search_targetpixelfile(f"TIC {tic}", mission="TESS", sector=sector, author="SPOC")
    if len(sr) > 0:
        print(f"[tpf] Using SPOC TPF for TIC {tic}, S{sector}")
        return sr.download()
    sc = lk.search_tesscut(f"TIC {tic}", sector=sector)
    if len(sc) == 0:
        raise FileNotFoundError(f"No TPF or TessCut for TIC {tic} S{sector}")
    print(f"[tpf] Using TessCut cutout for TIC {tic}, S{sector} (size={cutout_size})")
    return sc.download(cutout_size=cutout_size)

def transit_masks(time_btjd, P, T0, dur_h, pad=1.0):
    """Boolean masks for in- and out-of-transit."""
    dur_d = dur_h / 24.0
    phase = ((time_btjd - T0 + 0.5*P) % P) - 0.5*P
    in_tr  = np.abs(phase) < 0.5*dur_d
    out_tr = np.abs(phase) > pad * dur_d
    return in_tr, out_tr

def flux_weighted_centroid(img_float2d):
    """Return centroid (x,y) in pixel coords using positive flux."""
    yy, xx = np.indices(img_float2d.shape)
    w = np.clip(img_float2d, 0, np.inf)
    s = np.nansum(w)
    if not np.isfinite(s) or s <= 0:
        return np.nan, np.nan
    x = np.nansum(w * xx) / s
    y = np.nansum(w * yy) / s
    return float(x), float(y)

def pixel_scale_arcsec(tpf):
    try:
        scales_deg_per_pix = proj_plane_pixel_scales(tpf.wcs)  # deg/pix
        return float(np.mean(scales_deg_per_pix) * 3600.0)
    except Exception:
        return 21.0  # TESS nominal

def figureB_for_sector(tic, sector, P, T0, dur_h, name, pad=1.0, tesscut_size=15):
    tpf = get_tpf(tic, sector, cutout_size=tesscut_size)
    t = np.asarray(tpf.time.value, float)  # BTJD

    # Quality mask if available
    good = np.isfinite(t)
    if hasattr(tpf, "quality") and tpf.quality is not None:
        good &= (tpf.quality == 0)

    in_tr, out_tr = transit_masks(t, P, T0, dur_h, pad=pad)
    use_in, use_out = (good & in_tr), (good & out_tr)

    n_in, n_out = int(use_in.sum()), int(use_out.sum())
    print(f"[S{sector}] in-transit cadences = {n_in}, out-of-transit = {n_out}")
    if n_in < 10 or n_out < 50:
        print(f"[S{sector}] Warning: few cadences; difference image may be noisy.")

    # Convert Quantity (e-/s) -> float arrays before stats/plotting
    in_cube  = tpf.flux[use_in].value
    out_cube = tpf.flux[use_out].value
    in_img   = np.nanmedian(in_cube,  axis=0).astype(float)
    out_img  = np.nanmedian(out_cube, axis=0).astype(float)
    diff     = out_img - in_img  # positive where star dimmed

    # Centroids
    x_star, y_star = flux_weighted_centroid(out_img)
    x_diff, y_diff = flux_weighted_centroid(diff)
    dx_pix = x_diff - x_star
    dy_pix = y_diff - y_star
    r_pix  = float(np.hypot(dx_pix, dy_pix))

    # Arcsec conversion
    ps = pixel_scale_arcsec(tpf)
    r_arcsec = r_pix * ps

    # --- Plot panels ---
    fig, axes = plt.subplots(1, 3, figsize=(12, 4), constrained_layout=True)
    vmin = float(np.nanpercentile(out_img,  5.0))
    vmax = float(np.nanpercentile(out_img, 99.5))

    im0 = axes[0].imshow(out_img, origin="lower", vmin=vmin, vmax=vmax)
    axes[0].set_title("Out of transit")
    axes[0].plot(x_star, y_star, marker="+", ms=12, mfc="none", mec="w")

    im1 = axes[1].imshow(in_img, origin="lower", vmin=vmin, vmax=vmax)
    axes[1].set_title("In transit")
    axes[1].plot(x_star, y_star, marker="+", ms=12, mfc="none", mec="w")

    d_vmax = float(np.nanpercentile(np.abs(diff), 99.5))
    im2 = axes[2].imshow(diff, origin="lower", cmap="coolwarm", vmin=-d_vmax, vmax=d_vmax)
    axes[2].set_title("Difference (out − in)")
    axes[2].plot(x_diff, y_diff, marker="x", ms=12, mec="k", mew=2)
    axes[2].plot(x_star, y_star, marker="+", ms=12, mec="k")

    for ax in axes:
        ax.set_xticks([]); ax.set_yticks([])

    txt = (f"{name} (TIC {tic})  •  Sector {sector}\n"
           f"P={P:.6f} d, T0={T0:.6f} BTJD, dur≈{dur_h:.2f} h\n"
           f"N_in={n_in}, N_out={n_out}  •  centroid offset={r_pix:.2f} px ≈ {r_arcsec:.1f}\"")
    fig.suptitle(txt, fontsize=11)

    # Colorbars
    c0 = fig.colorbar(im0, ax=axes[0], fraction=0.046, pad=0.04); c0.ax.set_ylabel("flux (e-/s)")
    c1 = fig.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04); c1.ax.set_ylabel("flux (e-/s)")
    c2 = fig.colorbar(im2, ax=axes[2], fraction=0.046, pad=0.04); c2.ax.set_ylabel("Δ flux (e-/s)")

    out_png = f"figures/TIC{tic}_FigureB_S{sector}.png"
    fig.savefig(out_png, dpi=200)
    plt.close(fig)
    print(f"[save] {out_png}")

    return {
        "tic": tic, "name": name, "sector": sector,
        "P_days": P, "T0_btjd": T0, "dur_h": dur_h,
        "N_in": n_in, "N_out": n_out,
        "x_star": x_star, "y_star": y_star,
        "x_diff": x_diff, "y_diff": y_diff,
        "dx_pix": float(dx_pix), "dy_pix": float(dy_pix), "r_pix": r_pix,
        "pix_scale_arcsec": ps, "r_arcsec": r_arcsec
    }

# ---------- Run & save CSV + combo ----------
rows = []
for sec in SECTORS:
    try:
        rows.append(figureB_for_sector(TIC, sec, P_days, T0_btjd, DUR_hours, NAME))
    except Exception as e:
        print(f"[S{sec}] ERROR:", e)

csv_path = f"results/TIC{TIC}_centroid_offsets.csv"
if rows:
    with open(csv_path, "w", newline="") as f:
        w = csv.DictWriter(f, fieldnames=list(rows[0].keys()))
        w.writeheader(); w.writerows(rows)
    print(f"[save] {csv_path}")

    # Optional: combined gallery
    import matplotlib.image as mpimg
    fig, ax = plt.subplots(len(rows), 1, figsize=(6, 3.8*len(rows)))
    if len(rows) == 1:
        ax = [ax]
    for i, r in enumerate(rows):
        png = f"figures/TIC{r['tic']}_FigureB_S{r['sector']}.png"
        ax[i].imshow(mpimg.imread(png)); ax[i].axis("off")
        ax[i].set_title(f"{NAME} • Sector {r['sector']}")
    combo = f"figures/TIC{TIC}_FigureB_combo.png"
    plt.tight_layout(); plt.savefig(combo, dpi=180); plt.close()
    print(f"[save] {combo}")
else:
    print("[skip] No rows to write (all sectors failed).")

[S3] ERROR: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
[tpf] Using SPOC TPF for TIC 37749396, S42
[S42] in-transit cadences = 90, out-of-transit = 11293
[save] figures/TIC37749396_FigureB_S42.png
[tpf] Using SPOC TPF for TIC 37749396, S70


/Users/kobi.weitzman/miniforge3/envs/tess-ephem/lib/python3.10/site-packages/lightkurve/search.py:424: LightkurveWarning: Warning: 2 files available to download. Only the first file has been downloaded. Please use `download_all()` or specify additional criteria (e.g. quarter, campaign, or sector) to limit your search.
  warnings.warn(


[S70] in-transit cadences = 1063, out-of-transit = 82549
[save] figures/TIC37749396_FigureB_S70.png
[save] results/TIC37749396_centroid_offsets.csv
[save] figures/TIC37749396_FigureB_combo.png


In [6]:

# One-cell, copy/paste runnable block

import os, csv, numpy as np, matplotlib.pyplot as plt
import lightkurve as lk
from astropy.wcs.utils import proj_plane_pixel_scales

# ---------- Target & ephemeris (adjust here if needed) ----------
TIC    = 311183180        # Target C
NAME   = "TOI 550.02"
SECTORS = [5, 31]

# Refined ephemeris (your results)
P_days    = 9.34849077
T0_btjd   = 2149.633454
DUR_hours = 2.8   # use your measured duration from TLS/midtime fits
# ---------- Folders ----------
os.makedirs("figures", exist_ok=True)
os.makedirs("results", exist_ok=True)

# ---------- Helpers ----------
def get_tpf(tic, sector, cutout_size=15):
    """Prefer SPOC 2-min/20-sec TPF; else fall back to TessCut FFI cutout."""
    sr = lk.search_targetpixelfile(f"TIC {tic}", mission="TESS", sector=sector, author="SPOC")
    if len(sr) > 0:
        print(f"[tpf] Using SPOC TPF for TIC {tic}, S{sector}")
        return sr.download()
    sc = lk.search_tesscut(f"TIC {tic}", sector=sector)
    if len(sc) == 0:
        raise FileNotFoundError(f"No TPF or TessCut for TIC {tic} S{sector}")
    print(f"[tpf] Using TessCut cutout for TIC {tic}, S{sector} (size={cutout_size})")
    return sc.download(cutout_size=cutout_size)

def transit_masks(time_btjd, P, T0, dur_h, pad=1.0):
    """Boolean masks for in- and out-of-transit."""
    dur_d = dur_h / 24.0
    phase = ((time_btjd - T0 + 0.5*P) % P) - 0.5*P
    in_tr  = np.abs(phase) < 0.5*dur_d
    out_tr = np.abs(phase) > pad * dur_d
    return in_tr, out_tr

def flux_weighted_centroid(img_float2d):
    """Return centroid (x,y) in pixel coords using positive flux."""
    yy, xx = np.indices(img_float2d.shape)
    w = np.clip(img_float2d, 0, np.inf)
    s = np.nansum(w)
    if not np.isfinite(s) or s <= 0:
        return np.nan, np.nan
    x = np.nansum(w * xx) / s
    y = np.nansum(w * yy) / s
    return float(x), float(y)

def pixel_scale_arcsec(tpf):
    try:
        scales_deg_per_pix = proj_plane_pixel_scales(tpf.wcs)  # deg/pix
        return float(np.mean(scales_deg_per_pix) * 3600.0)
    except Exception:
        return 21.0  # TESS nominal

def figureB_for_sector(tic, sector, P, T0, dur_h, name, pad=1.0, tesscut_size=15):
    tpf = get_tpf(tic, sector, cutout_size=tesscut_size)
    t = np.asarray(tpf.time.value, float)  # BTJD

    # Quality mask if available
    good = np.isfinite(t)
    if hasattr(tpf, "quality") and tpf.quality is not None:
        good &= (tpf.quality == 0)

    in_tr, out_tr = transit_masks(t, P, T0, dur_h, pad=pad)
    use_in, use_out = (good & in_tr), (good & out_tr)

    n_in, n_out = int(use_in.sum()), int(use_out.sum())
    print(f"[S{sector}] in-transit cadences = {n_in}, out-of-transit = {n_out}")
    if n_in < 10 or n_out < 50:
        print(f"[S{sector}] Warning: few cadences; difference image may be noisy.")

    # Convert Quantity (e-/s) -> float arrays before stats/plotting
    in_cube  = tpf.flux[use_in].value
    out_cube = tpf.flux[use_out].value
    in_img   = np.nanmedian(in_cube,  axis=0).astype(float)
    out_img  = np.nanmedian(out_cube, axis=0).astype(float)
    diff     = out_img - in_img  # positive where star dimmed

    # Centroids
    x_star, y_star = flux_weighted_centroid(out_img)
    x_diff, y_diff = flux_weighted_centroid(diff)
    dx_pix = x_diff - x_star
    dy_pix = y_diff - y_star
    r_pix  = float(np.hypot(dx_pix, dy_pix))

    # Arcsec conversion
    ps = pixel_scale_arcsec(tpf)
    r_arcsec = r_pix * ps

    # --- Plot panels ---
    fig, axes = plt.subplots(1, 3, figsize=(12, 4), constrained_layout=True)
    vmin = float(np.nanpercentile(out_img,  5.0))
    vmax = float(np.nanpercentile(out_img, 99.5))

    im0 = axes[0].imshow(out_img, origin="lower", vmin=vmin, vmax=vmax)
    axes[0].set_title("Out of transit")
    axes[0].plot(x_star, y_star, marker="+", ms=12, mfc="none", mec="w")

    im1 = axes[1].imshow(in_img, origin="lower", vmin=vmin, vmax=vmax)
    axes[1].set_title("In transit")
    axes[1].plot(x_star, y_star, marker="+", ms=12, mfc="none", mec="w")

    d_vmax = float(np.nanpercentile(np.abs(diff), 99.5))
    im2 = axes[2].imshow(diff, origin="lower", cmap="coolwarm", vmin=-d_vmax, vmax=d_vmax)
    axes[2].set_title("Difference (out − in)")
    axes[2].plot(x_diff, y_diff, marker="x", ms=12, mec="k", mew=2)
    axes[2].plot(x_star, y_star, marker="+", ms=12, mec="k")

    for ax in axes:
        ax.set_xticks([]); ax.set_yticks([])

    txt = (f"{name} (TIC {tic})  •  Sector {sector}\n"
           f"P={P:.6f} d, T0={T0:.6f} BTJD, dur≈{dur_h:.2f} h\n"
           f"N_in={n_in}, N_out={n_out}  •  centroid offset={r_pix:.2f} px ≈ {r_arcsec:.1f}\"")
    fig.suptitle(txt, fontsize=11)

    # Colorbars
    c0 = fig.colorbar(im0, ax=axes[0], fraction=0.046, pad=0.04); c0.ax.set_ylabel("flux (e-/s)")
    c1 = fig.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04); c1.ax.set_ylabel("flux (e-/s)")
    c2 = fig.colorbar(im2, ax=axes[2], fraction=0.046, pad=0.04); c2.ax.set_ylabel("Δ flux (e-/s)")

    out_png = f"figures/TIC{tic}_FigureB_S{sector}.png"
    fig.savefig(out_png, dpi=200)
    plt.close(fig)
    print(f"[save] {out_png}")

    return {
        "tic": tic, "name": name, "sector": sector,
        "P_days": P, "T0_btjd": T0, "dur_h": dur_h,
        "N_in": n_in, "N_out": n_out,
        "x_star": x_star, "y_star": y_star,
        "x_diff": x_diff, "y_diff": y_diff,
        "dx_pix": float(dx_pix), "dy_pix": float(dy_pix), "r_pix": r_pix,
        "pix_scale_arcsec": ps, "r_arcsec": r_arcsec
    }

# ---------- Run & save CSV + combo ----------
rows = []
for sec in SECTORS:
    try:
        rows.append(figureB_for_sector(TIC, sec, P_days, T0_btjd, DUR_hours, NAME))
    except Exception as e:
        print(f"[S{sec}] ERROR:", e)

csv_path = f"results/TIC{TIC}_centroid_offsets.csv"
if rows:
    with open(csv_path, "w", newline="") as f:
        w = csv.DictWriter(f, fieldnames=list(rows[0].keys()))
        w.writeheader(); w.writerows(rows)
    print(f"[save] {csv_path}")

    # Optional: combined gallery
    import matplotlib.image as mpimg
    fig, ax = plt.subplots(len(rows), 1, figsize=(6, 3.8*len(rows)))
    if len(rows) == 1:
        ax = [ax]
    for i, r in enumerate(rows):
        png = f"figures/TIC{r['tic']}_FigureB_S{r['sector']}.png"
        ax[i].imshow(mpimg.imread(png)); ax[i].axis("off")
        ax[i].set_title(f"{NAME} • Sector {r['sector']}")
    combo = f"figures/TIC{TIC}_FigureB_combo.png"
    plt.tight_layout(); plt.savefig(combo, dpi=180); plt.close()
    print(f"[save] {combo}")
else:
    print("[skip] No rows to write (all sectors failed).")

[tpf] Using SPOC TPF for TIC 311183180, S5
[S5] in-transit cadences = 251, out-of-transit = 16783
[save] figures/TIC311183180_FigureB_S5.png
[tpf] Using SPOC TPF for TIC 311183180, S31
[S31] in-transit cadences = 252, out-of-transit = 15745
[save] figures/TIC311183180_FigureB_S31.png
[save] results/TIC311183180_centroid_offsets.csv
[save] figures/TIC311183180_FigureB_combo.png


In [5]:
import sys, numpy as np, triceratops
print(sys.executable)
print("numpy:", np.__version__)
print("tri at:", triceratops.__file__)

/Users/kobi.weitzman/miniforge3/envs/tess-ephem/bin/python
numpy: 1.26.4
tri at: /Users/kobi.weitzman/miniforge3/envs/tess-ephem/lib/python3.10/site-packages/triceratops/__init__.py


In [23]:
# === CLEAN TLS PIPELINE (CBV unit fix + robust-free flatten + quiet) ===

import os, warnings, numpy as np
from lightkurve import LightCurve, log, search_lightcurvefile
from transitleastsquares import transitleastsquares
from astropy.units import Unit

# -------- QUIET MODE --------
os.environ["TQDM_DISABLE"] = "1"                 # suppress tqdm across deps
log.setLevel('ERROR')                            # hush lightkurve logs
warnings.filterwarnings("ignore", category=RuntimeWarning, module=r"numpy\.core\._methods")

# -------- CONFIG --------
TARGET_TIC = int(globals().get("target_tic", 119584412))
PERIOD_MIN, PERIOD_MAX = 4.85, 5.15
N_THREADS = max(1, (os.cpu_count() or 2) - 1)

# If you know your injected depth in ppm, define injected_depth_ppm above.
depth_ppm = float(globals().get("injected_depth_ppm", 200.0))   # default 200 ppm
depth_frac = max(1e-6, 0.2 * depth_ppm * 1e-6)                  # avoid "No transit were fit"

tls_common = dict(
    period_min=PERIOD_MIN,
    period_max=PERIOD_MAX,
    transit_depth_min=depth_frac,
    use_threads=N_THREADS,
    show_progress_bar=False,
)

# -------- HELPERS --------
def _first_defined(*names):
    g = globals()
    for n in names:
        if n in g and g[n] is not None:
            return g[n]
    return None

def resolve_or_download_pdcsap_sap():
    """Return (lc_pdcsap_raw, lc_sap_raw) LightCurve objects.
       If missing, download SPOC LCFs for TARGET_TIC and stitch."""
    g = globals()
    pd = g.get("lc_pdcsap_raw", None)
    sa = g.get("lc_sap_raw", None)
    if isinstance(pd, LightCurve) and isinstance(sa, LightCurve):
        return pd, sa

    # Try common variable names
    pd_guess = _first_defined("lc_pdcsap", "pdcsap", "lc_pd", "pd")
    sa_guess = _first_defined("lc_sap", "sap", "lc_sa", "sa")
    if isinstance(pd_guess, LightCurve) and isinstance(sa_guess, LightCurve):
        return pd_guess, sa_guess

    # Download from SPOC LCFs so we can access both PDCSAP and SAP consistently
    print(f"[fetch] Downloading SPOC LCFs for TIC {TARGET_TIC} (TESS)...")
    sr = search_lightcurvefile(f"TIC {TARGET_TIC}", mission="TESS", author="SPOC")
    lcf_coll = sr.download_all()
    if lcf_coll is None or len(lcf_coll) == 0:
        raise RuntimeError(f"No SPOC light-curve files found for TIC {TARGET_TIC}.")

    # Stitch PDCSAP and SAP across all sectors
    try:
        lc_pdcsap_raw = lcf_coll.PDCSAP_FLUX.stitch()
        lc_sap_raw    = lcf_coll.SAP_FLUX.stitch()
    except Exception as e:
        raise RuntimeError(f"Could not build PDCSAP/SAP from LCFs: {e}")

    if not isinstance(lc_pdcsap_raw, LightCurve) or not isinstance(lc_sap_raw, LightCurve):
        raise RuntimeError("Unexpected objects from stitching; PDCSAP/SAP LightCurve not obtained.")

    return lc_pdcsap_raw, lc_sap_raw

def sap_cbv_or_flatten(lc_sap_raw: LightCurve) -> LightCurve:
    """Prefer SAP+CBV if available & in e-/s units; otherwise fall back to flatten().
       IMPORTANT: Do NOT pass a normalized curve here; CBV requires e-/s units."""
    # Try CBV on RAW (non-normalized) SAP if units are correct
    try:
        if getattr(lc_sap_raw.flux, "unit", None) != Unit("electron / second"):
            raise AssertionError("SAP flux not in e-/s; skipping CBV.")
        corr = lc_sap_raw.to_corrector("cbv")  # requires LK CBVs
        corrected = corr.correct(pca_components=5, sigma=5)
        corrected = corrected.remove_nans().normalize()
        corrected.meta = {**getattr(corrected, "meta", {}), "pipeline": "SAP+CBV"}
        return corrected
    except Exception as e:
        # Fallback: robust-free flatten for broad compatibility
        flat = lc_sap_raw.remove_nans().flatten(window_length=401, polyorder=2, niters=3, sigma=3)
        flat = flat.normalize()
        flat.meta = {**getattr(flat, "meta", {}), "pipeline": "SAP+flatten", "cbv_error": repr(e)}
        return flat

def run_tls_once(lc: LightCurve, label: str):
    t_all = getattr(lc.time, "value", lc.time)
    f_all = getattr(lc.flux, "value", lc.flux)
    m = np.isfinite(t_all) & np.isfinite(f_all)
    t = np.ascontiguousarray(t_all[m], dtype=float)
    f = np.ascontiguousarray(f_all[m], dtype=float)
    model = transitleastsquares(t, f)
    res = model.power(**tls_common)
    res.pipeline_label = label
    return res

def summarize_tls(res):
    return dict(
        pipeline=getattr(res, "pipeline_label", "NA"),
        period=res.period,
        period_unc=res.period_uncertainty,
        t0=res.T0,
        duration=res.duration,
        depth=res.depth,     # fractional
        sde=res.SDE,
        snr=res.snr,
        odd_even=res.odd_even_mismatch,
        n_transits=res.transit_count,
    )

# -------- PIPELINE --------
lc_pdcsap_raw, lc_sap_raw = resolve_or_download_pdcsap_sap()

# PDCSAP: clean & normalize
lc_pdcsap = lc_pdcsap_raw.remove_nans().normalize()
lc_pdcsap.meta = {**getattr(lc_pdcsap, "meta", {}), "pipeline": "PDCSAP"}

# SAP branch: run CBV on RAW (e-/s) if possible; otherwise flatten, then normalize
lc_sap_cbv = sap_cbv_or_flatten(lc_sap_raw)
sap_label = lc_sap_cbv.meta.get("pipeline", "SAP")

# Run TLS (status bars disabled)
pdcsap_res = run_tls_once(lc_pdcsap, "PDCSAP")
sap_res    = run_tls_once(lc_sap_cbv, sap_label)

# Compact summary
row_pdcsap = summarize_tls(pdcsap_res)
row_sap    = summarize_tls(sap_res)

print(f"[clean-run] TIC {TARGET_TIC}   period window: {PERIOD_MIN}–{PERIOD_MAX} d")
print(f"[clean-run] depth floor (fraction): {depth_frac:.2e}   (≈ {depth_ppm} ppm × 0.2)")
print("[PDCSAP]:", {k: row_pdcsap[k] for k in ("period","sde","snr","n_transits")})
print("[", sap_label, "]:", {k: row_sap[k] for k in ("period","sde","snr","n_transits")})

 97%|████████████████████████████████████ | 85937/88173 periods | 13:40:05<21:20
  4%|█▌                                  | 114/2572 periods | 13:12:41<284:51:39


Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 29 durations
Searching 29374 data points, 1065 periods from 4.85 to 5.15 days
Using 7 of 8 CPU threads
Searching for best T0 for period 5.14994 days
Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 29 durations
Searching 33536 data points, 1066 periods from 4.85 to 5.15 days
Using 7 of 8 CPU threads


/Users/kobi.weitzman/miniforge3/envs/tess-ephem/lib/python3.10/site-packages/transitleastsquares/main.py:411: UserWarning: 141 of 148 transits without data. The true period may be twice the given period.
  warnings.warn(text)


Searching for best T0 for period 4.97446 days
[clean-run] TIC 119584412   period window: 4.85–5.15 d
[clean-run] depth floor (fraction): 4.00e-05   (≈ 200.0 ppm × 0.2)
[PDCSAP]: {'period': 5.149944880035743, 'sde': 3.866641087303198, 'snr': 4.496647217554053, 'n_transits': 148}
[ SAP+flatten ]: {'period': 4.97446045188394, 'sde': 4.9504278791412695, 'snr': 3.618312997433482, 'n_transits': 153}


/Users/kobi.weitzman/miniforge3/envs/tess-ephem/lib/python3.10/site-packages/transitleastsquares/main.py:411: UserWarning: 143 of 153 transits without data. The true period may be twice the given period.
  warnings.warn(text)


In [30]:
# ==== SAVE TLS RESULTS & FIGURES (robust to TimeDelta/Quantity) ====
import os, csv, json, datetime as dt
import numpy as np
import matplotlib.pyplot as plt

# ---------- robust converters ----------
def _to_float(x, unit=None):
    """Safely turn astropy Quantity/TimeDelta/np scalars into float."""
    try:
        if hasattr(x, "to_value"):
            # Try unitless first; if Astropy requires a unit (e.g., TimeDelta), fall back to 'day'
            try:
                return float(x.to_value(unit)) if unit else float(x.to_value())
            except TypeError:
                return float(x.to_value(unit or "day"))
        if hasattr(x, "value"):
            return float(x.value)
        return float(x)
    except Exception:
        return np.nan

def _to_numpy_1d(x, unit=None):
    """Safely turn astropy arrays (incl. TimeDelta) into 1D float numpy arrays."""
    if hasattr(x, "to_value"):
        try:
            x = x.to_value(unit) if unit else x.to_value()
        except TypeError:
            # Astropy Time/TimeDelta needs a unit/format; default to days
            x = x.to_value(unit or "day")
    elif hasattr(x, "value"):
        x = x.value
    return np.asarray(x, dtype=float).ravel()
# ---------- small helpers ----------
def _get_target_id():
    for k in ("target_id", "tic_id", "TIC", "target"):
        if k in globals():
            return str(globals()[k])
    for name in ("lc_pdcsap", "lc_sap_cbv"):
        if name in globals():
            lc = globals()[name]
            meta = getattr(lc, "meta", {}) or {}
            for key in ("TICID", "TARGETID", "OBJECT", "TARGET"):
                if isinstance(meta, dict) and key in meta and meta[key] is not None:
                    return str(meta[key])
    return "unknown_target"

def _append_csv(path, row, fieldnames):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    write_header = not os.path.exists(path)
    with open(path, "a", newline="") as f:
        w = csv.DictWriter(f, fieldnames=fieldnames)
        if write_header:
            w.writeheader()
        w.writerow(row)

# ---------- paths ----------
TARGET = _get_target_id()
STAMP  = dt.datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
OUTDIR = os.path.join("results", TARGET)
os.makedirs(OUTDIR, exist_ok=True)
CSV_PATH = os.path.join(OUTDIR, "summary.csv")

# ---------- plotting ----------
def _save_periodogram(res, label):
    periods = getattr(res, "periods", None)
    power   = getattr(res, "power",   None)
    if periods is None or power is None:
        return None
    fig, ax = plt.subplots(figsize=(10,6))
    ax.plot(_to_numpy_1d(periods), _to_numpy_1d(power), lw=1.25)
    ax.axvline(_to_float(getattr(res, "period", np.nan)), ls="--", lw=1.25)
    ax.set_xlabel("Period [days]")
    ax.set_ylabel("TLS power (SDE proxy)")
    ax.set_title(f"{label} periodogram — best P = {_to_float(res.period):.5f} d")
    fpath = os.path.join(OUTDIR, f"{TARGET}_{label}_{STAMP}_periodogram.png")
    fig.savefig(fpath, dpi=150, bbox_inches="tight")
    plt.close(fig)
    return fpath

def _save_phase_fold(lc, res, label, nbins=120):
    """Fold using Lightkurve, handle TimeDelta/Quantity, and bin for clarity."""
    P = _to_float(getattr(res, "period", np.nan))
    T0 = _to_float(getattr(res, "T0", getattr(res, "t0", np.nan)))
    folded = lc.fold(period=P, t0=T0)

    # Phase array: prefer folded.phase, else derive from folded.time (days)
    if hasattr(folded, "phase") and folded.phase is not None:
        phase = _to_numpy_1d(folded.phase)  # already in cycles [-0.5, 0.5]
    else:
        tdays = _to_numpy_1d(getattr(folded, "time", []), unit="day")
        phase = ((tdays / P + 0.5) % 1.0) - 0.5

    flux  = _to_numpy_1d(folded.flux)

    # Robust y-lims
    if np.isfinite(flux).sum() > 10:
        p_lo, p_hi = np.nanpercentile(flux, [0.2, 99.8])
        pad = 0.002 * max(p_hi - p_lo, 1e-9)
        ylo, yhi = p_lo - pad, p_hi + pad
    else:
        ylo, yhi = np.nanmin(flux), np.nanmax(flux)

    # Median-bin
    bins = np.linspace(-0.5, 0.5, nbins+1)
    which = np.digitize(phase, bins) - 1
    bin_c = 0.5*(bins[1:] + bins[:-1])
    bin_f = np.array([np.nanmedian(flux[which==i]) if np.any(which==i) else np.nan
                      for i in range(nbins)])

    fig, ax = plt.subplots(figsize=(10,6))
    ax.scatter(phase, flux, s=3, alpha=0.25)
    ax.plot(bin_c, bin_f, lw=2)
    ax.set_xlim(-0.5, 0.5)
    if np.isfinite(ylo) and np.isfinite(yhi):
        ax.set_ylim(ylo, yhi)
    ax.set_xlabel("Phase [cycles]")
    ax.set_ylabel("Relative flux")
    ax.set_title(f"{label} phase-folded @ P={P:.5f} d, T0={T0:.5f}")
    fpath = os.path.join(OUTDIR, f"{TARGET}_{label}_{STAMP}_phasefold.png")
    fig.savefig(fpath, dpi=150, bbox_inches="tight")
    plt.close(fig)
    return fpath

# ---------- pack metrics & save everything ----------
def _save_all(label, res, lc):
    depth = _to_float(getattr(res, "depth", np.nan))
    metrics = dict(
        run_utc=STAMP,
        target=TARGET,
        label=label,
        period=_to_float(getattr(res, "period", np.nan)),
        sde=_to_float(getattr(res, "SDE", getattr(res, "sde", np.nan))),
        snr=_to_float(getattr(res, "SNR", getattr(res, "snr", np.nan))),
        n_transits=int(getattr(res, "transit_count", getattr(res, "n_transits", -1))),
        depth_frac=depth,
        depth_ppm=(depth * 1e6) if np.isfinite(depth) else np.nan,
        duration_d=_to_float(getattr(res, "duration", getattr(res, "duration_d", np.nan)), unit="day"),
        t0=_to_float(getattr(res, "T0", getattr(res, "t0", np.nan))),
    )

    json_path = os.path.join(OUTDIR, f"{TARGET}_{label}_{STAMP}_metrics.json")
    with open(json_path, "w") as jf:
        json.dump(metrics, jf, indent=2)

    csv_fields = ["run_utc","target","label","period","sde","snr","n_transits",
                  "depth_frac","depth_ppm","duration_d","t0"]
    _append_csv(CSV_PATH, metrics, csv_fields)

    pgram = _save_periodogram(res, label)
    pfold = _save_phase_fold(lc, res, label)

    print(f"[saved] {label}:")
    print(f"  JSON : {json_path}")
    print(f"  CSV  : {CSV_PATH}")
    if pgram: print(f"  Pgram: {pgram}")
    if pfold: print(f"  Fold : {pfold}")

# ---------- detect what exists in memory and save ----------
found_any = False
if "pdcsap_res" in globals() and "lc_pdcsap" in globals():
    _save_all("PDCSAP", pdcsap_res, lc_pdcsap); found_any = True

if "sap_res" in globals() and "lc_sap_cbv" in globals():
    plabel = (getattr(getattr(lc_sap_cbv, "meta", {}), "get", lambda *_: None)("pipeline")
              or "SAP+flatten")
    _save_all(plabel, sap_res, lc_sap_cbv); found_any = True

if not found_any:
    raise RuntimeError("Nothing to save: expected (pdcsap_res, lc_pdcsap) and/or (sap_res, lc_sap_cbv).")

[saved] PDCSAP:
  JSON : results/119584412/119584412_PDCSAP_20250925T010334Z_metrics.json
  CSV  : results/119584412/summary.csv
  Pgram: results/119584412/119584412_PDCSAP_20250925T010334Z_periodogram.png
  Fold : results/119584412/119584412_PDCSAP_20250925T010334Z_phasefold.png
[saved] SAP+flatten:
  JSON : results/119584412/119584412_SAP+flatten_20250925T010334Z_metrics.json
  CSV  : results/119584412/summary.csv
  Pgram: results/119584412/119584412_SAP+flatten_20250925T010334Z_periodogram.png
  Fold : results/119584412/119584412_SAP+flatten_20250925T010334Z_phasefold.png


In [37]:
# === Target A • Combine sectors • TLS near catalog P • Refit (P, T0) with covariance ===
# Minimal-verbosity; saves artifacts under results/TIC{TIC}/ and figures/TIC{TIC}/

import os, json, csv, warnings, math, datetime as dt
import numpy as np
import matplotlib.pyplot as plt
from astropy.stats import mad_std
from lightkurve import search_lightcurvefile, LightCurveCollection
from transitleastsquares import transitleastsquares

warnings.filterwarnings("ignore", category=UserWarning)
plt.rcParams.update({"figure.dpi": 120})

# ---------- USER KNOBS ----------
TIC          = 119584412                 # Target A
SECTORS_HINT = None                      # e.g., [22, 49]; set None to auto-use all available
CATALOG_P    = 16.027187                 # "catalog" or prior guess period (days)
WINDOW_FRAC  = 0.01                      # ±1% search window around the catalog period
DUR_HOURS    = 6.5                       # rough duration for masks & display (hours)
QUALITY_BITS = 175                       # SPOC quality bitmask (keep good)
FLAT_WIN_D   = 1.0                       # detrend window (days), gentle so we don't erase dips
POLY_ORDER   = 2

# ---------- PATHS ----------
STAMP   = dt.datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
RDIR    = f"results/TIC{TIC}"
FDIR    = f"figures/TIC{TIC}"
os.makedirs(RDIR, exist_ok=True)
os.makedirs(FDIR, exist_ok=True)

# ---------- HELPERS ----------
def _npify(arr):
    """Convert Quantity/Masked to plain np.ndarray; masked/invalid -> np.nan."""
    # Handle astropy masked arrays or numpy masked arrays
    if np.ma.isMaskedArray(arr):
        return np.asarray(np.ma.filled(arr, np.nan))
    # Handle astropy Quantity with .value
    if hasattr(arr, "value"):
        arr = arr.value
        if np.ma.isMaskedArray(arr):
            return np.asarray(np.ma.filled(arr, np.nan))
    return np.asarray(arr)

def _gentle_flatten(lc, window_days=1.0, polyorder=2):
    """Savitzky–Golay-like sliding poly via Lightkurve, but gently."""
    # estimate cadence in days; guard against divide-by-zero
    t = _npify(lc.time)
    dt_med = float(np.nanmedian(np.diff(t))) if t.size > 1 else 0.001
    w = int(round(window_days / (dt_med if dt_med > 0 else 0.001)))
    w = max(7, w)
    if w % 2 == 0:
        w += 1  # Lightkurve prefers odd window length
    try:
        flat = lc.flatten(window_length=w, polyorder=polyorder, return_trend=False)
    except TypeError:
        flat = lc.flatten(window_length=w, polyorder=polyorder)
    return flat.remove_nans().normalize()

def _load_stitched_pdcsap(tic, sectors_hint=None, quality_bits=175):
    sr = search_lightcurvefile(f"TIC {tic}", mission="TESS", author="SPOC")
    if len(sr) == 0:
        raise RuntimeError("No SPOC LightCurveFiles found.")
    files = sr.download_all()

    lcs = []
    for f in files:
        # optional sector filter
        if sectors_hint is not None:
            sec = getattr(f, "sector", None)
            if (sec is None) or (sec not in sectors_hint):
                continue

        # pull PDCSAP
        lc = f.PDCSAP_FLUX

        # cleanup (be version-tolerant)
        if hasattr(lc, "remove_nans"):
            lc = lc.remove_nans()
        if hasattr(lc, "remove_outliers"):
            try:
                lc = lc.remove_outliers(sigma=10)
            except Exception:
                pass

        # quality bitmask
        if hasattr(lc, "quality"):
            good = (lc.quality & quality_bits) == 0
            lc = lc[good]

        if len(lc.time) == 0:
            continue

        lcs.append(lc.normalize())

    if not lcs:
        raise RuntimeError("No usable PDCSAP cadences after quality mask.")
    return LightCurveCollection(lcs).stitch().remove_nans().normalize()

def _median_bin(x, y, nbins=200):
    x = _npify(x); y = _npify(y)
    good = np.isfinite(x) & np.isfinite(y)
    x, y = x[good], y[good]
    edges = np.linspace(np.nanmin(x), np.nanmax(x), nbins+1)
    idx   = np.digitize(x, edges)-1
    bx, by = [], []
    for i in range(nbins):
        sel = idx == i
        if np.any(sel):
            bx.append(np.nanmedian(x[sel]))
            by.append(np.nanmedian(y[sel]))
    return np.asarray(bx), np.asarray(by)

def _find_midtimes_quadratic(time_btjd, flux_norm, period, t0, dur_hours, k_half=1.0):
    """
    Per-event Tmid finder:
      - select windows around predicted midtimes (±k_half * duration/2)
      - detrend locally (linear)
      - fit quadratic to lowest ~30% of points -> vertex time = Tmid
      - estimate per-event sigma_Tmid from bootstrap-of-residuals
    Returns: epochs[], tmids[], tmid_errs[]
    """
    time_btjd = _npify(time_btjd)
    flux_norm = _npify(flux_norm)
    good0 = np.isfinite(time_btjd) & np.isfinite(flux_norm)
    time_btjd, flux_norm = time_btjd[good0], flux_norm[good0]

    dur_days = dur_hours/24.0
    # epochs that fall within data span
    kmin = math.floor((time_btjd.min() - t0)/period) - 1
    kmax = math.ceil((time_btjd.max() - t0)/period) + 1
    epochs, tmids, terrs = [], [], []

    for k in range(kmin, kmax+1):
        tc = t0 + k*period
        w  = (np.abs(time_btjd - tc) <= 0.5*k_half*dur_days)
        if np.count_nonzero(w) < 8:
            continue
        t  = time_btjd[w]
        f  = flux_norm[w]

        # remove any remaining NaNs/masked
        good = np.isfinite(t) & np.isfinite(f)
        t, f = t[good], f[good]
        if t.size < 8:
            continue

        # local linear detrend
        A = np.vstack([np.ones_like(t), t - np.nanmedian(t)]).T
        try:
            coeff, *_ = np.linalg.lstsq(A, f, rcond=None)
        except Exception:
            continue
        f_det = f - (A @ coeff)

        # take lowest ~30% points for quadratic
        q = np.nanpercentile(f_det, 30)
        sel = f_det <= q
        if np.count_nonzero(sel) < 6:
            continue
        tt = t[sel] - np.nanmedian(t[sel])
        ff = f_det[sel]

        # quadratic fit: a*tt^2 + b*tt + c
        X = np.vstack([tt**2, tt, np.ones_like(tt)]).T
        try:
            p, *_ = np.linalg.lstsq(X, ff, rcond=None)
        except Exception:
            continue
        a, b, c = p
        if a == 0:
            continue
        tmid = -b/(2*a) + np.nanmedian(t[sel])

        # rough uncertainty via bootstrap of residuals
        model = a*tt**2 + b*tt + c
        res   = ff - model
        if len(res) < 6:
            continue
        draws = []
        rng = np.random.default_rng(42)
        for _ in range(200):
            res_s = rng.choice(res, size=res.size, replace=True)
            ff_s  = model + res_s
            try:
                p_s, *_ = np.linalg.lstsq(X, ff_s, rcond=None)
            except Exception:
                continue
            a_s, b_s, c_s = p_s
            if a_s == 0:
                continue
            draws.append(-b_s/(2*a_s) + np.nanmedian(t[sel]))
        if len(draws) < 10:
            continue
        epochs.append(k)
        tmids.append(tmid)
        terrs.append(np.nanstd(draws, ddof=1))
    return np.asarray(epochs), np.asarray(tmids), np.asarray(terrs)

def _fit_linear_ephemeris(epochs, tmids, terrs):
    """Weighted least squares fit of T(n)=T0+nP; returns P, T0, Cov 2x2, redchisq."""
    if len(tmids) < 2:
        raise RuntimeError("Not enough mid-times to refit ephemeris.")
    w   = 1.0/np.clip(terrs, 1e-6, np.inf)**2
    X   = np.vstack([epochs, np.ones_like(epochs)]).T  # slope=P, intercept=T0
    XT  = (X.T*w)
    beta= np.linalg.inv(XT@X) @ (XT@tmids)
    P, T0 = beta[0], beta[1]
    yhat  = X@beta
    resid = tmids - yhat
    dof   = max(1, len(tmids)-2)
    chi2  = np.sum((resid**2)*w)
    rchi2 = chi2/dof
    Cov   = np.linalg.inv(XT@X) * rchi2  # inflate by reduced chi^2
    return P, T0, Cov, rchi2

def _save_json(path, obj):
    with open(path, "w") as f:
        json.dump(obj, f, indent=2)

# ---------- 1) LOAD & FLATTEN (gentle) ----------
print(f"[load] TIC {TIC} — stitching PDCSAP (sectors: {'all' if not SECTORS_HINT else SECTORS_HINT})")
lc = _load_stitched_pdcsap(TIC, sectors_hint=SECTORS_HINT, quality_bits=QUALITY_BITS)
lc_flat = _gentle_flatten(lc, window_days=FLAT_WIN_D, polyorder=POLY_ORDER)

# ---------- 2) TLS in a narrow window around catalog P ----------
pmin = CATALOG_P * (1.0 - WINDOW_FRAC)
pmax = CATALOG_P * (1.0 + WINDOW_FRAC)

time = _npify(lc_flat.time)      # BTJD (days) → plain numpy
flux = _npify(lc_flat.flux)      # normalized → plain numpy
good = np.isfinite(time) & np.isfinite(flux)
time, flux = time[good], flux[good]

print(f"[tls] Narrow search around P={CATALOG_P:.6f} d (window ±{100*WINDOW_FRAC:.1f}% → [{pmin:.5f}, {pmax:.5f}] d)")

tls = transitleastsquares(time, flux)

# TLS requires an integer >= 1 for use_threads (some builds don't accept "auto")
threads = max(1, (os.cpu_count() or 1) - 1)

# Run TLS with version tolerance for show_progress_bar
try:
    tls_res = tls.power(
        period_min=pmin,
        period_max=pmax,
        oversampling_factor=5,
        n_transits_min=2,
        use_threads=threads,
        show_progress_bar=False,
    )
except TypeError:
    tls_res = tls.power(
        period_min=pmin,
        period_max=pmax,
        oversampling_factor=5,
        n_transits_min=2,
        use_threads=threads,
    )

bestP  = float(tls_res.period)
bestT0 = float(tls_res.T0)
bestSDE = float(getattr(tls_res, "SDE", getattr(tls_res, "sde", np.nan)))
print(f"[tls] best P={bestP:.6f} d, T0={bestT0:.5f} BTJD, SDE≈{bestSDE:.2f} (threads={threads})")

# Save TLS periodogram
fig = plt.figure(figsize=(8.6,5.0))
plt.plot(tls_res.periods, tls_res.power, lw=1)
plt.axvline(bestP, ls="--")
plt.xlabel("Period [days]"); plt.ylabel("TLS power (SDE proxy)")
plt.title(f"TIC {TIC} — TLS around {CATALOG_P:.5f} d (best {bestP:.5f} d)")
plt.tight_layout()
pgram_path = os.path.join(FDIR, f"TIC{TIC}_TLS_narrow_{STAMP}.png")
fig.savefig(pgram_path, dpi=150); plt.close(fig)

# Save a phase-folded figure at TLS best
folded = lc_flat.fold(period=bestP, epoch_time=bestT0)
phase  = _npify(getattr(folded, "phase", None))
flux_f = _npify(getattr(folded, "flux", None))
goodf  = np.isfinite(phase) & np.isfinite(flux_f)
phase, flux_f = phase[goodf], flux_f[goodf]
px, py = _median_bin(phase, flux_f, nbins=180)
fig = plt.figure(figsize=(8.6,5.0))
plt.scatter(phase, flux_f, s=4, alpha=0.15)
plt.plot(px, py, lw=1.8)
dur_phase = (DUR_HOURS/24.0)/bestP
plt.axvspan(-0.5*dur_phase, 0.5*dur_phase, alpha=0.15)
plt.xlabel("Phase [cycles]"); plt.ylabel("Relative flux")
plt.title(f"TIC {TIC} — phase-fold @ P={bestP:.5f} d, T0={bestT0:.5f}")
plt.tight_layout()
fold_path = os.path.join(FDIR, f"TIC{TIC}_TLS_narrow_phase_{STAMP}.png")
fig.savefig(fold_path, dpi=150); plt.close(fig)

# ---------- 3) Refit P, T0 from per-event midtimes with full covariance ----------
print("[fit] estimating per-event mid-times and refitting linear ephemeris …")
epochs, tmids, terrs = _find_midtimes_quadratic(time, flux, bestP, bestT0, DUR_HOURS, k_half=1.7)
if len(tmids) < 2:
    raise RuntimeError(f"Only {len(tmids)} mid-times found; need ≥2 to fit ephemeris.")

P_fit, T0_fit, Cov, rchi2 = _fit_linear_ephemeris(epochs, tmids, terrs)
cov_dict = {"C00_T0": Cov[1,1], "C11_P": Cov[0,0], "C01_T0P": Cov[1,0]}

print(f"[fit] P = {P_fit:.8f} d, T0 = {T0_fit:.6f} BTJD, rχ² = {rchi2:.2f}")
print(f"[fit] σ(P) = {math.sqrt(Cov[0,0]):.6e} d, σ(T0) = {math.sqrt(Cov[1,1]):.6e} d, ρ = {Cov[0,1]/math.sqrt(Cov[0,0]*Cov[1,1]):.3f}")

# Save midtimes CSV
mt_csv = os.path.join(RDIR, f"TIC{TIC}_midtimes_{STAMP}.csv")
with open(mt_csv, "w", newline="") as f:
    w = csv.writer(f)
    w.writerow(["epoch", "tmid_btjd", "tmid_err_d"])
    for n, tmid, terr in zip(epochs, tmids, terrs):
        w.writerow([int(n), f"{tmid:.8f}", f"{terr:.8e}"])

# ---------- Save ephemeris JSON ----------
ephem_json = dict(
    tic=TIC,
    tls_narrow=dict(
        catalog_P=CATALOG_P,
        search_window_frac=WINDOW_FRAC,
        best_P=bestP,
        best_T0=bestT0,
        SDE=bestSDE,
        periodogram=pgram_path,
        phase_plot=fold_path,
    ),
    fit=dict(
        P=P_fit,
        T0=T0_fit,
        rchisq=rchi2,
        cov=cov_dict,
        n_midtimes=int(len(tmids)),
    ),
    detrend=dict(kind="sliding_poly", window_days=FLAT_WIN_D, polyorder=POLY_ORDER),
    quality_bits=QUALITY_BITS,
    run_utc=STAMP,
)
ephem_path = os.path.join(RDIR, f"TIC{TIC}_refit_ephemeris_{STAMP}.json")
_save_json(ephem_path, ephem_json)

# ---------- Simple O–C plot ----------
oc = tmids - (T0_fit + epochs * P_fit)
fig = plt.figure(figsize=(8.0, 4.8))
plt.errorbar(epochs, oc * 24 * 60, yerr=terrs * 24 * 60, fmt="o", ms=4, capsize=2)
plt.axhline(0, color="k", lw=1, ls="--")
plt.xlabel("Epoch (n)"); plt.ylabel("O–C [minutes]")
plt.title(f"TIC {TIC} — O–C after refit (P={P_fit:.6f} d)")
plt.tight_layout()
oc_path = os.path.join(FDIR, f"TIC{TIC}_OC_{STAMP}.png")
fig.savefig(oc_path, dpi=150); plt.close(fig)

print("\n[done]")
print(f"  TLS periodogram : {pgram_path}")
print(f"  Phase-fold plot : {fold_path}")
print(f"  Midtimes CSV    : {mt_csv}")
print(f"  Ephemeris JSON  : {ephem_path}")
print(f"  O–C plot        : {oc_path}")

[load] TIC 119584412 — stitching PDCSAP (sectors: all)
[tls] Narrow search around P=16.027187 d (window ±1.0% → [15.86692, 16.18746] d)
Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 29 durations
Searching 29374 data points, 401 periods from 15.868 to 16.187 days
Using 7 of 8 CPU threads
Searching for best T0 for period 15.96573 days
[tls] best P=15.965733 d, T0=1908.86451 BTJD, SDE≈5.11 (threads=7)
[fit] estimating per-event mid-times and refitting linear ephemeris …
[fit] P = 15.96593436 d, T0 = 1908.858885 BTJD, rχ² = 0.07
[fit] σ(P) = 4.319040e-04 d, σ(T0) = 1.587372e-02 d, ρ = -0.794

[done]
  TLS periodogram : figures/TIC119584412/TIC119584412_TLS_narrow_20250927T202323Z.png
  Phase-fold plot : figures/TIC119584412/TIC119584412_TLS_narrow_phase_20250927T202323Z.png
  Midtimes CSV    : results/TIC119584412/TIC119584412_midtimes_20250927T202323Z.csv
  Ephemeris JSON  : results/TIC119584412/TIC119584412_refit_ephemeris_20250927T202323Z.json
  O–C plot        : 

In [38]:
# === Target A • Combine sectors • TLS near catalog P • Refit (P, T0) with covariance ===
# Minimal-verbosity; saves artifacts under results/TIC{TIC}/ and figures/TIC{TIC}/

import os, json, csv, warnings, math, datetime as dt
import numpy as np
import matplotlib.pyplot as plt
from lightkurve import search_lightcurvefile, LightCurveCollection
from transitleastsquares import transitleastsquares

warnings.filterwarnings("ignore", category=UserWarning)
plt.rcParams.update({"figure.dpi": 120})

# ---------- USER KNOBS ----------
TIC             = 119584412                # Target A
SECTORS_HINT    = None                     # e.g., [22, 49]; set None to auto-use all available
CATALOG_P       = 16.027187                # prior/catalog period (days)
WINDOW_FRAC     = 0.01                     # ±1% TLS window around CATALOG_P
DUR_HOURS       = 6.5                      # rough duration (hours)
QUALITY_BITS    = 175                      # SPOC quality bitmask (keep good)
FLAT_WIN_D      = 1.0                      # detrend window (days)
POLY_ORDER      = 2
PROTECT_IN_TRANSIT = True                  # re-flatten excluding in-transit points around TLS best

# ---------- PATHS ----------
STAMP = dt.datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
RDIR  = f"results/TIC{TIC}"
FDIR  = f"figures/TIC{TIC}"
os.makedirs(RDIR, exist_ok=True)
os.makedirs(FDIR, exist_ok=True)

# ---------- HELPERS ----------
def _tls_threads():
    """Use N-1 cores, but at least 1."""
    n = (os.cpu_count() or 1) - 1
    return max(1, n)

def _gentle_flatten(lc, window_days=1.0, polyorder=2):
    """Lightkurve flatten with conservative window; avoids shaving ~hour dips."""
    # Estimate window length in cadences (min 7 to satisfy LK)
    dt_med = np.nanmedian(np.diff(lc.time.value)) or 1e-3
    wl = int(max(7, round(window_days / dt_med)))
    try:
        flat = lc.flatten(window_length=wl, polyorder=polyorder, return_trend=False)
    except TypeError:
        flat = lc.flatten(window_length=wl, polyorder=polyorder)
    return flat.remove_nans().normalize()

def _load_stitched_pdcsap(tic, sectors_hint=None, quality_bits=175):
    sr = search_lightcurvefile(f"TIC {tic}", mission="TESS", author="SPOC")
    if len(sr) == 0:
        raise RuntimeError("No SPOC LightCurveFiles found.")
    files = sr.download_all()

    lcs = []
    for f in files:
        # optional sector filter
        if sectors_hint is not None:
            sec = getattr(f, "sector", None)
            if (sec is None) or (sec not in sectors_hint):
                continue

        lc = f.PDCSAP_FLUX
        # cleanup (version tolerant)
        if hasattr(lc, "remove_nans"):
            lc = lc.remove_nans()
        if hasattr(lc, "remove_outliers"):
            try:
                lc = lc.remove_outliers(sigma=10)
            except Exception:
                pass
        # quality bitmask
        if hasattr(lc, "quality"):
            good = (lc.quality & quality_bits) == 0
            lc = lc[good]
        if len(lc.time) == 0:
            continue
        lcs.append(lc.normalize())

    if not lcs:
        raise RuntimeError("No usable PDCSAP cadences after quality mask.")
    return LightCurveCollection(lcs).stitch().remove_nans().normalize()

def _median_bin(x, y, nbins=200):
    edges = np.linspace(np.nanmin(x), np.nanmax(x), nbins+1)
    idx   = np.digitize(x, edges) - 1
    outx, outy = [], []
    for i in range(nbins):
        sel = idx == i
        if np.any(sel):
            outx.append(np.nanmedian(x[sel]))
            outy.append(np.nanmedian(y[sel]))
    return np.asarray(outx), np.asarray(outy)

def _find_midtimes_quadratic(time_btjd, flux_norm, period, t0, dur_hours, k_half=1.0):
    """
    Per-event Tmid finder:
      - window around predicted T0+kP (±k_half * duration/2)
      - local linear detrend
      - quadratic fit to lowest ~30% points -> vertex = Tmid
      - bootstrap residuals for σ_Tmid
    """
    dur_days = dur_hours/24.0
    kmin = math.floor((time_btjd.min() - t0)/period) - 1
    kmax = math.ceil((time_btjd.max() - t0)/period) + 1
    epochs, tmids, terrs = [], [], []

    rng = np.random.default_rng(42)
    tb = np.asarray(time_btjd, float)
    fb = np.asarray(flux_norm, float)

    for k in range(kmin, kmax+1):
        tc = t0 + k*period
        w  = (np.abs(tb - tc) <= 0.5*k_half*dur_days)
        if np.count_nonzero(w) < 8:
            continue
        t = tb[w].astype(float)
        f = fb[w].astype(float)

        # local linear detrend: f ~ a + b*(t - median)
        A = np.vstack([np.ones_like(t), t - np.nanmedian(t)]).T
        coeff, *_ = np.linalg.lstsq(A, f, rcond=None)
        f_det = f - (A @ coeff)

        # take lowest ~30% points
        q = np.nanpercentile(f_det, 30.0)
        sel = f_det <= q
        if np.count_nonzero(sel) < 6:
            continue
        tt = t[sel] - np.nanmedian(t[sel])
        ff = f_det[sel]

        # quadratic fit: a*tt^2 + b*tt + c
        X = np.vstack([tt**2, tt, np.ones_like(tt)]).T
        p, *_ = np.linalg.lstsq(X, ff, rcond=None)
        a, b, c = p
        if a == 0:
            continue
        tmid = -b/(2*a) + np.nanmedian(t[sel])

        # bootstrap uncertainty
        model = a*tt**2 + b*tt + c
        res   = ff - model
        if len(res) < 6:
            continue
        draws = []
        for _ in range(200):
            res_s = rng.choice(res, size=res.size, replace=True)
            ff_s  = model + res_s
            p_s, *_ = np.linalg.lstsq(X, ff_s, rcond=None)
            a_s, b_s, c_s = p_s
            if a_s == 0:
                continue
            draws.append(-b_s/(2*a_s) + np.nanmedian(t[sel]))
        if len(draws) < 10:
            continue

        epochs.append(k)
        tmids.append(tmid)
        terrs.append(np.nanstd(draws, ddof=1))

    return np.asarray(epochs), np.asarray(tmids), np.asarray(terrs)

def _fit_linear_ephemeris(epochs, tmids, terrs):
    """Weighted least squares fit of T(n)=T0+nP; returns P, T0, Cov 2x2, rchi2."""
    if len(tmids) < 2:
        raise RuntimeError("Not enough mid-times to refit ephemeris.")
    w   = 1.0/np.clip(terrs, 1e-6, np.inf)**2
    X   = np.vstack([epochs, np.ones_like(epochs)]).T  # slope=P, intercept=T0
    XT  = (X.T * w)
    beta= np.linalg.inv(XT @ X) @ (XT @ tmids)
    P, T0 = beta[0], beta[1]
    yhat  = X @ beta
    resid = tmids - yhat
    dof   = max(1, len(tmids)-2)
    chi2  = np.sum((resid**2) * w)
    rchi2 = chi2 / dof

    # Only inflate when rchi2 > 1 (do not shrink when rchi2 < 1)
    Cov = np.linalg.inv(XT @ X)
    if rchi2 > 1:
        Cov = Cov * rchi2
    return P, T0, Cov, rchi2

def _save_json(path, obj):
    with open(path, "w") as f:
        json.dump(obj, f, indent=2)

# ---------- 1) LOAD ----------
print(f"[load] TIC {TIC} — stitching PDCSAP (sectors: {'all' if not SECTORS_HINT else SECTORS_HINT})")
lc = _load_stitched_pdcsap(TIC, sectors_hint=SECTORS_HINT, quality_bits=QUALITY_BITS)

# ---------- 2) FIRST FLATTEN & TLS (narrow window) ----------
lc_flat = _gentle_flatten(lc, window_days=FLAT_WIN_D, polyorder=POLY_ORDER)
pmin = CATALOG_P * (1.0 - WINDOW_FRAC)
pmax = CATALOG_P * (1.0 + WINDOW_FRAC)
time = lc_flat.time.value
flux = lc_flat.flux.value
print(f"[tls] Narrow search around P={CATALOG_P:.6f} d (±{100*WINDOW_FRAC:.1f}%)")

tls = transitleastsquares(time, flux)
tls_res = tls.power(
    period_min=pmin,
    period_max=pmax,
    oversampling_factor=5,
    use_threads=_tls_threads(),
    show_progress_bar=False
)
bestP  = float(tls_res.period)
bestT0 = float(tls_res.T0)
bestSDE= float(getattr(tls_res, "SDE", getattr(tls_res, "sde", np.nan)))
print(f"[tls] best P={bestP:.5f} d, T0={bestT0:.5f} BTJD, SDE≈{bestSDE:.2f} (threads={_tls_threads()})")

# ---------- Optional: re-flatten protecting in-transit windows & re-run TLS ----------
if PROTECT_IN_TRANSIT:
    dur_days = DUR_HOURS/24.0
    t_all = lc.time.value
    mask_in = np.zeros_like(t_all, dtype=bool)
    kmin = int(np.floor((t_all.min()-bestT0)/bestP))-1
    kmax = int(np.ceil ((t_all.max()-bestT0)/bestP))+1
    for k in range(kmin, kmax+1):
        tc = bestT0 + k*bestP
        mask_in |= np.abs(t_all - tc) < 0.6*dur_days  # protect ~60% of est. duration
    lc_flat = _gentle_flatten(lc[~mask_in], window_days=FLAT_WIN_D, polyorder=POLY_ORDER)

    # update arrays and re-run TLS in same narrow window
    time = lc_flat.time.value
    flux = lc_flat.flux.value
    tls   = transitleastsquares(time, flux)
    tls_res = tls.power(
        period_min=pmin, period_max=pmax,
        oversampling_factor=5,
        use_threads=_tls_threads(),
        show_progress_bar=False
    )
    bestP  = float(tls_res.period)
    bestT0 = float(tls_res.T0)
    bestSDE= float(getattr(tls_res, "SDE", getattr(tls_res, "sde", np.nan)))
    print(f"[tls] (protected) best P={bestP:.5f} d, T0={bestT0:.5f} BTJD, SDE≈{bestSDE:.2f}")

# ---------- Save TLS periodogram ----------
fig = plt.figure(figsize=(8.6,5.0))
plt.plot(tls_res.periods, tls_res.power, lw=1)
plt.axvline(bestP, ls="--")
plt.xlabel("Period [days]")
plt.ylabel("TLS power (SDE proxy)")
plt.title(f"TIC {TIC} — TLS around {CATALOG_P:.5f} d (best {bestP:.5f} d)")
plt.tight_layout()
pgram_path = os.path.join(FDIR, f"TIC{TIC}_TLS_narrow_{STAMP}.png")
fig.savefig(pgram_path, dpi=150); plt.close(fig)

# ---------- Phase-fold plot (wrapped to one cycle) ----------
folded = lc_flat.fold(period=bestP, epoch_time=bestT0)
phase = folded.phase.value if hasattr(folded.phase, "value") else folded.phase
phase = ((np.asarray(phase) + 0.5) % 1.0) - 0.5  # wrap to [-0.5, 0.5)
px, py = _median_bin(phase, folded.flux.value, nbins=180)

fig = plt.figure(figsize=(8.6,5.0))
plt.scatter(phase, folded.flux.value, s=4, alpha=0.15)
plt.plot(px, py, lw=1.8)
dur_phase = (DUR_HOURS/24.0)/bestP
plt.axvspan(-0.5*dur_phase, 0.5*dur_phase, alpha=0.15)
plt.axvline(0.0, color="k", lw=0.8, alpha=0.4)
plt.xlabel("Phase [cycles]"); plt.ylabel("Relative flux")
plt.title(f"TIC {TIC} — phase-fold @ P={bestP:.5f} d, T0={bestT0:.5f}")
plt.tight_layout()
fold_path = os.path.join(FDIR, f"TIC{TIC}_TLS_narrow_phase_{STAMP}.png")
fig.savefig(fold_path, dpi=150); plt.close(fig)

# ---------- 3) Per-event midtimes & linear ephemeris fit ----------
print("[fit] estimating per-event mid-times and refitting linear ephemeris …")
epochs, tmids, terrs = _find_midtimes_quadratic(time, flux, bestP, bestT0, DUR_HOURS, k_half=1.7)
if len(tmids) < 2:
    raise RuntimeError(f"Only {len(tmids)} mid-times found; need ≥2 to fit ephemeris.")

P_fit, T0_fit, Cov, rchi2 = _fit_linear_ephemeris(epochs, tmids, terrs)
cov_dict = {"C00_T0": float(Cov[1,1]), "C11_P": float(Cov[0,0]), "C01_T0P": float(Cov[1,0])}

print(f"[fit] P = {P_fit:.8f} d, T0 = {T0_fit:.6f} BTJD, rχ² = {rchi2:.2f}")
print(f"[fit] σ(P) = {math.sqrt(Cov[0,0]):.6e} d, σ(T0) = {math.sqrt(Cov[1,1]):.6e} d, ρ = {Cov[0,1]/math.sqrt(Cov[0,0]*Cov[1,1]):.3f}")

# ---------- Save midtimes CSV ----------
mt_csv = os.path.join(RDIR, f"TIC{TIC}_midtimes_{STAMP}.csv")
with open(mt_csv, "w", newline="") as f:
    w = csv.writer(f)
    w.writerow(["epoch", "tmid_btjd", "tmid_err_d"])
    for n, tmid, terr in zip(epochs, tmids, terrs):
        w.writerow([int(n), f"{tmid:.8f}", f"{terr:.8e}"])

# ---------- Save ephemeris JSON ----------
ephem_json = dict(
    tic=TIC,
    tls_narrow=dict(
        catalog_P=CATALOG_P,
        search_window_frac=WINDOW_FRAC,
        best_P=bestP,
        best_T0=bestT0,
        SDE=bestSDE,
        periodogram=pgram_path,
        phase_plot=fold_path,
        protected_in_transit=bool(PROTECT_IN_TRANSIT),
    ),
    fit=dict(
        P=P_fit, T0=T0_fit, rchisq=rchi2, cov=cov_dict,
        n_midtimes=int(len(tmids)),
    ),
    detrend=dict(kind="sliding_poly", window_days=FLAT_WIN_D, polyorder=POLY_ORDER),
    quality_bits=QUALITY_BITS,
    run_utc=STAMP,
)
ephem_path = os.path.join(RDIR, f"TIC{TIC}_refit_ephemeris_{STAMP}.json")
_save_json(ephem_path, ephem_json)

# ---------- Simple O–C plot ----------
oc = tmids - (T0_fit + epochs * P_fit)
fig = plt.figure(figsize=(8.0, 4.8))
plt.errorbar(epochs, oc * 24 * 60, yerr=terrs * 24 * 60, fmt="o", ms=4, capsize=2)
plt.axhline(0, color="k", lw=1, ls="--")
plt.xlabel("Epoch (n)"); plt.ylabel("O–C [minutes]")
plt.title(f"TIC {TIC} — O–C after refit (P={P_fit:.6f} d)")
plt.tight_layout()
oc_path = os.path.join(FDIR, f"TIC{TIC}_OC_{STAMP}.png")
fig.savefig(oc_path, dpi=150); plt.close(fig)

print("\n[done]")
print(f"  TLS periodogram : {pgram_path}")
print(f"  Phase-fold plot : {fold_path}")
print(f"  Midtimes CSV    : {mt_csv}")
print(f"  Ephemeris JSON  : {ephem_path}")
print(f"  O–C plot        : {oc_path}")

[load] TIC 119584412 — stitching PDCSAP (sectors: all)
[tls] Narrow search around P=16.027187 d (±1.0%)
Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 29 durations
Searching 29374 data points, 401 periods from 15.868 to 16.187 days
Using 7 of 8 CPU threads
Searching for best T0 for period 15.96573 days
[tls] best P=15.96573 d, T0=1908.86451 BTJD, SDE≈5.09 (threads=7)
Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 29 durations
Searching 28438 data points, 401 periods from 15.868 to 16.187 days
Using 7 of 8 CPU threads
Searching for best T0 for period 15.92682 days
[tls] (protected) best P=15.92682 d, T0=1900.05682 BTJD, SDE≈5.04
[fit] estimating per-event mid-times and refitting linear ephemeris …
[fit] P = 15.92584226 d, T0 = 1900.077494 BTJD, rχ² = 0.00
[fit] σ(P) = 1.948167e-02 d, σ(T0) = 1.972095e-01 d, ρ = -0.215

[done]
  TLS periodogram : figures/TIC119584412/TIC119584412_TLS_narrow_20250927T205655Z.png
  Phase-fold plot : figures/TIC11

In [39]:
# === Target B (TOI 260.01 / TIC 37749396) • Combine sectors • TLS near catalog P • Refit (P, T0) with covariance ===
# Minimal-verbosity; saves artifacts under results/TIC{TIC}/ and figures/TIC{TIC}/

import os, json, csv, warnings, math, datetime as dt
import numpy as np
import matplotlib.pyplot as plt
from lightkurve import search_lightcurvefile, LightCurveCollection
from transitleastsquares import transitleastsquares

warnings.filterwarnings("ignore", category=UserWarning)
plt.rcParams.update({"figure.dpi": 120})

# ---------- USER KNOBS ----------
TIC          = 37749396                  # Target B
SECTORS_HINT = None                      # e.g., [3, 42, 70]; set None to auto-use all available
CATALOG_P    = 13.475725                 # narrow-center period (days); from your stitched TLS top-1
WINDOW_FRAC  = 0.01                      # ±1% search window
DUR_HOURS    = 3.0                       # rough duration for masks & display (hours)
QUALITY_BITS = 175                       # SPOC quality bitmask (keep good)
FLAT_WIN_D   = 1.0                       # detrend window (days), gentle so we don't erase dips
POLY_ORDER   = 2

# ---------- PATHS ----------
STAMP   = dt.datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
RDIR    = f"results/TIC{TIC}"
FDIR    = f"figures/TIC{TIC}"
os.makedirs(RDIR, exist_ok=True)
os.makedirs(FDIR, exist_ok=True)

# ---------- HELPERS ----------
def _median_bin(x, y, nbins=180):
    x = np.asarray(x, float); y = np.asarray(y, float)
    lo, hi = np.nanmin(x), np.nanmax(x)
    edges = np.linspace(lo, hi, nbins+1)
    idx = np.digitize(x, edges) - 1
    px = 0.5*(edges[:-1] + edges[1:])
    py = np.full(nbins, np.nan)
    for i in range(nbins):
        m = y[idx == i]
        if m.size:
            py[i] = np.nanmedian(m)
    return px, py

def _gentle_flatten(lc, window_days=1.0, polyorder=2):
    # Lightkurve flatten with conservative window; avoids shaving ~hour dips
    try:
        flat = lc.flatten(
            window_length=int(max(7, round(window_days/(np.nanmedian(np.diff(lc.time.value)) or 1e-3)))),
            polyorder=polyorder,
            return_trend=False
        )
    except TypeError:
        flat = lc.flatten(
            window_length=int(max(7, round(window_days/(np.nanmedian(np.diff(lc.time.value)) or 1e-3)))),
            polyorder=polyorder
        )
    return flat.remove_nans().normalize()

def _load_stitched_pdcsap(tic, sectors_hint=None, quality_bits=175):
    sr = search_lightcurvefile(f"TIC {tic}", mission="TESS", author="SPOC")
    if len(sr) == 0:
        raise RuntimeError("No SPOC LightCurveFiles found.")
    files = sr.download_all()

    lcs = []
    for f in files:
        if sectors_hint is not None:
            sec = getattr(f, "sector", None)
            if (sec is None) or (sec not in sectors_hint):
                continue

        lc = f.PDCSAP_FLUX

        if hasattr(lc, "remove_nans"):
            lc = lc.remove_nans()
        if hasattr(lc, "remove_outliers"):
            try:
                lc = lc.remove_outliers(sigma=10)
            except Exception:
                pass

        if hasattr(lc, "quality"):
            good = (lc.quality & quality_bits) == 0
            lc = lc[good]

        if len(lc.time) == 0:
            continue

        lcs.append(lc.normalize())

    if not lcs:
        raise RuntimeError("No usable PDCSAP cadences after quality mask.")
    return LightCurveCollection(lcs).stitch().remove_nans().normalize()

def _find_midtimes_quadratic(time_btjd, flux_norm, period, t0, dur_hours, k_half=1.0):
    """
    Per-event Tmid finder:
      - select windows around predicted midtimes (±k_half * duration/2)
      - detrend locally (linear)
      - fit quadratic to lowest ~30% of points -> vertex time = Tmid
      - estimate per-event sigma_Tmid from bootstrap-of-residuals
    Returns: epochs[], tmids[], tmid_errs[]
    """
    tarr = np.asarray(time_btjd, float)
    farr = np.asarray(flux_norm, float)
    dur_days = dur_hours/24.0

    kmin = math.floor((tarr.min() - t0)/period) - 1
    kmax = math.ceil((tarr.max() - t0)/period) + 1
    epochs, tmids, terrs = [], [], []

    for k in range(kmin, kmax+1):
        tc = t0 + k*period
        w  = (np.abs(tarr - tc) <= 0.5*k_half*dur_days)
        if np.count_nonzero(w) < 8:
            continue
        t  = tarr[w]
        f  = farr[w]

        # local linear detrend
        A = np.vstack([np.ones_like(t), t - np.nanmedian(t)]).T
        coeff, *_ = np.linalg.lstsq(A, f, rcond=None)
        f_det = f - (A @ coeff)

        # take lowest ~30% points for quadratic
        q = np.nanpercentile(f_det, 30)
        sel = f_det <= q
        if np.count_nonzero(sel) < 6:
            continue
        tt = t[sel] - np.nanmedian(t[sel])
        ff = f_det[sel]

        X = np.vstack([tt**2, tt, np.ones_like(tt)]).T
        p, *_ = np.linalg.lstsq(X, ff, rcond=None)
        a, b, c = p
        if a == 0:
            continue
        tmid = -b/(2*a) + np.nanmedian(t[sel])

        # rough uncertainty via bootstrap of residuals
        model = a*tt**2 + b*tt + c
        res   = ff - model
        if len(res) < 6:
            continue
        draws = []
        rng = np.random.default_rng(42)
        for _ in range(200):
            res_s = rng.choice(res, size=res.size, replace=True)
            ff_s  = model + res_s
            p_s, *_ = np.linalg.lstsq(X, ff_s, rcond=None)
            a_s, b_s, c_s = p_s
            if a_s == 0:
                continue
            draws.append(-b_s/(2*a_s) + np.nanmedian(t[sel]))
        if len(draws) < 10:
            continue
        epochs.append(k)
        tmids.append(tmid)
        terrs.append(np.nanstd(draws, ddof=1))
    return np.asarray(epochs), np.asarray(tmids), np.asarray(terrs)

def _fit_linear_ephemeris(epochs, tmids, terrs):
    """Weighted least squares fit of T(n)=T0+nP; returns P, T0, Cov 2x2, redchisq."""
    if len(tmids) < 2:
        raise RuntimeError("Not enough mid-times to refit ephemeris.")
    w   = 1.0/np.clip(terrs, 1e-6, np.inf)**2
    X   = np.vstack([epochs, np.ones_like(epochs)]).T
    XT  = (X.T*w)
    beta= np.linalg.inv(XT@X) @ (XT@tmids)
    P, T0 = beta[0], beta[1]
    yhat  = X@beta
    resid = tmids - yhat
    dof   = max(1, len(tmids)-2)
    chi2  = np.sum((resid**2)*w)
    rchi2 = chi2/dof
    Cov   = np.linalg.inv(XT@X) * rchi2
    return P, T0, Cov, rchi2

def _save_json(path, obj):
    with open(path, "w") as f:
        json.dump(obj, f, indent=2)

# ---------- 1) LOAD & FLATTEN (gentle) ----------
print(f"[load] TIC {TIC} — stitching PDCSAP (sectors: {'all' if not SECTORS_HINT else SECTORS_HINT})")
lc = _load_stitched_pdcsap(TIC, sectors_hint=SECTORS_HINT, quality_bits=QUALITY_BITS)
lc_flat = _gentle_flatten(lc, window_days=FLAT_WIN_D, polyorder=POLY_ORDER)

# ---------- 2) TLS in a narrow window around catalog P ----------
pmin = CATALOG_P * (1.0 - WINDOW_FRAC)
pmax = CATALOG_P * (1.0 + WINDOW_FRAC)

time = lc_flat.time.value
flux = lc_flat.flux.value
nthreads = max(1, (os.cpu_count() or 2) - 1)
print(f"[tls] Narrow search around P={CATALOG_P:.6f} d (±{100*WINDOW_FRAC:.1f}%) using {nthreads} threads")

tls = transitleastsquares(time, flux)
tls_res = tls.power(
    period_min=pmin,
    period_max=pmax,
    oversampling_factor=5,
    use_threads=nthreads,
    show_progress_bar=False
)

bestP = float(tls_res.period)
bestT0= float(tls_res.T0)
bestSDE = float(getattr(tls_res, "SDE", getattr(tls_res, "sde", np.nan)))
print(f"[tls] best P={bestP:.6f} d, T0={bestT0:.5f} BTJD, SDE≈{bestSDE:.2f}")

# Save TLS periodogram
fig = plt.figure(figsize=(8.6,5.0))
plt.plot(tls_res.periods, tls_res.power, lw=1)
plt.axvline(bestP, ls="--")
plt.xlabel("Period [days]"); plt.ylabel("TLS power (SDE proxy)")
plt.title(f"TIC {TIC} — TLS around {CATALOG_P:.5f} d (best {bestP:.5f} d)")
plt.tight_layout()
pgram_path = os.path.join(FDIR, f"TIC{TIC}_TLS_narrow_{STAMP}.png")
fig.savefig(pgram_path, dpi=150); plt.close(fig)

# Save a phase-folded figure at TLS best
folded = lc_flat.fold(period=bestP, epoch_time=bestT0)
phase  = folded.phase.value if hasattr(folded.phase, "value") else folded.phase
px, py = _median_bin(phase, folded.flux.value, nbins=180)
fig = plt.figure(figsize=(8.6,5.0))
plt.scatter(phase, folded.flux.value, s=4, alpha=0.15)
plt.plot(px, py, lw=1.8)
dur_phase = (DUR_HOURS/24.0)/bestP
plt.axvspan(-0.5*dur_phase, 0.5*dur_phase, alpha=0.15)
plt.xlabel("Phase [cycles]"); plt.ylabel("Relative flux")
plt.title(f"TIC {TIC} — phase-fold @ P={bestP:.5f} d, T0={bestT0:.5f}")
plt.tight_layout()
fold_path = os.path.join(FDIR, f"TIC{TIC}_TLS_narrow_phase_{STAMP}.png")
fig.savefig(fold_path, dpi=150); plt.close(fig)

# ---------- 3) Refit P, T0 from per-event midtimes with full covariance ----------
print("[fit] estimating per-event mid-times and refitting linear ephemeris …")
epochs, tmids, terrs = _find_midtimes_quadratic(time, flux, bestP, bestT0, DUR_HOURS, k_half=1.7)
if len(tmids) < 2:
    raise RuntimeError(f"Only {len(tmids)} mid-times found; need ≥2 to fit ephemeris.")

P_fit, T0_fit, Cov, rchi2 = _fit_linear_ephemeris(epochs, tmids, terrs)
cov_dict = {"C00_T0": float(Cov[1,1]), "C11_P": float(Cov[0,0]), "C01_T0P": float(Cov[1,0])}

print(f"[fit] P = {P_fit:.8f} d, T0 = {T0_fit:.6f} BTJD, rχ² = {rchi2:.2f}")
print(f"[fit] σ(P) = {math.sqrt(Cov[0,0]):.6e} d, σ(T0) = {math.sqrt(Cov[1,1]):.6e} d, ρ = {Cov[0,1]/math.sqrt(Cov[0,0]*Cov[1,1]):.3f}")

# Save midtimes CSV
mt_csv = os.path.join(RDIR, f"TIC{TIC}_midtimes_{STAMP}.csv")
with open(mt_csv, "w", newline="") as f:
    w = csv.writer(f)
    w.writerow(["epoch", "tmid_btjd", "tmid_err_d"])
    for n, tmid, terr in zip(epochs, tmids, terrs):
        w.writerow([int(n), f"{tmid:.8f}", f"{terr:.8e}"])

# ---------- Save ephemeris JSON ----------
ephem_json = dict(
    tic=TIC,
    tls_narrow=dict(
        catalog_P=CATALOG_P,
        search_window_frac=WINDOW_FRAC,
        best_P=bestP,
        best_T0=bestT0,
        SDE=bestSDE,
        periodogram=pgram_path,
        phase_plot=fold_path,
    ),
    fit=dict(
        P=P_fit,
        T0=T0_fit,
        rchisq=rchi2,
        cov=cov_dict,
        n_midtimes=int(len(tmids)),
    ),
    detrend=dict(kind="sliding_poly", window_days=FLAT_WIN_D, polyorder=POLY_ORDER),
    quality_bits=QUALITY_BITS,
    run_utc=STAMP,
)
ephem_path = os.path.join(RDIR, f"TIC{TIC}_refit_ephemeris_{STAMP}.json")
_save_json(ephem_path, ephem_json)

# ---------- Simple O–C plot ----------
oc = tmids - (T0_fit + epochs * P_fit)
fig = plt.figure(figsize=(8.0, 4.8))
plt.errorbar(epochs, oc * 24 * 60, yerr=terrs * 24 * 60, fmt="o", ms=4, capsize=2)
plt.axhline(0, color="k", lw=1, ls="--")
plt.xlabel("Epoch (n)"); plt.ylabel("O–C [minutes]")
plt.title(f"TIC {TIC} — O–C after refit (P={P_fit:.6f} d)")
plt.tight_layout()
oc_path = os.path.join(FDIR, f"TIC{TIC}_OC_{STAMP}.png")
fig.savefig(oc_path, dpi=150); plt.close(fig)

print("\n[done]")
print(f"  TLS periodogram : {pgram_path}")
print(f"  Phase-fold plot : {fold_path}")
print(f"  Midtimes CSV    : {mt_csv}")
print(f"  Ephemeris JSON  : {ephem_path}")
print(f"  O–C plot        : {oc_path}")

[load] TIC 37749396 — stitching PDCSAP (sectors: all)
[tls] Narrow search around P=13.475725 d (±1.0%) using 7 threads
Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 29 durations
Searching 125002 data points, 1027 periods from 13.341 to 13.61 days
Using 7 of 8 CPU threads
Searching for best T0 for period 13.47214 days
[tls] best P=13.472137 d, T0=1392.79917 BTJD, SDE≈8.15
[fit] estimating per-event mid-times and refitting linear ephemeris …
[fit] P = 13.47232744 d, T0 = 1392.785303 BTJD, rχ² = 0.04
[fit] σ(P) = 1.839523e-05 d, σ(T0) = 1.462941e-03 d, ρ = -0.986

[done]
  TLS periodogram : figures/TIC37749396/TIC37749396_TLS_narrow_20250927T213104Z.png
  Phase-fold plot : figures/TIC37749396/TIC37749396_TLS_narrow_phase_20250927T213104Z.png
  Midtimes CSV    : results/TIC37749396/TIC37749396_midtimes_20250927T213104Z.csv
  Ephemeris JSON  : results/TIC37749396/TIC37749396_refit_ephemeris_20250927T213104Z.json
  O–C plot        : figures/TIC37749396/TIC37749396_OC_2

In [40]:
# === Target C (TOI 550.02 / TIC 311183180) • Combine sectors • TLS near catalog P • Refit (P, T0) with covariance ===
# Minimal-verbosity; saves artifacts under results/TIC{TIC}/ and figures/TIC{TIC}/

import os, json, csv, warnings, math, datetime as dt
import numpy as np
import matplotlib.pyplot as plt
from lightkurve import search_lightcurvefile, LightCurveCollection
from transitleastsquares import transitleastsquares

warnings.filterwarnings("ignore", category=UserWarning)
plt.rcParams.update({"figure.dpi": 120})

# ---------- USER KNOBS ----------
TIC          = 311183180                 # Target C
SECTORS_HINT = None                      # e.g., [5, 31]; set None to auto-use all available
CATALOG_P    = 9.348442                  # narrow-center period (days); from your stitched TLS top-1
WINDOW_FRAC  = 0.01                      # ±1% search window
DUR_HOURS    = 2.8                       # rough duration for masks & display (hours)
QUALITY_BITS = 175
FLAT_WIN_D   = 1.0
POLY_ORDER   = 2

# ---------- PATHS ----------
STAMP   = dt.datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
RDIR    = f"results/TIC{TIC}"
FDIR    = f"figures/TIC{TIC}"
os.makedirs(RDIR, exist_ok=True)
os.makedirs(FDIR, exist_ok=True)

# ---------- HELPERS ----------
def _median_bin(x, y, nbins=180):
    x = np.asarray(x, float); y = np.asarray(y, float)
    lo, hi = np.nanmin(x), np.nanmax(x)
    edges = np.linspace(lo, hi, nbins+1)
    idx = np.digitize(x, edges) - 1
    px = 0.5*(edges[:-1] + edges[1:])
    py = np.full(nbins, np.nan)
    for i in range(nbins):
        m = y[idx == i]
        if m.size:
            py[i] = np.nanmedian(m)
    return px, py

def _gentle_flatten(lc, window_days=1.0, polyorder=2):
    try:
        flat = lc.flatten(
            window_length=int(max(7, round(window_days/(np.nanmedian(np.diff(lc.time.value)) or 1e-3)))),
            polyorder=polyorder,
            return_trend=False
        )
    except TypeError:
        flat = lc.flatten(
            window_length=int(max(7, round(window_days/(np.nanmedian(np.diff(lc.time.value)) or 1e-3)))),
            polyorder=polyorder
        )
    return flat.remove_nans().normalize()

def _load_stitched_pdcsap(tic, sectors_hint=None, quality_bits=175):
    sr = search_lightcurvefile(f"TIC {tic}", mission="TESS", author="SPOC")
    if len(sr) == 0:
        raise RuntimeError("No SPOC LightCurveFiles found.")
    files = sr.download_all()

    lcs = []
    for f in files:
        if sectors_hint is not None:
            sec = getattr(f, "sector", None)
            if (sec is None) or (sec not in sectors_hint):
                continue

        lc = f.PDCSAP_FLUX

        if hasattr(lc, "remove_nans"):
            lc = lc.remove_nans()
        if hasattr(lc, "remove_outliers"):
            try:
                lc = lc.remove_outliers(sigma=10)
            except Exception:
                pass

        if hasattr(lc, "quality"):
            good = (lc.quality & quality_bits) == 0
            lc = lc[good]

        if len(lc.time) == 0:
            continue

        lcs.append(lc.normalize())

    if not lcs:
        raise RuntimeError("No usable PDCSAP cadences after quality mask.")
    return LightCurveCollection(lcs).stitch().remove_nans().normalize()

def _find_midtimes_quadratic(time_btjd, flux_norm, period, t0, dur_hours, k_half=1.0):
    tarr = np.asarray(time_btjd, float)
    farr = np.asarray(flux_norm, float)
    dur_days = dur_hours/24.0

    kmin = math.floor((tarr.min() - t0)/period) - 1
    kmax = math.ceil((tarr.max() - t0)/period) + 1
    epochs, tmids, terrs = [], [], []

    for k in range(kmin, kmax+1):
        tc = t0 + k*period
        w  = (np.abs(tarr - tc) <= 0.5*k_half*dur_days)
        if np.count_nonzero(w) < 8:
            continue
        t  = tarr[w]
        f  = farr[w]

        A = np.vstack([np.ones_like(t), t - np.nanmedian(t)]).T
        coeff, *_ = np.linalg.lstsq(A, f, rcond=None)
        f_det = f - (A @ coeff)

        q = np.nanpercentile(f_det, 30)
        sel = f_det <= q
        if np.count_nonzero(sel) < 6:
            continue
        tt = t[sel] - np.nanmedian(t[sel])
        ff = f_det[sel]

        X = np.vstack([tt**2, tt, np.ones_like(tt)]).T
        p, *_ = np.linalg.lstsq(X, ff, rcond=None)
        a, b, c = p
        if a == 0:
            continue
        tmid = -b/(2*a) + np.nanmedian(t[sel])

        model = a*tt**2 + b*tt + c
        res   = ff - model
        if len(res) < 6:
            continue
        draws = []
        rng = np.random.default_rng(42)
        for _ in range(200):
            res_s = rng.choice(res, size=res.size, replace=True)
            ff_s  = model + res_s
            p_s, *_ = np.linalg.lstsq(X, ff_s, rcond=None)
            a_s, b_s, c_s = p_s
            if a_s == 0:
                continue
            draws.append(-b_s/(2*a_s) + np.nanmedian(t[sel]))
        if len(draws) < 10:
            continue
        epochs.append(k)
        tmids.append(tmid)
        terrs.append(np.nanstd(draws, ddof=1))
    return np.asarray(epochs), np.asarray(tmids), np.asarray(terrs)

def _fit_linear_ephemeris(epochs, tmids, terrs):
    if len(tmids) < 2:
        raise RuntimeError("Not enough mid-times to refit ephemeris.")
    w   = 1.0/np.clip(terrs, 1e-6, np.inf)**2
    X   = np.vstack([epochs, np.ones_like(epochs)]).T
    XT  = (X.T*w)
    beta= np.linalg.inv(XT@X) @ (XT@tmids)
    P, T0 = beta[0], beta[1]
    yhat  = X@beta
    resid = tmids - yhat
    dof   = max(1, len(tmids)-2)
    chi2  = np.sum((resid**2)*w)
    rchi2 = chi2/dof
    Cov   = np.linalg.inv(XT@X) * rchi2
    return P, T0, Cov, rchi2

def _save_json(path, obj):
    with open(path, "w") as f:
        json.dump(obj, f, indent=2)

# ---------- 1) LOAD & FLATTEN ----------
print(f"[load] TIC {TIC} — stitching PDCSAP (sectors: {'all' if not SECTORS_HINT else SECTORS_HINT})")
lc = _load_stitched_pdcsap(TIC, sectors_hint=SECTORS_HINT, quality_bits=QUALITY_BITS)
lc_flat = _gentle_flatten(lc, window_days=FLAT_WIN_D, polyorder=POLY_ORDER)

# ---------- 2) TLS narrow around catalog P ----------
pmin = CATALOG_P * (1.0 - WINDOW_FRAC)
pmax = CATALOG_P * (1.0 + WINDOW_FRAC)

time = lc_flat.time.value
flux = lc_flat.flux.value
nthreads = max(1, (os.cpu_count() or 2) - 1)
print(f"[tls] Narrow search around P={CATALOG_P:.6f} d (±{100*WINDOW_FRAC:.1f}%) using {nthreads} threads")

tls = transitleastsquares(time, flux)
tls_res = tls.power(
    period_min=pmin,
    period_max=pmax,
    oversampling_factor=5,
    use_threads=nthreads,
    show_progress_bar=False
)

bestP = float(tls_res.period)
bestT0= float(tls_res.T0)
bestSDE = float(getattr(tls_res, "SDE", getattr(tls_res, "sde", np.nan)))
print(f"[tls] best P={bestP:.6f} d, T0={bestT0:.5f} BTJD, SDE≈{bestSDE:.2f}")

# Save TLS periodogram
fig = plt.figure(figsize=(8.6,5.0))
plt.plot(tls_res.periods, tls_res.power, lw=1)
plt.axvline(bestP, ls="--")
plt.xlabel("Period [days]"); plt.ylabel("TLS power (SDE proxy)")
plt.title(f"TIC {TIC} — TLS around {CATALOG_P:.5f} d (best {bestP:.5f} d)")
plt.tight_layout()
pgram_path = os.path.join(FDIR, f"TIC{TIC}_TLS_narrow_{STAMP}.png")
fig.savefig(pgram_path, dpi=150); plt.close(fig)

# Save a phase-folded figure at TLS best
folded = lc_flat.fold(period=bestP, epoch_time=bestT0)
phase  = folded.phase.value if hasattr(folded.phase, "value") else folded.phase
px, py = _median_bin(phase, folded.flux.value, nbins=180)
fig = plt.figure(figsize=(8.6,5.0))
plt.scatter(phase, folded.flux.value, s=4, alpha=0.15)
plt.plot(px, py, lw=1.8)
dur_phase = (DUR_HOURS/24.0)/bestP
plt.axvspan(-0.5*dur_phase, 0.5*dur_phase, alpha=0.15)
plt.xlabel("Phase [cycles]"); plt.ylabel("Relative flux")
plt.title(f"TIC {TIC} — phase-fold @ P={bestP:.5f} d, T0={bestT0:.5f}")
plt.tight_layout()
fold_path = os.path.join(FDIR, f"TIC{TIC}_TLS_narrow_phase_{STAMP}.png")
fig.savefig(fold_path, dpi=150); plt.close(fig)

# ---------- 3) Refit P, T0 with covariance ----------
print("[fit] estimating per-event mid-times and refitting linear ephemeris …")
epochs, tmids, terrs = _find_midtimes_quadratic(time, flux, bestP, bestT0, DUR_HOURS, k_half=1.7)
if len(tmids) < 2:
    raise RuntimeError(f"Only {len(tmids)} mid-times found; need ≥2 to fit ephemeris.")

P_fit, T0_fit, Cov, rchi2 = _fit_linear_ephemeris(epochs, tmids, terrs)
cov_dict = {"C00_T0": float(Cov[1,1]), "C11_P": float(Cov[0,0]), "C01_T0P": float(Cov[1,0])}

print(f"[fit] P = {P_fit:.8f} d, T0 = {T0_fit:.6f} BTJD, rχ² = {rchi2:.2f}")
print(f"[fit] σ(P) = {math.sqrt(Cov[0,0]):.6e} d, σ(T0) = {math.sqrt(Cov[1,1]):.6e} d, ρ = {Cov[0,1]/math.sqrt(Cov[0,0]*Cov[1,1]):.3f}")

# Save midtimes CSV
mt_csv = os.path.join(RDIR, f"TIC{TIC}_midtimes_{STAMP}.csv")
with open(mt_csv, "w", newline="") as f:
    w = csv.writer(f)
    w.writerow(["epoch", "tmid_btjd", "tmid_err_d"])
    for n, tmid, terr in zip(epochs, tmids, terrs):
        w.writerow([int(n), f"{tmid:.8f}", f"{terr:.8e}"])

# ---------- Save ephemeris JSON ----------
ephem_json = dict(
    tic=TIC,
    tls_narrow=dict(
        catalog_P=CATALOG_P,
        search_window_frac=WINDOW_FRAC,
        best_P=bestP,
        best_T0=bestT0,
        SDE=bestSDE,
        periodogram=pgram_path,
        phase_plot=fold_path,
    ),
    fit=dict(
        P=P_fit,
        T0=T0_fit,
        rchisq=rchi2,
        cov=cov_dict,
        n_midtimes=int(len(tmids)),
    ),
    detrend=dict(kind="sliding_poly", window_days=FLAT_WIN_D, polyorder=POLY_ORDER),
    quality_bits=QUALITY_BITS,
    run_utc=STAMP,
)
ephem_path = os.path.join(RDIR, f"TIC{TIC}_refit_ephemeris_{STAMP}.json")
_save_json(ephem_path, ephem_json)

# ---------- Simple O–C plot ----------
oc = tmids - (T0_fit + epochs * P_fit)
fig = plt.figure(figsize=(8.0, 4.8))
plt.errorbar(epochs, oc * 24 * 60, yerr=terrs * 24 * 60, fmt="o", ms=4, capsize=2)
plt.axhline(0, color="k", lw=1, ls="--")
plt.xlabel("Epoch (n)"); plt.ylabel("O–C [minutes]")
plt.title(f"TIC {TIC} — O–C after refit (P={P_fit:.6f} d)")
plt.tight_layout()
oc_path = os.path.join(FDIR, f"TIC{TIC}_OC_{STAMP}.png")
fig.savefig(oc_path, dpi=150); plt.close(fig)

print("\n[done]")
print(f"  TLS periodogram : {pgram_path}")
print(f"  Phase-fold plot : {fold_path}")
print(f"  Midtimes CSV    : {mt_csv}")
print(f"  Ephemeris JSON  : {ephem_path}")
print(f"  O–C plot        : {oc_path}")

[load] TIC 311183180 — stitching PDCSAP (sectors: all)
[tls] Narrow search around P=9.348442 d (±1.0%) using 7 threads
Transit Least Squares TLS 1.32 (5 Apr 2024)
Creating model cache for 29 durations
Searching 33536 data points, 459 periods from 9.255 to 9.442 days
Using 7 of 8 CPU threads
Searching for best T0 for period 9.34288 days
[tls] best P=9.342883 d, T0=1439.58140 BTJD, SDE≈5.97
[fit] estimating per-event mid-times and refitting linear ephemeris …
[fit] P = 9.34282090 d, T0 = 1439.584770 BTJD, rχ² = 0.01
[fit] σ(P) = 3.254905e-07 d, σ(T0) = 2.089374e-05 d, ρ = -0.831

[done]
  TLS periodogram : figures/TIC311183180/TIC311183180_TLS_narrow_20250927T214103Z.png
  Phase-fold plot : figures/TIC311183180/TIC311183180_TLS_narrow_phase_20250927T214103Z.png
  Midtimes CSV    : results/TIC311183180/TIC311183180_midtimes_20250927T214103Z.csv
  Ephemeris JSON  : results/TIC311183180/TIC311183180_refit_ephemeris_20250927T214103Z.json
  O–C plot        : figures/TIC311183180/TIC311183180_

In [1]:
# %% [markdown]
# # Quick Scan — Target D (BLS wide → TLS narrow + fast vetting)
# Edits: set TARGET_TIC (and optional TARGET_NAME). Optional: SECTORS to force a subset.

# %%
import os, time, warnings, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# ---- USER INPUT --------------------------------------------------------------
TARGET_TIC   = 123456789          # <-- EDIT: TIC ID for Target D
TARGET_NAME  = "Target D"         # <-- optional nickname for titles
SECTORS      = None               # e.g., [14, 15] to force; or None to auto-discover

# ---- RUNTIME SETTINGS (keep as-is) ------------------------------------------
QUALITY_BITS = 175
WINDOW_DAYS  = 0.75   # gentle high-pass
MAD_SIGMA    = 5.0
BLS_PMIN, BLS_PMAX = 0.5, 50.0
BLS_NPER     = 5000
DUR_GRID_HR  = np.linspace(0.5, 3.0, 18)  # BLS duration grid
TLS_NARROW_FRAC = 0.01   # ±1% around each BLS peak
TLS_MIN_TRANSITS_SECTOR = 3
TLS_MIN_TRANSITS_STITCH = 2

os.makedirs("figures", exist_ok=True)
os.makedirs("results", exist_ok=True)

# ---- Imports that may be optional -------------------------------------------
import lightkurve as lk
from astropy.timeseries import BoxLeastSquares
try:
    from transitleastsquares import transitleastsquares as TLS
    HAVE_TLS = True
except Exception:
    HAVE_TLS = False

# ---- Helpers ----------------------------------------------------------------
def _mad(x): 
    m = np.nanmedian(x); return np.nanmedian(np.abs(x - m)) * 1.4826

def detrend_gentle(lc):
    """quality mask, sigma-clip, running-median high-pass, normalize."""
    lc2 = lc.remove_outliers(sigma=20)  # remove NaNs/inf
    lc2 = lc2[np.bitwise_and(lc2.quality, 0) == 0] if hasattr(lc2, "quality") else lc2
    lc2 = lc2.remove_nans()
    # sigma-clip on flux
    f = lc2.flux.value.copy()
    s = _mad(f)
    if np.isfinite(s) and s > 0:
        m = np.nanmedian(f)
        ok = (f > m - MAD_SIGMA*s) & (f < m + MAD_SIGMA*s)
        lc2 = lc2[ok]
    # high-pass via running median
    dt = np.nanmedian(np.diff(lc2.time.value))
    win = max(3, int(round((WINDOW_DAYS / dt))))
    trend = pd.Series(lc2.flux.value).rolling(win, center=True, min_periods=max(3,win//3)).median()
    trend = np.interp(np.arange(len(trend)), np.flatnonzero(np.isfinite(trend)), trend[np.isfinite(trend)])
    flat = lc2.copy()
    flat.flux = (lc2.flux.value / trend)
    return flat.normalize().remove_nans()

def load_pdcsap_sector(tic, sector=None, tries=3, sleep=2):
    """Prefer SPOC PDCSAP; robust to MAST flaps."""
    for k in range(tries):
        try:
            sr = lk.search_lightcurvefile(f"TIC {tic}", mission="TESS", author="SPOC", sector=sector)
            if len(sr) == 0:
                return None
            lcf = sr.download()
            if lcf is None:
                return None
            lc = lcf.PDCSAP_FLUX
            # Apply official LIGHTKURVE quality mask if present
            lc = lc.remove_outliers(sigma=20)
            if hasattr(lc, "remove_quality"):
                lc = lc.remove_quality(QUALITY_BITS)
            return lc.remove_nans()
        except Exception as e:
            if k == tries-1:
                print(f"[load] sector {sector} failed: {e}")
                return None
            time.sleep(sleep*(2**k))

def discover_sectors(tic):
    try:
        sr = lk.search_lightcurvefile(f"TIC {tic}", mission="TESS", author="SPOC")
        sectors = sorted(list({r.mission_sectors[0] for r in sr if r.mission_sectors}))
        return sectors
    except Exception:
        return []

def bls_scan(time, flux, dur_grid_hr=DUR_GRID_HR, pmin=BLS_PMIN, pmax=BLS_PMAX, nper=BLS_NPER):
    y = flux - np.nanmedian(flux)
    bls = BoxLeastSquares(time, y)
    durations = dur_grid_hr/24.0
    periods = np.geomspace(pmin, min(pmax, (time.max()-time.min())*0.95), nper)
    res = bls.power(periods, durations, method='fast', normalization='standard')
    return res, periods, durations

def tls_scan_narrow(time, flux, per_center, frac=TLS_NARROW_FRAC, min_transits=2):
    """TLS around per_center ± frac; returns dict or None."""
    if not HAVE_TLS:
        return None
    y = flux/np.nanmedian(flux)
    mask = np.isfinite(time) & np.isfinite(y)
    time, y = time[mask], y[mask]
    Pmin = per_center*(1.0 - frac)
    Pmax = per_center*(1.0 + frac)
    try:
        model = TLS(time, y)
        out = model.power(period_min=Pmin, period_max=Pmax, show_progress_bar=False, n_transits_min=min_transits)
        return {
            "period": float(out.period),
            "t0": float(out.T0),
            "sde": float(out.SDE),
            "duration_d": float(out.duration),
            "depth": float(out.depth)
        }
    except Exception:
        return None

def fold_plot(time, flux, period, t0=None, title="", outpath=None):
    if t0 is None:
        t0 = time[0]
    phase = ((time - t0 + 0.5*period) % period)/period - 0.5
    order = np.argsort(phase); phase, f = phase[order], flux[order]
    # mean-binned curve for visibility
    nb = 100
    bins = np.linspace(-0.5, 0.5, nb+1)
    idx = np.digitize(phase, bins)-1
    bmean = np.array([np.nanmean(f[idx==i]) for i in range(nb)])
    bcent = 0.5*(bins[:-1]+bins[1:])
    plt.figure(figsize=(7,3.2))
    plt.scatter(phase, f, s=2, alpha=0.15, rasterized=True)
    plt.plot(bcent, bmean, lw=2)
    plt.title(title)
    plt.xlabel("Phase (P)")
    plt.ylabel("Normalized flux")
    plt.tight_layout()
    if outpath:
        plt.savefig(outpath, dpi=180)
        plt.close()
    else:
        plt.show()

def odd_even_secondary_checks(time, flux, period, t0, dur_hr, tag, outdir="figures"):
    """Quick QC panels & CSV row for odd/even + secondary@0.5P."""
    phase = ((time - t0 + 0.5*period) % period)/period - 0.5
    dur = dur_hr/24.0
    inodd  = (np.abs(phase) < 0.5*dur) & (np.floor(((time-t0)/period) % 2)==0)
    ineven = (np.abs(phase) < 0.5*dur) & (np.floor(((time-t0)/period) % 2)==1)
    insec  = (np.abs(phase-0.5) < 0.5*dur)
    oot    = (np.abs(phase) > 1.5*dur) & (np.abs(phase-0.5) > 1.5*dur)

    def depth_ppm(mask):
        if mask.sum()<5 or oot.sum()<20: return np.nan
        return (np.nanmedian(flux[oot]) - np.nanmedian(flux[mask]))*1e6

    d_odd  = depth_ppm(inodd)
    d_even = depth_ppm(ineven)
    d_sec  = depth_ppm(insec)

    # quick figure
    plt.figure(figsize=(7.2,3.2))
    plt.scatter(phase, flux, s=2, alpha=0.15, label="all", rasterized=True)
    plt.scatter(phase[inodd], flux[inodd], s=5, alpha=0.6, label="odd")
    plt.scatter(phase[ineven], flux[ineven], s=5, alpha=0.6, label="even")
    plt.scatter(phase[insec], flux[insec], s=5, alpha=0.6, label="secondary (0.5P)")
    plt.legend(frameon=False, ncol=3, fontsize=8)
    plt.title(f"{TARGET_NAME} TIC {TARGET_TIC} — Odd/Even/Secondary @ P={period:.5f} d")
    plt.xlabel("Phase"); plt.ylabel("Normalized flux"); plt.tight_layout()
    outpng = f"{outdir}/TIC{TARGET_TIC}_{tag}_odd_even_secondary.png"
    plt.savefig(outpng, dpi=180); plt.close()
    return {"odd_ppm": d_odd, "even_ppm": d_even, "secondary_ppm": d_sec, "png": outpng}

# ---- Load sectors ------------------------------------------------------------
if SECTORS is None:
    SECTORS = discover_sectors(TARGET_TIC)
print(f"[{TARGET_NAME}] TIC {TARGET_TIC} — sectors: {SECTORS if SECTORS else 'None found'}")

per_sector = []
for s in (SECTORS or []):
    lc = load_pdcsap_sector(TARGET_TIC, sector=s)
    if lc is None or len(lc.flux.value)==0:
        print(f"  sector {s}: no PDCSAP")
        continue
    flat = detrend_gentle(lc)
    per_sector.append((s, flat))

if not per_sector:
    raise SystemExit("No usable PDCSAP sectors found. Provide SECTORS or check network.")

# ---- Per-sector scans --------------------------------------------------------
stitched = None
toprows = []
for s, flat in per_sector:
    print(f"[scan] Sector {s}: N={len(flat.time)}")
    res, periods, durations = bls_scan(flat.time.value, flat.flux.value)
    # top-3 by power
    order = np.argsort(res.power)[::-1]
    top_idx = order[:3]
    tops = []
    for i in top_idx:
        tops.append({"period": float(res.period[i]), "power": float(res.power[i]),
                     "duration_d": float(res.duration[i])})
    # Save BLS periodogram
    plt.figure(figsize=(7,3.0))
    plt.plot(res.period, res.power, lw=1)
    plt.xlabel("Period (days)"); plt.ylabel("BLS power")
    plt.title(f"{TARGET_NAME} TIC {TARGET_TIC} — Sector {s} BLS")
    plt.tight_layout()
    bls_png = f"figures/TIC{TARGET_TIC}_S{s}_BLS_periodogram.png"
    plt.savefig(bls_png, dpi=180); plt.close()

    # TLS narrow around each top BLS peak
    tls_rows = []
    for j, trow in enumerate(tops):
        tls = tls_scan_narrow(flat.time.value, flat.flux.value, trow["period"],
                              frac=TLS_NARROW_FRAC, min_transits=TLS_MIN_TRANSITS_SECTOR)
        if tls is not None:
            # quick fold plot
            fold_plot(flat.time.value, flat.flux.value, tls["period"], tls["t0"],
                      title=f"{TARGET_NAME} TIC {TARGET_TIC} — S{s} TLS fold @ {tls['period']:.5f} d",
                      outpath=f"figures/TIC{TARGET_TIC}_S{s}_TLS_fold_P{tls['period']:.5f}.png")
            tls_rows.append(tls)

    # write CSVs
    bls_df = pd.DataFrame(tops)
    bls_df.to_csv(f"results/TIC{TARGET_TIC}_S{s}_BLS_top3.csv", index=False)
    if tls_rows:
        pd.DataFrame(tls_rows).to_csv(f"results/TIC{TARGET_TIC}_S{s}_TLS_top3.csv", index=False)

    stitched = flat if stitched is None else stitched.append(flat)

# ---- Stitched scan -----------------------------------------------------------
print(f"[scan] Stitched: N={len(stitched.time)}")
res, periods, durations = bls_scan(stitched.time.value, stitched.flux.value)

# top-3 stitched
order = np.argsort(res.power)[::-1]
top_idx = order[:3]
tops_st = [{"period": float(res.period[i]), "power": float(res.power[i]),
            "duration_d": float(res.duration[i])} for i in top_idx]

plt.figure(figsize=(7.5,3.0))
plt.plot(res.period, res.power, lw=1)
plt.xlabel("Period (days)"); plt.ylabel("BLS power")
plt.title(f"{TARGET_NAME} TIC {TARGET_TIC} — Stitched BLS")
plt.tight_layout()
plt.savefig(f"figures/TIC{TARGET_TIC}_stitched_BLS_periodogram.png", dpi=180); plt.close()

tls_rows_st = []
best_fold_path = None
best_tls = None
for trow in tops_st:
    tls = tls_scan_narrow(stitched.time.value, stitched.flux.value, trow["period"],
                          frac=TLS_NARROW_FRAC, min_transits=TLS_MIN_TRANSITS_STITCH)
    if tls is None: 
        continue
    outpng = f"figures/TIC{TARGET_TIC}_stitched_TLS_fold_P{tls['period']:.5f}.png"
    fold_plot(stitched.time.value, stitched.flux.value, tls["period"], tls["t0"],
              title=f"{TARGET_NAME} TIC {TARGET_TIC} — Stitched TLS fold @ {tls['period']:.5f} d",
              outpath=outpng)
    tls_rows_st.append(tls)
    if best_tls is None or tls["sde"] > best_tls["sde"]:
        best_tls, best_fold_path = tls, outpng

pd.DataFrame(tops_st).to_csv(f"results/TIC{TARGET_TIC}_stitched_BLS_top3.csv", index=False)
if tls_rows_st:
    pd.DataFrame(tls_rows_st).to_csv(f"results/TIC{TARGET_TIC}_stitched_TLS_top3.csv", index=False)

# ---- Fast odd/even + secondary on best stitched TLS -------------------------
qc_row = {}
if best_tls is not None:
    est_dur_hr = max(0.5, min(3.0, best_tls["duration_d"]*24.0))
    qc_row = odd_even_secondary_checks(stitched.time.value, stitched.flux.value,
                                       best_tls["period"], best_tls["t0"], est_dur_hr,
                                       tag="stitched")
    qc_row.update({"period_best": best_tls["period"], "t0_best": best_tls["t0"],
                   "sde_best": best_tls["sde"], "duration_d_best": best_tls["duration_d"],
                   "fold_png": best_fold_path})
    pd.DataFrame([qc_row]).to_csv(f"results/TIC{TARGET_TIC}_stitched_quick_qc.csv", index=False)

# ---- Console summary ---------------------------------------------------------
print("\n=== Quick Scan Summary ===")
print(f"TIC {TARGET_TIC}  ({TARGET_NAME})")
print(f" Sectors used: {[s for s,_ in per_sector]}")
if tls_rows_st:
    print(" Stitched TLS top (by SDE):")
    print(pd.DataFrame(tls_rows_st).sort_values("sde", ascending=False).head(3))
else:
    print(" TLS unavailable or no stitched TLS peaks found.")
if qc_row:
    print("\n Quick QC @ best stitched period:")
    print(qc_row)
print("\nSaved CSVs in results/ and plots in figures/. Done.")

/var/folders/0v/nmv_q34n0hz650dnlq12lhl1jf_dp7/T/ipykernel_79405/1512842400.py:91: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
  sr = lk.search_lightcurvefile(f"TIC {tic}", mission="TESS", author="SPOC")
No data found for target "TIC 123456789".


[Target D] TIC 123456789 — sectors: None found


SystemExit: No usable PDCSAP sectors found. Provide SECTORS or check network.

In [7]:
# %% Offline Target D picker (no network): use local tables & stored sector lists
import os, ast, pandas as pd

# Skip these (A–C)
USED_TICS = {119584412, 37749396, 311183180}

# Order of preference for local tables
LOCAL_TABLES = [
    "results/targets_with_sectors.csv",  # best: has sector lists
    "results/priority_targets.csv",
    "results/targets_ranked.csv",
    "results/targets.csv",
]

def _first_existing(paths):
    for p in paths:
        if os.path.exists(p):
            df = pd.read_csv(p)
            if len(df):
                return p, df
    return None, None

src, df = _first_existing(LOCAL_TABLES)
if df is None:
    raise SystemExit("No local target tables found. Expected one of:\n  - " + "\n  - ".join(LOCAL_TABLES))

# Normalize columns
lower = {c.lower(): c for c in df.columns}
tic_col   = lower.get("tic") or lower.get("tic_id") or lower.get("ticid") or lower.get("tic_id_norm")
name_col  = lower.get("name") or lower.get("toi") or lower.get("target") or lower.get("designation")
sectors_c = None
for k in ("sectors_now","sectors","observed_sectors","sector_list"):
    if k in lower:
        sectors_c = lower[k]; break
if tic_col is None:
    raise SystemExit(f"No TIC-like column in {src}. Columns: {list(df.columns)}")

# Build pool: drop A–C, dedupe, keep valid TIC
df["__tic"] = pd.to_numeric(df[tic_col], errors="coerce").astype("Int64")
pool = df.dropna(subset=["__tic"]).drop_duplicates(subset="__tic", keep="first")
pool = pool[~pool["__tic"].isin(USED_TICS)].copy()

def _parse_sectors(x):
    # Accept "[14, 15]" or "14,15" or list; return list[int]
    if pd.isna(x): return []
    if isinstance(x, (list,tuple)): return [int(s) for s in x if str(s).strip()!=""]
    s = str(x).strip()
    try:
        # Try literal (e.g., "[14, 15]")
        val = ast.literal_eval(s)
        if isinstance(val, (list,tuple)):
            return [int(v) for v in val if str(v).strip()!=""]
    except Exception:
        pass
    # Fallback: split by comma
    return [int(t) for t in s.strip("[]").split(",") if t.strip().isdigit()]

if sectors_c is not None:
    pool["__sectors"] = pool[sectors_c].map(_parse_sectors)
    pool["__nsec"]    = pool["__sectors"].map(len)
    # Prefer more sectors, then keep current row order as tiebreak
    pool = pool.sort_values(["__nsec"], ascending=False).reset_index(drop=True)
else:
    pool["__sectors"] = [[] for _ in range(len(pool))]
    pool["__nsec"]    = 0

if pool.empty:
    raise SystemExit("After skipping A–C, no candidates remain in local table(s).")

# Pick first with ≥2 sectors if available; else first with ≥1; else first row
picked = None
for thresh in (2, 1, 0):
    sub = pool[pool["__nsec"] >= thresh]
    if len(sub):
        picked = sub.iloc[0]
        break

TARGET_NAME = "Target D"
TARGET_TIC  = int(picked["__tic"])
SECTORS     = list(picked["__sectors"]) if isinstance(picked["__sectors"], list) else []
CAND_NAME   = str(picked.get(name_col, f"TIC {TARGET_TIC}")) if name_col else f"TIC {TARGET_TIC}"

print("Chosen Target D (offline selection)")
print("-----------------------------------")
print(f"Source table : {src}")
print(f"Name         : {CAND_NAME}")
print(f"TIC          : {TARGET_TIC}")
print(f"Sectors(list): {SECTORS if SECTORS else '— (not in table)'}")
print(f"N sectors    : {len(SECTORS)}")
print("\nNext steps:")
if len(SECTORS) >= 1:
    print(" - Run your PDCSAP-first loader with SECTORS as hints; if a sector lacks PDCSAP, fall back to TESSCut.")
else:
    print(" - Run your loader in auto-discovery mode (SPOC first, else TESSCut) since the table lacks sector info.")

Chosen Target D (offline selection)
-----------------------------------
Source table : results/targets_with_sectors.csv
Name         : 1487.01
TIC          : 459978312
Sectors(list): — (not in table)
N sectors    : 0

Next steps:
 - Run your loader in auto-discovery mode (SPOC first, else TESSCut) since the table lacks sector info.


In [8]:
# %% Re-pick Target D, scanning ANY 'sector*' column in the local table (offline)
import os, ast, pandas as pd

USED_TICS = {119584412, 37749396, 311183180}
src = "results/targets_with_sectors.csv"
if not os.path.exists(src):
    raise SystemExit(f"Missing {src}")

df = pd.read_csv(src)
lower = {c.lower(): c for c in df.columns}
tic_col  = lower.get("tic") or lower.get("tic_id") or lower.get("ticid") or lower.get("tic_id_norm")
name_col = lower.get("name") or lower.get("toi") or lower.get("target") or lower.get("designation")
if tic_col is None:
    raise SystemExit(f"No TIC-like column in {src}. Columns: {list(df.columns)}")

# Build pool (skip A–C)
df["__tic"] = pd.to_numeric(df[tic_col], errors="coerce").astype("Int64")
pool = (df.dropna(subset=["__tic"])
          .drop_duplicates(subset="__tic", keep="first")
          [~df["__tic"].isin(USED_TICS)].copy())

# Find ANY sectorish column
sectorish_cols = [c for c in df.columns if "sector" in c.lower()]
def _parse_sectors(x):
    if pd.isna(x): return []
    if isinstance(x, (list, tuple)): return [int(s) for s in x if str(s).strip()!=""]
    s = str(x).strip()
    try:
        val = ast.literal_eval(s)
        if isinstance(val, (list,tuple)):
            return [int(v) for v in val if str(v).strip()!=""]
    except Exception:
        pass
    return [int(t) for t in s.strip("[]").split(",") if t.strip().isdigit()]

pool["__sectors"] = [[] for _ in range(len(pool))]
for c in sectorish_cols:
    # prefer the first sectorish column that yields non-empty lists
    parsed = pool[c].map(_parse_sectors)
    if parsed.map(len).sum() > pool["__sectors"].map(len).sum():
        pool["__sectors"] = parsed

pool["__nsec"] = pool["__sectors"].map(len)
# Prefer lots of sectors; tie-break by table order
pool = pool.sort_values("__nsec", ascending=False).reset_index(drop=True)

# Pick with threshold 2→1→0
picked = None
for t in (2,1,0):
    sub = pool[pool["__nsec"]>=t]
    if len(sub):
        picked = sub.iloc[0]; break

TARGET_TIC  = int(picked["__tic"])
TARGET_NAME = "Target D"
CAND_NAME   = str(picked.get(name_col, f"TIC {TARGET_TIC}")) if name_col else f"TIC {TARGET_TIC}"
SECTORS     = list(picked["__sectors"]) if isinstance(picked["__sectors"], list) else []

print("Chosen Target D (offline, sector-aware)")
print("---------------------------------------")
print(f"Source table : {src}")
print(f"Name         : {CAND_NAME}")
print(f"TIC          : {TARGET_TIC}")
print(f"Sectors(list): {SECTORS if SECTORS else '— (none found in table)'}")
print(f"N sectors    : {len(SECTORS)}")
print("\nNext steps:")
print(" - Use your PDCSAP-first loader on these sectors (or auto-discover if empty).")

Chosen Target D (offline, sector-aware)
---------------------------------------
Source table : results/targets_with_sectors.csv
Name         : 1487.01
TIC          : 459978312
Sectors(list): — (none found in table)
N sectors    : 0

Next steps:
 - Use your PDCSAP-first loader on these sectors (or auto-discover if empty).


In [15]:
# %% Auto-pick & quick-scan Target D (robust + speed mode): find a candidate WITH data, then BLS→TLS
import os, time, math, warnings, numpy as np, pandas as pd, matplotlib.pyplot as plt, lightkurve as lk
from astropy.timeseries import BoxLeastSquares as BLS

# ---- OPTIONAL TLS ----
try:
    from transitleastsquares import transitleastsquares as TLS
    HAVE_TLS = True
except Exception:
    HAVE_TLS = False

# ===========================
# SPEED MODE TOGGLES (edit me)
# ===========================
SPEED_MODE = True        # <- flip to False for a deeper scan
TLS_ENABLE = False       # <- keep False for fastest; set True to run TLS in ±1% windows

# ---- BASE CONFIG (defaults; may be overridden by SPEED MODE) ----
USED_TICS     = {119584412, 37749396, 311183180}   # skip A–C
TABLE_CANDIDATES = [
    "results/targets_with_sectors.csv",
    "results/priority_targets.csv",
    "results/targets_ranked.csv",
]
REQUIRE_SECT  = 2            # prefer ≥2 sectors; SPEED_MODE may relax to 1
MAX_TRY_ROWS  = 200          # how many candidates to test before giving up
MAX_SPOC_SECT = 6            # cap SPOC sectors per candidate for speed
MAX_TCUT_SECT = 3            # cap TESSCut sectors per candidate for speed
WINDOW_DAYS   = 0.9          # gentle detrend window
BLS_PMIN, BLS_PMAX = 0.5, 50.0
BLS_NPER      = 5000         # number of BLS trial periods
DURATIONS_H   = np.linspace(0.5, 3.0, 18)
TLS_N_TOP_BLS = 3
TLS_FRAC_WIN  = 0.01         # ±1%
TLS_OVERSAMP  = 5            # TLS oversampling factor

# === Runtime guards (add under CONFIG) ===
TIME_LIMIT_S = 12000        # hard wall-clock cap (~20 minutes); tune as you like
t_start = time.time()
def _timed_out():
    return (time.time() - t_start) > TIME_LIMIT_S

# ---- SPEED MODE OVERRIDES ----
if SPEED_MODE:
    REQUIRE_SECT  = 1
    MAX_TRY_ROWS  = 40
    MAX_SPOC_SECT = 2
    MAX_TCUT_SECT = 1
    WINDOW_DAYS   = 0.8
    BLS_NPER      = 2000
    DURATIONS_H   = np.linspace(0.75, 2.0, 8)
    TLS_N_TOP_BLS = 1
    TLS_FRAC_WIN  = 0.01
    TLS_OVERSAMP  = 3
    if not HAVE_TLS:
        TLS_ENABLE = False  # no TLS installed
    else:
        TLS_ENABLE = bool(TLS_ENABLE)  # respect your toggle above

os.makedirs("figures", exist_ok=True)
os.makedirs("results", exist_ok=True)

def _print(msg): print(time.strftime("[%H:%M:%S]"), msg)

def _parse_sector_list(val):
    if isinstance(val, (list, tuple)): return [int(x) for x in val]
    if isinstance(val, str):
        s = val.strip().strip("[]")
        if not s: return []
        out = []
        for tok in s.split(","):
            tok = tok.strip()
            if tok:
                try: out.append(int(tok))
                except: pass
        return out
    return []

def _discover_spoc_sectors(tic):
    if _timed_out(): return []
    sr = lk.search_lightcurve(f"TIC {tic}", mission="TESS", author="SPOC")
    secs = sorted({getattr(r, "sector", None) for r in sr if getattr(r, "sector", None) is not None})
    return secs[:MAX_SPOC_SECT]

def _discover_tesscut_sectors(tic):
    if _timed_out(): return []
    try:
        sr = lk.search_tesscut(f"TIC {tic}")
        secs = sorted({getattr(r, "sector", None) for r in sr if getattr(r, "sector", None) is not None})
        return secs[-MAX_TCUT_SECT:]
    except Exception:
        return []

def _download_spoc_pdcsap(tic, sector):
    if _timed_out(): return None
    try:
        sr = lk.search_lightcurve(f"TIC {tic}", mission="TESS", author="SPOC", sector=sector)
        if len(sr) == 0: return None
        lc = sr.download(flux_column="pdcsap_flux")
        if lc is None or len(lc.time.value) == 0: return None
        return lc.remove_nans()
    except Exception:
        return None

def _download_tesscut_lc(tic, sector, cutout_size=15):
    if _timed_out(): return None
    try:
        sr = lk.search_tesscut(f"TIC {tic}", sector=sector)
        if len(sr) == 0: return None
        tpf = sr.download(cutout_size=cutout_size)
        if tpf is None: return None
        try:
            ap = tpf.create_threshold_mask(threshold=3)
            if ap.sum() == 0: ap = None
        except Exception:
            ap = None
        lc = tpf.to_lightcurve(aperture_mask=ap) if ap is not None else tpf.to_lightcurve()
        return lc.remove_nans()
    except Exception:
        return None

def _gentle_flatten(lc, window_days=WINDOW_DAYS):
    t = lc.time.value
    if len(t) < 10: return lc.normalize()
    dt = np.nanmedian(np.diff(np.sort(t)))
    wl = int(max(51, window_days/dt))
    if wl % 2 == 0: wl += 1
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        try:
            fl = lc.remove_outliers(sigma=6.0).flatten(window_length=wl, polyorder=2)
        except Exception:
            fl = lc.normalize()
    return fl.remove_nans()

def _run_bls(t, f):
    m = np.isfinite(t) & np.isfinite(f)
    t, f = t[m], f[m]
    if len(t) < 300: return None
    span = t.max() - t.min()
    pmax_eff = max(1.01, min(BLS_PMAX, span/2.0))
    periods = np.exp(np.linspace(np.log(BLS_PMIN), np.log(pmax_eff), BLS_NPER))
    model = BLS(t, f)
    res = model.power(periods, DURATIONS_H/24.0)
    return dict(periods=periods, power=res.power, bls=res, model=model,
                best_period=periods[np.argmax(res.power)])

def _top_peaks(periods, power, n=3, sep=0.02):
    idx = np.argsort(power)[::-1]
    picks = []
    for i in idx:
        p = periods[i]
        if all(abs(p - q)/q > sep for q,_ in picks):
            picks.append((p, power[i]))
        if len(picks) >= n: break
    return picks

def _run_tls_narrow(t, f, p0):
    if not (HAVE_TLS and TLS_ENABLE): return None
    m = np.isfinite(t) & np.isfinite(f); t, f = t[m], f[m]
    if len(t) < 300: return None
    tls = TLS(t, f)
    return tls.power(period_min=p0*(1-TLS_FRAC_WIN), period_max=p0*(1+TLS_FRAC_WIN),
                     use_threads='auto', oversampling_factor=TLS_OVERSAMP)

def _plot_bls(periods, power, out_png, title):
    plt.figure(figsize=(7.2,3.8))
    plt.plot(periods, power, lw=1)
    plt.xlabel("Period [days]"); plt.ylabel("BLS power")
    plt.title(title); plt.grid(alpha=0.3); plt.tight_layout(); plt.savefig(out_png, dpi=160); plt.close()

def _plot_fold(t, f, period, t0, out_png, title):
    m = np.isfinite(t) & np.isfinite(f); t, f = t[m], f[m]
    phase = ((t - t0 + 0.5*period) % period) / period - 0.5
    nb = 100
    bins = np.linspace(-0.5, 0.5, nb+1)
    idx = np.digitize(phase, bins) - 1
    y = np.array([np.nanmean(f[idx==k]) for k in range(nb)])
    x = 0.5*(bins[:-1]+bins[1:])
    plt.figure(figsize=(6.6,3.6))
    plt.scatter(phase, f, s=2, alpha=0.25, rasterized=True)
    plt.plot(x, y, lw=2)
    plt.xlabel("Phase"); plt.ylabel("Normalized flux"); plt.title(title)
    plt.grid(alpha=0.3); plt.tight_layout(); plt.savefig(out_png, dpi=160); plt.close()

# ---- Load a candidates table ----
cand = None
src = None
for p in TABLE_CANDIDATES:
    if os.path.exists(p):
        cand = pd.read_csv(p); src = p; break
if cand is None or cand.empty:
    raise SystemExit("No candidate table found. Expected one of:\n  - " + "\n  - ".join(TABLE_CANDIDATES))

# Normalize columns
cols = {c.lower(): c for c in cand.columns}
tic_col  = cols.get("tic") or cols.get("tic_id") or cols.get("ticid") or cols.get("tic_id_norm")
name_col = cols.get("name") or cols.get("toi") or cols.get("target") or cols.get("designation")
sects_col = cols.get("sectors_now") or cols.get("sectors") or cols.get("observed_sectors") or cols.get("sector_list")
if tic_col is None: raise SystemExit(f"No TIC column in {src}")

cand["__tic"] = pd.to_numeric(cand[tic_col], errors="coerce").astype("Int64")
pool = cand.dropna(subset=["__tic"])
pool = pool[~pool["__tic"].isin(USED_TICS)]

# Prefer many sectors if a list exists
if sects_col:
    pool = pool.assign(__nsec=pool[sects_col].map(_parse_sector_list).map(len))
    pool = pool.sort_values("__nsec", ascending=False)

# ---- Try candidates until one has data ----
picked = None
rows_tried = 0
for _, r in pool.head(MAX_TRY_ROWS).iterrows():
    if _timed_out():
        _print("Time limit reached — stopping auto-pick loop.")
        break
    rows_tried += 1
    tic = int(r["__tic"])
    name = str(r.get(name_col, f"TIC {tic}"))
    hint_secs = _parse_sector_list(r[sects_col]) if sects_col else []
    _print(f"Check {name}  TIC {tic}  (hint sectors: {hint_secs if hint_secs else '—'})")

    spoc_secs = _discover_spoc_sectors(tic)
    cut_secs  = _discover_tesscut_sectors(tic)
    all_secs  = list(dict.fromkeys(hint_secs + spoc_secs + cut_secs))
    if REQUIRE_SECT and len(all_secs) < REQUIRE_SECT:
        continue

    lcs, used, src_kind = [], [], []
    tried = all_secs if all_secs else spoc_secs or cut_secs
    tried = tried[:max(MAX_SPOC_SECT, MAX_TCUT_SECT)]
    for s in tried:
        if _timed_out():
            _print("Time limit reached during downloads — stopping.")
            break
        lc = _download_spoc_pdcsap(tic, s)
        if lc is not None and len(lc.time.value) > 300:
            lcs.append(lc); used.append(s); src_kind.append("SPOC"); continue
        lc = _download_tesscut_lc(tic, s)
        if lc is not None and len(lc.time.value) > 300:
            lcs.append(lc); used.append(s); src_kind.append("TESSCut")
    if len(lcs) == 0:
        continue

    picked = dict(tic=tic, name=name, sectors=used, sources=sorted(set(src_kind)))
    break

if picked is None:
    msg = []
    if _timed_out():
        msg.append("Stopped due to time cap.")
    msg.append(f"No suitable Target D found.\nTried {rows_tried} rows.")
    msg.append("Tips:\n - Increase MAX_TRY_ROWS and/or TIME_LIMIT_S.\n - Keep REQUIRE_SECT=1 (we will use TESSCut if SPOC missing).\n - Or do a manual pick from results/targets_with_sectors.csv (choose a TIC with ≥2 sectors).")
    raise SystemExit("\n".join(msg))

# ---- Detrend, stitch, search ----
TARGET_TIC, TARGET_NAME = picked["tic"], picked["name"]
_print(f"\nChosen Target D: {TARGET_NAME} — TIC {TARGET_TIC}")
_print(f"Sectors used: {picked['sectors']}  (sources: {picked['sources']})")

flats = []
for s, lc in zip(picked["sectors"], lcs):
    fl = _gentle_flatten(lc, WINDOW_DAYS)
    try: fl = fl.normalize()
    except Exception: pass
    fl.meta["sector"] = s
    flats.append(fl)

stitch = lk.LightCurveCollection(flats).stitch().remove_nans()
t, f = stitch.time.value, stitch.flux.value

_print("Running BLS (wide)…")
bls = _run_bls(t, f)
if bls is None: raise SystemExit("BLS failed (too few points).")

top = _top_peaks(bls["periods"], bls["power"], n=TLS_N_TOP_BLS)
bls_png = f"figures/TIC{TARGET_TIC}_quickscan_BLS.png"
_plot_bls(bls["periods"], bls["power"], bls_png, f"{TARGET_NAME} — BLS (wide)")

tls_rows, fold_pngs = [], []
if HAVE_TLS and TLS_ENABLE:
    _print(f"Running TLS in ±{int(TLS_FRAC_WIN*100)}% windows around top BLS peaks…")
    for i, (p0, pow0) in enumerate(top, 1):
        res = _run_tls_narrow(t, f, p0)
        if res is None: continue
        tls_rows.append({
            "rank": i,
            "p0_bls_days": float(p0),
            "tls_best_period_days": float(res.period),
            "tls_sde": float(res.SDE),
            "tls_depth_ppm": float(1e6*res.depth),
            "tls_duration_hr": float(24.0*res.duration),
            "tls_transit_count": int(res.transit_count),
        })
        fold_png = f"figures/TIC{TARGET_TIC}_quickscan_TLS_fold_rank{i}.png"
        _plot_fold(t, f, res.period, t0=np.nanmin(t), out_png=fold_png,
                   title=f"{TARGET_NAME} — TLS fold (rank {i}, P≈{res.period:.5f} d)")
        fold_pngs.append(fold_png)
else:
    _print("TLS disabled (speed mode) or not installed — skipping TLS step (BLS results still saved).")

# ---- Save tables & summary ----
bls_top = pd.DataFrame(
    [{"rank": i, "bls_period_days": float(p), "bls_power": float(pow_)} for i,(p,pow_) in enumerate(top, 1)]
)
tls_df = pd.DataFrame(tls_rows) if len(tls_rows) else pd.DataFrame(
    columns=["rank","p0_bls_days","tls_best_period_days","tls_sde","tls_depth_ppm","tls_duration_hr","tls_transit_count"]
)
summary = pd.DataFrame([{
    "tic": TARGET_TIC,
    "name": TARGET_NAME,
    "n_sectors": len(picked["sectors"]),
    "sectors": ",".join(str(s) for s in picked["sectors"]),
    "sources": ",".join(picked["sources"]),
    "bls_best_period_days": float(bls["best_period"]),
    "tls_best_period_days": float(tls_df.iloc[0]["tls_best_period_days"]) if len(tls_df) else np.nan,
    "tls_best_sde": float(tls_df.iloc[0]["tls_sde"]) if len(tls_df) else np.nan,
    "speed_mode": SPEED_MODE,
    "tls_enabled": bool(HAVE_TLS and TLS_ENABLE),
}])

bls_csv = f"results/TIC{TARGET_TIC}_quickscan_BLS_top.csv"
tls_csv = f"results/TIC{TARGET_TIC}_quickscan_TLS_top.csv"
sum_csv = f"results/TIC{TARGET_TIC}_quickscan_summary.csv"
bls_top.to_csv(bls_csv, index=False)
tls_df.to_csv(tls_csv, index=False)
summary.to_csv(sum_csv, index=False)

_print("\nQuick scan complete.")
_print(f"Saved:\n - {bls_png}\n - {bls_csv}\n - {tls_csv}\n - {sum_csv}")
for p in fold_pngs: _print(f" - {p})")

[20:18:17] Check 1487.01  TIC 459978312  (hint sectors: —)
[20:18:18] Check 5789.01  TIC 87216634  (hint sectors: —)
[20:18:18] Check 186.01  TIC 279741379  (hint sectors: —)
[20:18:18] Check 2009.01  TIC 243187830  (hint sectors: —)
[20:18:19] Check 197.01  TIC 441462736  (hint sectors: —)
[20:18:19] Check 2194.01  TIC 271478281  (hint sectors: —)
[20:18:19] Check 1611.01  TIC 264678534  (hint sectors: —)
[20:18:19] Check 4328.01  TIC 77175217  (hint sectors: —)
[20:18:19] Check 2134.02  TIC 75878355  (hint sectors: —)
[20:18:20] Check 1793.01  TIC 304142124  (hint sectors: —)
[20:18:20] Check 7032.01  TIC 22903436  (hint sectors: —)
[20:18:20] Check 431.01  TIC 31374837  (hint sectors: —)
[20:18:20] Check 2540.01  TIC 354518617  (hint sectors: —)
[20:18:20] Check 2443.01  TIC 318753380  (hint sectors: —)
[20:18:20] Check 402.02  TIC 120896927  (hint sectors: —)
[20:18:20] Check 2076.01  TIC 27491137  (hint sectors: —)
[20:18:20] Check 1405.01  TIC 387834907  (hint sectors: —)
[20:18:

SystemExit: No suitable Target D found.
Tried 40 rows.
Tips:
 - Increase MAX_TRY_ROWS and/or TIME_LIMIT_S.
 - Keep REQUIRE_SECT=1 (we will use TESSCut if SPOC missing).
 - Or do a manual pick from results/targets_with_sectors.csv (choose a TIC with ≥2 sectors).

In [16]:
# %% Shortlist: top-by-sectors (skip A–C)
import pandas as pd, os
USED_TICS = {119584412, 37749396, 311183180}
paths = ["results/targets_with_sectors.csv","results/priority_targets.csv","results/targets_ranked.csv"]
df, src = None, None
for p in paths:
    if os.path.exists(p):
        df = pd.read_csv(p); src = p; break
assert df is not None and not df.empty, "No candidates table found."

def _parse_sectors(val):
    if isinstance(val, str):
        s = val.strip().strip("[]")
        if not s: return []
        return [int(x) for x in s.split(",") if x.strip().isdigit()]
    return []

cols = {c.lower(): c for c in df.columns}
tic_col = cols.get("tic") or cols.get("tic_id") or cols.get("ticid") or cols.get("tic_id_norm")
name_col = cols.get("name") or cols.get("toi") or cols.get("target") or cols.get("designation")
sect_col = cols.get("sectors_now") or cols.get("sectors") or cols.get("observed_sectors") or cols.get("sector_list")

df["__tic"] = pd.to_numeric(df[tic_col], errors="coerce").astype("Int64")
pool = df.dropna(subset=["__tic"])
if sect_col:
    pool["__sectors"] = pool[sect_col].map(_parse_sectors)
    pool["__nsec"] = pool["__sectors"].map(len)
else:
    pool["__sectors"] = [[]]*len(pool); pool["__nsec"] = 0
pool = pool[~pool["__tic"].isin(USED_TICS)].sort_values("__nsec", ascending=False)

show = pool[[name_col, "__tic", "__nsec", sect_col if sect_col else name_col]].head(10).copy()
print(f"Source table: {src}")
print("Top candidates by sector count (skipping A–C):")
for _, r in show.iterrows():
    nm = str(r[name_col]); tic = int(r["__tic"]); nsec = int(r["__nsec"])
    secs = str(r.get(sect_col, "—"))
    print(f" - {nm:12s}  TIC {tic:<10d}  sectors:{nsec:<2d}  list:{secs}")

Source table: results/targets_with_sectors.csv
Top candidates by sector count (skipping A–C):
 - toi    1487.01
toi    1487.01
Name: 0, dtype: object  TIC 459978312   sectors:0   list:—
 - toi    880.03
toi    880.03
Name: 31, dtype: object  TIC 34077285    sectors:0   list:—
 - toi    815.01
toi    815.01
Name: 33, dtype: object  TIC 102840239   sectors:0   list:—
 - toi    139.01
toi    139.01
Name: 34, dtype: object  TIC 62483237    sectors:0   list:—
 - toi    2459.01
toi    2459.01
Name: 35, dtype: object  TIC 192790476   sectors:0   list:—
 - toi    6075.01
toi    6075.01
Name: 36, dtype: object  TIC 424388628   sectors:0   list:—
 - toi    1563.01
toi    1563.01
Name: 37, dtype: object  TIC 249945230   sectors:0   list:—
 - toi    454.01
toi    454.01
Name: 38, dtype: object  TIC 153077621   sectors:0   list:—
 - toi    4310.01
toi    4310.01
Name: 39, dtype: object  TIC 303317324   sectors:0   list:—
 - toi    1030.01
toi    1030.01
Name: 40, dtype: object  TIC 464296022   sect

In [21]:
# %% Quick MAST connectivity check (hard per-call caps; non-blocking)
import time, concurrent.futures as cf
import lightkurve as lk

# Tighten astroquery service timeouts
try:
    from astroquery.mast import Observations
    Observations.TIMEOUT = 15  # seconds
except Exception:
    pass
try:
    from astroquery.mast import Tesscut
    Tesscut.TIMEOUT = 15
except Exception:
    pass

TARGET = "TIC 307210830"  # bright, well-known TIC to probe

def probe(label, fn, timeout=15):
    t0 = time.time()
    with cf.ThreadPoolExecutor(max_workers=1) as ex:
        fut = ex.submit(fn)
        try:
            r = fut.result(timeout=timeout)
            ok = (len(r) > 0)
            dt = time.time() - t0
            print(f"[{label:7s}] {'OK' if ok else 'EMPTY'} in {dt:.1f}s")
            return ok
        except cf.TimeoutError:
            dt = time.time() - t0
            print(f"[{label:7s}] TIMEOUT at {dt:.1f}s")
            return False
        except Exception as e:
            dt = time.time() - t0
            print(f"[{label:7s}] ERROR in {dt:.1f}s: {type(e).__name__}")
            return False

print("Probing MAST endpoints with strict caps...")
ok_spoc = probe("SPOC", lambda: lk.search_lightcurve(TARGET, mission="TESS", author="SPOC"), timeout=15)
ok_qlp  = probe("QLP",  lambda: lk.search_lightcurve(TARGET, mission="TESS", author="QLP"),  timeout=15)

# Optional: TESSCut is heavier; skip by default. Flip RUN_TESSCUT=True if you really want to test it.
RUN_TESSCUT = False
ok_tcut = False
if RUN_TESSCUT:
    ok_tcut = probe("TESSCut", lambda: lk.search_tesscut(TARGET), timeout=12)

if any([ok_spoc, ok_qlp, ok_tcut]):
    print("=> MAST reachable (at least one endpoint).")
else:
    print("=> All probes failed within the caps; treat MAST as unavailable right now.")

Probing MAST endpoints with strict caps...
[SPOC   ] OK in 2.8s
[QLP    ] OK in 0.0s
=> MAST reachable (at least one endpoint).


In [28]:
# %% MAST health probe (3s). If this says "UNHEALTHY", skip online scans and run the local-scan cell below.
import time, lightkurve as lk
try: lk.log.setLevel('ERROR')
except Exception: pass

def mast_products_ok(timeout=3.0):
    try:
        t0 = time.time()
        # Known evergreen target: LHS 3844 (TIC 307210830)
        r = lk.search_lightcurve("TIC 307210830", mission="TESS")
        # We only need *some* product list back, not a download
        return len(r) > 0 and (time.time()-t0) <= max(timeout, 0.1)
    except Exception:
        return False

ok = mast_products_ok()
print("MAST product discovery:", "HEALTHY" if ok else "UNHEALTHY")

MAST product discovery: HEALTHY


In [32]:
# %% Quick scan Target D (OFFLINE-ONLY, robust to columns): harvest local LCFs -> pick TIC != {A,B,C} -> BLS
import os, sys, glob, time, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightkurve as lk
from astropy.timeseries import BoxLeastSquares as BLS
from lightkurve import LightCurve

# Quiet down warnings/logs
warnings.filterwarnings("ignore")
try:
    lk.log.setLevel("ERROR")
except Exception:
    pass

# ---------- CONFIG ----------
USED_TICS         = {119584412, 37749396, 311183180}   # A, B, C (skip)
SECTORS_PER_TIC   = 2                                   # up to N files per TIC
WINDOW_DAYS       = 0.8
BLS_PMIN, BLS_PMAX, BLS_NPER = 0.5, 50.0, 2000
DURATIONS_H       = np.linspace(0.75, 2.0, 9)
OUT_FIG_DIR       = "figures"
OUT_RES_DIR       = "results"

# Likely cache roots
CAND_ROOTS = [
    "./mastDownload",
    os.path.expanduser("~/Downloads/mastDownload"),
    os.path.expanduser("~/.lightkurve"),
    os.path.expanduser("~/.astropy/cache"),
    os.path.expanduser("~/.astropy/cache/astroquery"),
    os.path.expanduser("~/.cache/astroquery"),
    os.path.expanduser("~/Library/Caches/astroquery"),  # macOS
]
if os.environ.get("ASTROPY_CACHE_DIR"):
    CAND_ROOTS.append(os.path.expanduser(os.environ["ASTROPY_CACHE_DIR"]))

os.makedirs(OUT_FIG_DIR, exist_ok=True)
os.makedirs(OUT_RES_DIR, exist_ok=True)
def _print(*a): print(time.strftime("[%H:%M:%S]"), *a)

# ---------- helpers ----------
def _best_flux_columns(ts):
    """Return (flux_col, err_col) names to use, preferring PDCSAP where available."""
    cols = set(map(str.lower, ts.colnames))
    # Map lowercase -> actual name (preserve original case)
    name_map = {c.lower(): c for c in ts.colnames}
    # Preferred flux columns in order
    pref_flux = ["pdcsap_flux", "flux", "sap_flux", "kspsap_flux"]
    flux_col = next((name_map[c] for c in pref_flux if c in cols), None)
    if flux_col is None:
        return None, None
    # Try a good matching error column
    candidates_err = [
        flux_col.lower().replace("flux", "flux_err"),
        "pdcsap_flux_err", "sap_flux_err", "flux_err"
    ]
    err_col = next((name_map[c] for c in candidates_err if c in cols), None)
    return flux_col, err_col

def _to_clean_lightcurve(ts, time_col="time"):
    """Create a LightCurve with .time, .flux, .flux_err using the best available columns."""
    if time_col not in ts.colnames:
        return None
    flux_col, err_col = _best_flux_columns(ts)
    if flux_col is None:
        return None
    # Build a fresh LightCurve
    lc = LightCurve(time=ts[time_col])
    lc["flux"] = ts[flux_col]
    if err_col is not None:
        lc["flux_err"] = ts[err_col]
    return lc.remove_nans()

def _extract_tic_from_meta(meta):
    """Try to pull TIC from header metadata."""
    keys = ["TICID", "TIC_ID", "TIC", "OBJECT", "TARGETID", "TARGET_ID"]
    for k in keys:
        if k in meta and meta[k] not in (None, "", "UNKNOWN"):
            s = str(meta[k]).strip()
            s = s.replace("TIC", "").replace("tic", "").replace(" ", "")
            v = pd.to_numeric(s, errors="coerce")
            if pd.notnull(v):
                try:
                    return int(v)
                except Exception:
                    pass
    return None

def _load_lc_from_file(path):
    """Open an *lc.fits(.gz) and return (LightCurve, TIC) or (None, None)."""
    try:
        obj = lk.read(path)  # robust, replaces deprecated open()
    except Exception:
        return None, None
    # obj can be LightCurve, TessLightCurve, or a collection. Normalize to a TimeSeries-ish view.
    try:
        ts = obj.to_timeseries()
    except Exception:
        # Some LK objects already behave like TimeSeries; try attributes directly
        try:
            ts = obj
            ts.colnames  # probes TimeSeries-like
        except Exception:
            return None, None
    # Create clean LC with chosen flux
    lc = _to_clean_lightcurve(ts)
    if lc is None or len(lc.time.value) < 50:
        return None, None
    # TIC from meta if present; fallback to None
    tic = _extract_tic_from_meta(getattr(obj, "meta", {}))
    return lc, tic

def _gentle_flatten(lc, window_days=WINDOW_DAYS):
    t = lc.time.value
    if len(t) < 10:
        return lc.normalize()
    dt = np.nanmedian(np.diff(np.sort(t)))
    wl = int(max(51, window_days/dt))
    if wl % 2 == 0:
        wl += 1
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        try:
            fl = lc.remove_outliers(sigma=6.0).flatten(window_length=wl, polyorder=2)
        except Exception:
            fl = lc.normalize()
    return fl.remove_nans()

def _run_bls(t, f):
    m = np.isfinite(t) & np.isfinite(f)
    t, f = t[m], f[m]
    if len(t) < 300:
        return None
    span = t.max() - t.min()
    pmax_eff = max(1.01, min(BLS_PMAX, span/2.0))
    periods = np.exp(np.linspace(np.log(BLS_PMIN), np.log(pmax_eff), BLS_NPER))
    res = BLS(t, f).power(periods, DURATIONS_H/24.0)
    return dict(best=periods[np.argmax(res.power)], periods=periods, power=res.power)

def _plot_bls(periods, power, out_png, title):
    plt.figure(figsize=(7.2, 3.8))
    plt.plot(periods, power, lw=1)
    plt.xlabel("Period [days]"); plt.ylabel("BLS power"); plt.title(title)
    plt.grid(alpha=0.3); plt.tight_layout(); plt.savefig(out_png, dpi=160); plt.close()

# ---------- 1) harvest local LCFs ----------
_print("Scanning local caches for *lc.fits …")
found_files = []
for root in CAND_ROOTS:
    if not root or not os.path.isdir(root):
        continue
    found_files += glob.glob(os.path.join(root, "**", "*lc.fits"), recursive=True)
    found_files += glob.glob(os.path.join(root, "**", "*lc.fits.gz"), recursive=True)

found_files = sorted(set(found_files))
_print(f"Found {len(found_files)} candidate LCFs across caches.")

# ---------- 2) group by TIC (skip A–C) ----------
tic_to_lcs = {}
for path in found_files:
    lc, tic = _load_lc_from_file(path)
    if lc is None or tic is None:
        continue
    if tic in USED_TICS:
        continue
    tic_to_lcs.setdefault(tic, []).append(lc)

if not tic_to_lcs:
    raise SystemExit(
        "No usable local light curves (with recognizable TIC) outside A–C.\n"
        "Options right now:\n"
        " - Drop any known SPOC/QLP *lc.fits into ./mastDownload and re-run.\n"
        " - Or temporarily re-run B/C and label that as a ‘Target D quick-scan (fallback)’ in today’s log."
    )

# ---------- 3) pick richest TIC and use up to SECTORS_PER_TIC ----------
cands = sorted(tic_to_lcs.items(), key=lambda kv: len(kv[1]), reverse=True)
picked_tic, lc_list = cands[0]
use_lc = lc_list[:SECTORS_PER_TIC]
_print(f"Picked offline Target D: TIC {picked_tic} (using {len(use_lc)} file(s), found {len(lc_list)} total)")

# ---------- 4) detrend, stitch, BLS ----------
flats = []
for lc in use_lc:
    fl = _gentle_flatten(lc, WINDOW_DAYS)
    try:
        fl = fl.normalize()
    except Exception:
        pass
    flats.append(fl)

stitch = lk.LightCurveCollection(flats).stitch().remove_nans()
t, f = stitch.time.value, stitch.flux.value
_print(f"Stitched points: N={len(t)}")

bls = _run_bls(t, f)
if bls is None:
    raise SystemExit("BLS failed (too few points). Try increasing SECTORS_PER_TIC or placing more LCFs in cache.")

# ---------- 5) save artifacts ----------
bls_png = os.path.join(OUT_FIG_DIR, f"TIC{picked_tic}_quickscan_BLS_offline.png")
_plot_bls(bls["periods"], bls["power"], bls_png, f"TIC {picked_tic} (offline) — BLS (wide)")

summary = pd.DataFrame([{
    "tic": int(picked_tic),
    "name": f"Target D (offline cache) — TIC {picked_tic}",
    "n_lc_files_used": int(len(use_lc)),
    "N_points": int(len(t)),
    "bls_best_period_days": float(bls["best"]),
}])
sum_csv = os.path.join(OUT_RES_DIR, f"TIC{picked_tic}_quickscan_summary_offline.csv")
summary.to_csv(sum_csv, index=False)

print("\nQuick scan complete (OFFLINE).")
print("Saved:\n -", bls_png, "\n -", sum_csv)

[06:43:59] Scanning local caches for *lc.fits …
[06:43:59] Found 121 candidate LCFs across caches.
[06:44:07] Picked offline Target D: TIC 150428135 (using 2 file(s), found 41 total)
[06:44:07] Stitched points: N=30658

Quick scan complete (OFFLINE).
Saved:
 - figures/TIC150428135_quickscan_BLS_offline.png 
 - results/TIC150428135_quickscan_summary_offline.csv


In [34]:
# %% Locate local MAST/Lightkurve caches and list candidate *lc.fits files
import os, glob, sys, platform, pathlib, time
def _print(*a): print(time.strftime("[%H:%M:%S]"), *a)

# Common cache roots across macOS/Linux/conda
home = os.path.expanduser("~")
roots = [
    os.getcwd(),
    os.path.join(os.getcwd(), "mastDownload"),
    os.path.join(home, "mastDownload"),
    os.path.join(home, "Downloads", "mastDownload"),
    os.path.join(home, ".lightkurve-cache", "mastDownload"),
    os.path.join(home, "Library", "Caches", "lightkurve", "mastDownload"),  # macOS
    os.path.join(home, ".astropy", "cache"),  # sometimes nested under here
]

# Add any env-specified cache dirs if present
for k in ("XDG_CACHE_HOME", "ASTROPY_CACHE_DIR", "LIGHTKURVE_CACHE_DIR"):
    v = os.environ.get(k)
    if v:
        roots.append(v)
        roots.append(os.path.join(v, "mastDownload"))

# De-dup and keep only existing dirs
roots = [str(pathlib.Path(r).expanduser()) for r in roots]
roots = [r for r in dict.fromkeys(roots) if os.path.isdir(r)]

_print("Searching roots:")
for r in roots: print(" -", r)

patterns = ["**/*lc.fits", "**/*lc.fits.gz", "**/*-lc.fits", "**/*-lc.fits.gz"]
found = []
for r in roots:
    for pat in patterns:
        found += glob.glob(os.path.join(r, pat), recursive=True)

# De-dup while preserving order
seen=set(); files=[]
for p in found:
    q=os.path.abspath(p)
    if q not in seen:
        files.append(q); seen.add(q)

print(f"\nFound {len(files)} candidate light-curve files:")
for p in files[:20]:
    print(" •", p)
if len(files) > 20:
    print(f" … (+{len(files)-20} more)")

[09:06:33] Searching roots:
 - /Users/kobi.weitzman/Documents/tess-ephem
 - /Users/kobi.weitzman/.astropy/cache

Found 34 candidate light-curve files:
 • /Users/kobi.weitzman/Documents/tess-ephem/data_raw_fresh/mastDownload/TESS/tess2023209231226-s0068-0000000062483237-0262-a_fast/tess2023209231226-s0068-0000000062483237-0262-a_fast-lc.fits
 • /Users/kobi.weitzman/Documents/tess-ephem/data_raw_fresh/mastDownload/TESS/tess2022057073128-s0049-0000000119584412-0221-s/tess2022057073128-s0049-0000000119584412-0221-s_lc.fits
 • /Users/kobi.weitzman/Documents/tess-ephem/data_raw_fresh/mastDownload/TESS/tess2020212050318-s0028-0000000150428135-0190-a_fast/tess2020212050318-s0028-0000000150428135-0190-a_fast-lc.fits
 • /Users/kobi.weitzman/Documents/tess-ephem/data_raw_fresh/mastDownload/TESS/tess2021065132309-s0036-0000000150428135-0207-a_fast/tess2021065132309-s0036-0000000150428135-0207-a_fast-lc.fits
 • /Users/kobi.weitzman/Documents/tess-ephem/data_raw_fresh/mastDownload/TESS/tess201831909

In [37]:
# %% Quick-scan FINISH (OFFLINE, robust header-based TIC parsing)
import os, glob, warnings, re, numpy as np, pandas as pd, matplotlib.pyplot as plt, time, pathlib
import lightkurve as lk
from astropy.timeseries import BoxLeastSquares as BLS
from astropy.io import fits

def _print(*a): print(time.strftime("[%H:%M:%S]"), *a)

# === prefer this TIC if present; else auto-pick ===
PREFERRED_TIC = None   # e.g. 150428135; leave None to auto-pick first usable

WINDOW_DAYS = 0.8
BLS_PMIN, BLS_PMAX, BLS_NPER = 0.5, 50.0, 5000
DURATIONS_H = np.linspace(0.75, 2.5, 12)

os.makedirs("figures", exist_ok=True); os.makedirs("results", exist_ok=True)

# ---------- locate cached LCFs (reuse common roots) ----------
home = os.path.expanduser("~")
roots = [
    os.getcwd(),
    os.path.join(os.getcwd(), "mastDownload"),
    os.path.join(home, "mastDownload"),
    os.path.join(home, "Downloads", "mastDownload"),
    os.path.join(home, ".lightkurve-cache", "mastDownload"),
    os.path.join(home, "Library", "Caches", "lightkurve", "mastDownload"),
    os.path.join(home, ".astropy", "cache"),
]
for k in ("XDG_CACHE_HOME", "ASTROPY_CACHE_DIR", "LIGHTKURVE_CACHE_DIR"):
    v = os.environ.get(k); 
    if v: roots += [v, os.path.join(v, "mastDownload")]
roots = [str(pathlib.Path(r).expanduser()) for r in roots]
roots = [r for r in dict.fromkeys(roots) if os.path.isdir(r)]

patterns = ["**/*lc.fits", "**/*lc.fits.gz", "**/*-lc.fits", "**/*-lc.fits.gz"]
all_files=[]
for r in roots:
    for pat in patterns:
        all_files += glob.glob(os.path.join(r, pat), recursive=True)
# de-dup
seen=set(); files=[]
for p in all_files:
    q=os.path.abspath(p)
    if q not in seen:
        files.append(q); seen.add(q)

if not files:
    raise SystemExit("No local *lc.fits files found. Copy any SPOC/QLP LCFs into ./mastDownload and re-run.")

# ---------- robust TIC parsing ----------
# 1) Try to parse the long zero-padded number in SPOC filenames, e.g. ...-0000000123456789-... -> 123456789
re_longnum = re.compile(r"(\d{12,18})")  # padded TICs often 16-17 digits
# 2) Try direct 'TIC 123456789' in filename/dirs
re_ticword  = re.compile(r"TIC[\s\-_]?(\d{5,12})", re.I)

def _tic_from_basename_or_dirs(path):
    base = os.path.basename(path)
    m = re_ticword.search(base) or re_ticword.search(os.path.dirname(path))
    if m: 
        return int(m.group(1))
    m2 = re_longnum.search(base)
    if m2:
        return int(m2.group(1).lstrip("0") or "0")  # unpad
    return None

def _tic_from_header(path):
    # Look in FITS headers for TICID / OBJECT / TARGNAME / TARGETID
    try:
        with fits.open(path, memmap=False) as hdul:
            # check primary then first extension
            cards = {}
            for h in (hdul[0].header, *(h.header for h in hdul[1:2] if len(hdul)>1)):
                for key in ("TICID","OBJECT","TARGNAME","TARGETID"):
                    if key in h:
                        cards[key] = str(h[key])
            # Priority: explicit TICID
            for key in ("TICID","TARGETID"):
                if key in cards:
                    try:
                        return int(str(cards[key]).strip())
                    except Exception:
                        pass
            # OBJECT/TARGNAME may be "TIC 123..." — extract digits
            for key in ("OBJECT","TARGNAME"):
                if key in cards:
                    m = re_ticword.search(cards[key])
                    if m: 
                        return int(m.group(1))
                    # as a last resort, grab the longest digit run
                    m2 = re.search(r"(\d{5,12})", cards[key])
                    if m2:
                        return int(m2.group(1))
    except Exception:
        pass
    # fallback: ask Lightkurve for meta if possible
    try:
        obj = lk.open(path)
        meta = getattr(obj, "meta", {}) or {}
        for key in ("TICID","TARGETID","OBJECT"):
            if key in meta:
                try: return int(str(meta[key]).strip())
                except: pass
        # lightkurve sometimes exposes targetid on the object directly
        for key in ("targetid","target_id"):
            if hasattr(obj, key):
                try: return int(getattr(obj, key))
                except: pass
    except Exception:
        pass
    return None

tic_to_files = {}
for path in files:
    tic = _tic_from_basename_or_dirs(path)
    if tic is None:
        tic = _tic_from_header(path)
    if tic:
        tic_to_files.setdefault(tic, []).append(path)

if not tic_to_files:
    # Help the user debug by showing a few filenames/headers
    sample = files[:3]
    msg = "Found LCFs but still could not parse TIC IDs—even from headers.\n"
    msg += "Examples:\n" + "\n".join(" - "+os.path.basename(p) for p in sample)
    raise SystemExit(msg)

# ---------- choose TIC ----------
if PREFERRED_TIC and (PREFERRED_TIC in tic_to_files):
    target_tic = PREFERRED_TIC
else:
    # pick the TIC with the most files (likely best coverage)
    target_tic = max(tic_to_files, key=lambda k: len(tic_to_files[k]))

_print(f"Using cached TIC: {target_tic}  (#files={len(tic_to_files[target_tic])})")

# ---------- light curve loading / quick scan ----------
def _open_lc(path):
    try:
        obj = lk.open(path)
    except Exception:
        return None
    for attr in ("PDCSAP_FLUX", "SAP_FLUX"):
        lc = getattr(obj, attr, None)
        if lc is not None:
            try:
                lc = lc.remove_nans()
                if len(lc.time.value) >= 300:
                    return lc
            except Exception:
                pass
    try:
        lc = obj.to_lightcurve().remove_nans()
        if len(lc.time.value) >= 300:
            return lc
    except Exception:
        pass
    return None

def _gentle_flatten(lc, window_days=WINDOW_DAYS):
    t = lc.time.value
    if len(t) < 10: return lc.normalize()
    dt = np.nanmedian(np.diff(np.sort(t)))
    wl = int(max(51, window_days/dt)); 
    if wl % 2 == 0: wl += 1
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        try:
            fl = lc.remove_outliers(sigma=6.0).flatten(window_length=wl, polyorder=2)
        except Exception:
            fl = lc.normalize()
    return fl.remove_nans()

def _run_bls(t, f):
    m = np.isfinite(t) & np.isfinite(f); t, f = t[m], f[m]
    if len(t) < 300: return None
    span = t.max() - t.min()
    pmax_eff = max(1.01, min(BLS_PMAX, span/2.0))
    periods = np.exp(np.linspace(np.log(BLS_PMIN), np.log(pmax_eff), BLS_NPER))
    model = BLS(t, f); res = model.power(periods, DURATIONS_H/24.0)
    return dict(periods=periods, power=res.power, best=periods[np.argmax(res.power)])

def _plot_bls(periods, power, out_png, title):
    plt.figure(figsize=(7.6,4.1))
    plt.plot(periods, power, lw=1)
    plt.xlabel("Period [days]"); plt.ylabel("BLS power"); plt.title(title)
    plt.grid(alpha=0.3); plt.tight_layout(); plt.savefig(out_png, dpi=160); plt.close()

def _plot_fold(t, f, period, t0, out_png, title):
    m = np.isfinite(t) & np.isfinite(f); t, f = t[m], f[m]
    phase = ((t - t0 + 0.5*period) % period) / period - 0.5
    nb = 120; bins = np.linspace(-0.5, 0.5, nb+1)
    idx = np.digitize(phase, bins) - 1
    y = np.array([np.nanmean(f[idx==k]) for k in range(nb)])
    x = 0.5*(bins[:-1]+bins[1:])
    plt.figure(figsize=(6.8,3.8))
    plt.scatter(phase, f, s=2, alpha=0.25, rasterized=True); plt.plot(x, y, lw=2)
    plt.xlabel("Phase"); plt.ylabel("Normalized flux"); plt.title(title)
    plt.grid(alpha=0.3); plt.tight_layout(); plt.savefig(out_png, dpi=160); plt.close()

# Use up to 2 files for speed
picked_files = tic_to_files[target_tic][:2]
lcs=[]
for path in picked_files:
    lc = _open_lc(path)
    if lc is not None:
        lcs.append(lc)
if not lcs:
    raise SystemExit(f"No usable light curves found for TIC {target_tic} among {len(picked_files)} files.")

# Detrend + stitch
flats=[_gentle_flatten(lc, WINDOW_DAYS).normalize() for lc in lcs]
stitch = lk.LightCurveCollection(flats).stitch().remove_nans()
t, f = stitch.time.value, stitch.flux.value
_print(f"Stitched points: N={len(t)} (from {len(lcs)} file(s))")

# BLS
bls = _run_bls(t, f)
if bls is None:
    raise SystemExit("BLS failed (too few points).")
p_best = float(bls["best"])

# Save artifacts
bls_png = f"figures/TIC{target_tic}_quickscan_BLS_offline.png"
_plot_bls(bls["periods"], bls["power"], bls_png, f"TIC {target_tic} (offline) — BLS (wide)")

t0 = float(np.nanmin(t))
folds = []
for tag, factor in [("Pbest",1.0), ("halfP",0.5), ("doubleP",2.0)]:
    P = p_best * factor
    out = f"figures/TIC{target_tic}_quickscan_fold_{tag}.png"
    _plot_fold(t, f, P, t0, out, f"TIC {target_tic} — fold @ {tag} (P≈{P:.5f} d)")
    folds.append(out)

out_csv = f"results/TIC{target_tic}_quickscan_offline_summary.csv"
pd.DataFrame([{
    "tic": target_tic,
    "n_input_files": len(picked_files),
    "n_points": len(t),
    "bls_best_period_days": p_best,
    "bls_png": os.path.basename(bls_png),
    "fold_png_p": os.path.basename(folds[0]),
    "fold_png_halfp": os.path.basename(folds[1]),
    "fold_png_doublep": os.path.basename(folds[2]),
}]).to_csv(out_csv, index=False)

_print("\nQuick scan complete (OFFLINE).")
_print("Saved:")
_print(f" - {bls_png}")
for p in folds: _print(f" - {p}")
_print(f" - {out_csv}")

[13:35:57] Using cached TIC: 2020212050318  (#files=3)
[13:35:59] Stitched points: N=151909 (from 2 file(s))
[13:36:06] 
Quick scan complete (OFFLINE).
[13:36:06] Saved:
[13:36:06]  - figures/TIC2020212050318_quickscan_BLS_offline.png
[13:36:06]  - figures/TIC2020212050318_quickscan_fold_Pbest.png
[13:36:06]  - figures/TIC2020212050318_quickscan_fold_halfP.png
[13:36:06]  - figures/TIC2020212050318_quickscan_fold_doubleP.png
[13:36:06]  - results/TIC2020212050318_quickscan_offline_summary.csv


In [40]:
# %% Target D quick-scan (coords-first cone search, non-blocking downloads)
import os, time, warnings, concurrent.futures as cf
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import lightkurve as lk
from astropy.timeseries import BoxLeastSquares as BLS
from astropy.coordinates import SkyCoord
import astropy.units as u

# Noise + network hygiene
try: lk.log.setLevel("ERROR")
except: pass
try:
    from astroquery.mast import Observations
    Observations.TIMEOUT = 8   # keep MAST calls snappy
except Exception:
    pass

# ======= CONFIG =======
TABLE_PATH       = "results/targets_with_TIC_join.csv"  # needs RA/Dec columns
USED_TICS        = {119584412, 37749396, 311183180}     # skip A–C
MAX_ROWS         = 120
CONE_RADIUS      = "60 arcsec"                          # wide to catch TIC renumbers
AUTHORS          = ("SPOC", "QLP", "TESSCut")
SECTS_PER_AUTH   = {"SPOC": 1, "QLP": 1, "TESSCut": 1}  # 1 sector only (fast probe)
DL_TIMEOUT       = 25                                    # hard cap for single download
DOWNLOAD_DIR     = "net_probe"                           # tiny cache
# BLS knobs
WINDOW_DAYS      = 0.8
BLS_PMIN, BLS_PMAX, BLS_NPER = 0.5, 50.0, 2000
DURATIONS_H      = np.linspace(0.75, 2.0, 9)
# ======================

os.makedirs("figures", exist_ok=True)
os.makedirs("results", exist_ok=True)
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

def _print(*a): print(time.strftime("[%H:%M:%S]"), *a)

def _parse_sector_list(val):
    if isinstance(val, (list, tuple)): return [int(x) for x in val]
    if isinstance(val, str):
        s = val.strip().strip("[]")
        if not s: return []
        out=[]
        for tok in s.split(","):
            tok=tok.strip()
            if tok:
                try: out.append(int(tok))
                except: pass
        return out
    return []

def _row_to_coord(r):
    for ra_key in ["ra","ra_deg","tic_ra","RA","RA_deg"]:
        for de_key in ["dec","dec_deg","tic_dec","Dec","DEC"]:
            if ra_key in r.index and de_key in r.index:
                try:
                    ra = float(r[ra_key]); de = float(r[de_key])
                    if np.isfinite(ra) and np.isfinite(de):
                        return SkyCoord(ra*u.deg, de*u.deg)
                except Exception:
                    pass
    return None

def _discover_by_author(target, author, cap=4, radius=CONE_RADIUS):
    try:
        if author in ("SPOC","QLP"):
            sr = lk.search_lightcurve(target, mission="TESS", author=author, radius=radius)
        else:
            sr = lk.search_tesscut(target)
    except Exception:
        return []
    secs = sorted({getattr(r, "sector", None) for r in sr if getattr(r, "sector", None) is not None})
    return secs[:cap]

def _probe_download(search_result, author):
    """Download exactly 1 file with a hard timeout thread; return LightCurve-like object or None."""
    if len(search_result) == 0:
        return None
    def _do():
        if author in ("SPOC","QLP"):
            obj = search_result[0].download(download_dir=DOWNLOAD_DIR)
            return obj
        else:
            tpf = search_result[0].download(cutout_size=7, download_dir=DOWNLOAD_DIR)
            # minimal LC from TPF
            try:
                ap = tpf.create_threshold_mask(threshold=3)
                if ap.sum()==0: ap=None
            except Exception:
                ap=None
            lc = tpf.to_lightcurve(aperture_mask=ap) if ap is not None else tpf.to_lightcurve()
            return lc
    with cf.ThreadPoolExecutor(max_workers=1) as ex:
        fut = ex.submit(_do)
        try:
            return fut.result(timeout=DL_TIMEOUT)
        except cf.TimeoutError:
            return None
        except Exception:
            return None

def _gentle_flatten(lc, window_days=WINDOW_DAYS):
    try:
        t = lc.time.value
    except Exception:
        return None
    if len(t) < 10: return lc.normalize()
    dt = np.nanmedian(np.diff(np.sort(t)))
    wl = int(max(51, window_days/dt));  wl += (wl % 2 == 0)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        try:
            fl = lc.remove_outliers(sigma=6.0).flatten(window_length=wl, polyorder=2)
        except Exception:
            fl = lc.normalize()
    return fl.remove_nans()

def _run_bls(t, f):
    m = np.isfinite(t) & np.isfinite(f); t, f = t[m], f[m]
    if len(t) < 300: return None
    span = t.max() - t.min()
    pmax_eff = max(1.01, min(BLS_PMAX, span/2.0))
    periods = np.exp(np.linspace(np.log(BLS_PMIN), np.log(pmax_eff), BLS_NPER))
    model = BLS(t, f); res = model.power(periods, DURATIONS_H/24.0)
    return dict(periods=periods, power=res.power, best=periods[np.argmax(res.power)])

def _plot_bls(periods, power, out_png, title):
    plt.figure(figsize=(7.2,3.8)); plt.plot(periods, power, lw=1)
    plt.xlabel("Period [days]"); plt.ylabel("BLS power"); plt.title(title); plt.grid(alpha=0.3)
    plt.tight_layout(); plt.savefig(out_png, dpi=160); plt.close()

# --- load table and rank rows: has coords first, then more hinted sectors (if any)
if not os.path.exists(TABLE_PATH):
    raise SystemExit(f"Missing table: {TABLE_PATH} (needs RA/Dec columns).")

df = pd.read_csv(TABLE_PATH)
if not len(df):
    raise SystemExit(f"Table is empty: {TABLE_PATH}")

cols = {c.lower(): c for c in df.columns}
tic_col  = cols.get("tic") or cols.get("tic_id") or cols.get("ticid") or cols.get("tic_id_norm")
name_col = cols.get("name") or cols.get("toi") or cols.get("target") or cols.get("designation")
sect_col = cols.get("sectors_now") or cols.get("sectors") or cols.get("observed_sectors") or cols.get("sector_list")

df["__tic"] = pd.to_numeric(df[tic_col], errors="coerce").astype("Int64") if tic_col else pd.Series([pd.NA]*len(df))
pool = df.dropna(subset=["__tic"]) if tic_col else df.copy()
pool = pool[~pool["__tic"].isin(USED_TICS)] if tic_col else pool
pool = pool.reset_index(drop=True)

has_coord = []
for _, r in pool.iterrows():
    has_coord.append(_row_to_coord(r) is not None)
pool["__has_coord"] = has_coord
if sect_col:
    pool["__nsec_hint"] = pool[sect_col].map(_parse_sector_list).map(len).fillna(0)
else:
    pool["__nsec_hint"] = 0

pool = pool.sort_values(by=["__has_coord","__nsec_hint"], ascending=[False, False]).reset_index(drop=True)

# --- scan until one succeeds ---
_print(f"Source table: {TABLE_PATH}")
picked = None

for i, r in pool.head(MAX_ROWS).iterrows():
    coord = _row_to_coord(r)
    if coord is None:
        continue  # coords-only strategy to avoid name/TIC resolver pitfalls
    name  = str(r[name_col]) if name_col in r.index else "Target D"
    tic   = int(r["__tic"]) if r["__tic"] is not pd.NA else None

    _print(f"Trying {name} — TIC {tic if tic else '—'}  @ RA={coord.ra.deg:.6f}, Dec={coord.dec.deg:.6f}")
    lcs, used_auth, used_sect = None, None, None

    for auth in AUTHORS:
        secs = _discover_by_author(coord, author=auth, cap=4, radius=CONE_RADIUS)
        _print(f"  {auth:<7s}sectors: {secs if secs else 'none'}")
        if not secs:
            continue
        # fetch exactly one sector quickly
        if auth in ("SPOC","QLP"):
            sr = lk.search_lightcurve(coord, mission="TESS", author=auth, sector=secs[0], radius=CONE_RADIUS)
        else:
            sr = lk.search_tesscut(coord, sector=secs[0])
        obj = _probe_download(sr, auth)
        if obj is None:
            continue
        # Convert to LightCurve if needed
        try:
            lc = obj.remove_nans()
        except Exception:
            try:
                lc = obj.to_lightcurve().remove_nans()
            except Exception:
                continue
        if len(getattr(lc, "time", [])) < 300:
            continue
        lcs, used_auth, used_sect = lc, auth, secs[0]
        break

    if lcs is not None:
        picked = dict(name=name, tic=tic, coord=coord, author=used_auth, sector=used_sect, lc=lcs)
        break

if picked is None:
    raise SystemExit("Quick scan aborted: no usable data via 60″ cone for top rows.\n"
                     "Options: increase MAX_ROWS, widen CONE_RADIUS (e.g., 120 arcsec), "
                     "or switch to a known-bright seed to verify connectivity.")

# --- detrend + BLS on the single sector we landed ---
fl = _gentle_flatten(picked["lc"], WINDOW_DAYS)
if fl is None or len(fl.time.value) < 300:
    raise SystemExit("Landed a file but failed to flatten/use it (too few points).")

t, f = fl.time.value, fl.flux.value
_print(f"Stitched points: N={len(t)} (author={picked['author']}, sector={picked['sector']})")
_print("Running BLS (wide)…")
bls = _run_bls(t, f)
if bls is None:
    raise SystemExit("BLS failed (not enough points after cleaning).")

p_best = float(bls["best"])
png = f"figures/TargetD_BLS_online.png"
ttl = f"{picked['name']} — {picked['author']} S{picked['sector']} — BLS (wide)"
_plot_bls(bls["periods"], bls["power"], png, ttl)

summary = pd.DataFrame([{
    "name": picked["name"],
    "tic": picked["tic"],
    "author": picked["author"],
    "sector": picked["sector"],
    "ra_deg": picked["coord"].ra.deg,
    "dec_deg": picked["coord"].dec.deg,
    "N_points": len(t),
    "bls_best_period_days": p_best,
}])
csv = "results/TargetD_quickscan_online_summary.csv"
summary.to_csv(csv, index=False)

_print("\nQuick scan complete (ONLINE).")
_print("Saved:\n -", png, "\n -", csv)

[19:07:07] Source table: results/targets_with_TIC_join.csv
[19:07:07] Trying 1487.01 — TIC 459978312  @ RA=255.565616, Dec=64.600740
[19:07:07]   SPOC   sectors: none
[19:07:07]   QLP    sectors: none
[19:09:09]   TESSCutsectors: none
[19:09:09] Trying 5789.01 — TIC 87216634  @ RA=302.775307, Dec=16.187998
[19:09:09]   SPOC   sectors: none
[19:09:09]   QLP    sectors: none
[19:09:09]   TESSCutsectors: none
[19:09:09] Trying 186.01 — TIC 279741379  @ RA=51.746762, Dec=-63.499101
[19:09:09]   SPOC   sectors: none
[19:09:09]   QLP    sectors: none
[19:09:09]   TESSCutsectors: none
[19:09:09] Trying 2009.01 — TIC 243187830  @ RA=16.907810, Dec=22.954980
[19:09:09]   SPOC   sectors: none
[19:09:09]   QLP    sectors: none
[19:09:09]   TESSCutsectors: none
[19:09:09] Trying 197.01 — TIC 441462736  @ RA=353.033635, Dec=-21.801421
[19:09:09]   SPOC   sectors: none
[19:09:09]   QLP    sectors: none
[19:09:09]   TESSCutsectors: none
[19:09:09] Trying 2194.01 — TIC 271478281  @ RA=299.154276, Dec=

SystemExit: Quick scan aborted: no usable data via 60″ cone for top rows.
Options: increase MAX_ROWS, widen CONE_RADIUS (e.g., 120 arcsec), or switch to a known-bright seed to verify connectivity.